<h1><b><i>SCRIPT PRINCIPAL</b></i></h1>
<h2><b><i>Importação de Bibliotécas</b></i></h2>

In [ ]:
import pandas as pd
import openpyxl
import numpy as np
import os


In [ ]:
# ==============================================================================
# 1. SETUP INICIAL E CARREGAMENTO
#    (Melhor prática: definir constantes e carregar o arquivo de forma segura)
# ==============================================================================

# Definição do caminho do arquivo em uma variável para fácil manutenção
# Baseado na sua estrutura de diretórios
FILE_PATH = r'C:\Users\carlos.ramos\OneDrive - Presidência da República\Área de Trabalho\Acompanhamento-Concessoes-Rodoviarias-main-\Acompanhamento-Concessoes-Rodoviarias-main\Pandas\Notebooks\RODOVIÁRIO\RISCOS\Planilha.xlsx'
OUTPUT_DIR = 'Dados Gerados' 

# Verifica se o arquivo existe antes de tentar carregá-lo
if not os.path.exists(FILE_PATH):
    print(f"ERRO: O arquivo '{FILE_PATH}' não foi encontrado.")
    # Encerra o script ou lança uma exceção se o arquivo principal não existir
    exit()

# Cria a pasta de saída se ela não existir
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Arquivo '{FILE_PATH}' encontrado. Iniciando processamento...")

## 📋 Extração de Campos de Licença das Metas

Esta seção contém funções para extrair os seguintes campos que aparecem em todas as metas de todos os tipos de projeto (PAVIMENTAÇÃO, DUPLICAÇÃO, OAE, CONTORNO, FX ADICIONAL, TERCEIRA FAIXA):

- **Nº PROCESSO**
- **DATA DO PROTOCOLO**
- **LICENÇA**
- **SITUAÇÃO GERAL** (caso o processo já tenha sido protocolado)
- **Data do último documento encaminhado (D/M/A)**
- **Foi solicitado licença para o segmento?**
- **O pedido de licença desse segmento está dentro de um pedido de trecho maior?**
- **Se positivo, especificar o trecho completo requisitado - km(i) - km(f)**
- **Observações gerais** (Colocar aqui observações gerais; Quando escolher a opção Outros, descrever a situação de forma suscinta)

### Como usar:

1. **Carregar a planilha "Planilha"** (se existir): A célula abaixo tenta carregar automaticamente
2. **Extrair campos de uma meta específica**: Use `extrair_campos_licenca_da_meta('META(2024)')`
3. **Adicionar campos a uma meta processada**: Use `adicionar_campos_licenca_a_meta(df_meta_24_copy, 'META(2024)')`


In [ ]:
# Listar todas as planilhas disponíveis no arquivo
xls = pd.ExcelFile(FILE_PATH)
sheet_names = xls.sheet_names
sheet_names

In [ ]:
# Carregar a planilha "Planilha" se existir
try:
    df_planilha = pd.read_excel(FILE_PATH, sheet_name='Planilha')
    print("✅ Planilha 'Planilha' carregada com sucesso!")
    print(f"Shape: {df_planilha.shape}")
    print(f"Colunas: {list(df_planilha.columns[:10])}")  # Primeiras 10 colunas
except Exception as e:
    print(f"⚠️ Aviso: Não foi possível carregar a planilha 'Planilha': {e}")
    print("Tentando verificar se existe com nome diferente...")
    # Listar todas as planilhas para verificar
    xls = pd.ExcelFile(FILE_PATH)
    print(f"Planilhas disponíveis: {xls.sheet_names}")
    df_planilha = None


In [ ]:
# Função para extrair campos comuns de todas as metas
def extrair_campos_licenca_meta(df_meta, tipo_projeto=''):
    """
    Extrai os campos relacionados a licenças e processos que aparecem em todas as metas.
    
    Parâmetros:
    - df_meta: DataFrame da meta
    - tipo_projeto: Tipo do projeto (PAVIMENTAÇÃO, DUPLICAÇÃO, OAE, etc.)
    
    Retorna:
    - DataFrame com os campos extraídos
    """
    campos_licenca = {}
    
    # Lista de campos a serem extraídos (com variações possíveis de nomes)
    campos_procurados = {
        'Nº PROCESSO': ['Nº PROCESSO', 'N PROCESSO', 'NUMERO PROCESSO', 'PROCESSO', 'Nº PROC'],
        'DATA DO PROTOCOLO': ['DATA DO PROTOCOLO', 'DATA PROTOCOLO', 'PROTOCOLO', 'DT PROTOCOLO'],
        'LICENÇA': ['LICENÇA', 'LICENCA', 'LICENÇAS', 'LICENCAS'],
        'SITUAÇÃO GERAL': ['SITUAÇÃO GERAL', 'SITUACAO GERAL', 'SITUAÇÃO', 'SITUACAO'],
        'DATA ÚLTIMO DOCUMENTO': ['Data do último documento encaminhado', 'DATA ÚLTIMO DOC', 'ULTIMO DOCUMENTO', 
                                  'DATA DOCUMENTO', 'Data do último documento'],
        'LICENÇA SOLICITADA': ['Foi solicitado licença para o segmento?', 'LICENÇA SOLICITADA', 
                               'LICENCA SOLICITADA', 'SOLICITADO LICENÇA'],
        'TRECHO MAIOR': ['O pedido de licença desse segmento está dentro de um pedido de trecho maior?',
                        'TRECHO MAIOR', 'PEDIDO TRECHO MAIOR'],
        'TRECHO COMPLETO': ['Se positivo, especificar o trecho completo requisitado', 
                           'TRECHO COMPLETO', 'KM INICIAL FINAL', 'km(i) - km(f)'],
        'OBSERVAÇÕES': ['Colocar aqui observações gerais', 'OBSERVAÇÕES', 'OBSERVACOES', 
                       'OBSERVAÇÕES GERAIS', 'OBSERVACOES GERAIS']
    }
    
    # Procurar pelos campos nas colunas do DataFrame
    for campo_principal, variacoes in campos_procurados.items():
        encontrado = False
        for variacao in variacoes:
            # Procurar coluna que contenha a variação (case insensitive)
            colunas_encontradas = [col for col in df_meta.columns 
                                  if variacao.upper() in str(col).upper()]
            if colunas_encontradas:
                campos_licenca[campo_principal] = df_meta[colunas_encontradas[0]].copy()
                encontrado = True
                break
        
        if not encontrado:
            # Se não encontrou, criar coluna vazia
            campos_licenca[campo_principal] = pd.Series([None] * len(df_meta), index=df_meta.index)
    
    # Criar DataFrame com os campos extraídos
    df_campos = pd.DataFrame(campos_licenca, index=df_meta.index)
    
    # Adicionar prefixo do tipo de projeto se fornecido
    if tipo_projeto:
        df_campos.columns = [f'{tipo_projeto} - {col}' for col in df_campos.columns]
    
    return df_campos

print("✅ Função extrair_campos_licenca_meta criada com sucesso!")


In [ ]:
# Processar campos de licença para todas as metas
# Esta função será usada para extrair os campos comuns de todas as metas

def processar_campos_licenca_todas_metas():
    """
    Processa os campos de licença para todas as metas carregadas.
    Retorna um dicionário com os campos extraídos por meta e tipo de projeto.
    """
    tipos_projeto = ['PAVIMENTAÇÃO', 'DUPLICAÇÃO', 'OAE', 'CONTORNO', 'FX ADICIONAL', 'TERCEIRA FAIXA']
    metas = {
        'META_2023': df_meta_23 if 'df_meta_23' in globals() else None,
        'META_2024': df_meta_24 if 'df_meta_24' in globals() else None,
        'META_2025': df_meta_25 if 'df_meta_25' in globals() else None,
        'META_2026': df_meta_26 if 'df_meta_26' in globals() else None,
        'META_2027': df_meta_27 if 'df_meta_27' in globals() else None,
    }
    
    campos_extraidos = {}
    
    for nome_meta, df_meta in metas.items():
        if df_meta is not None:
            print(f"\n📋 Processando {nome_meta}...")
            campos_extraidos[nome_meta] = {}
            
            # Ler a meta com header=2 para obter a estrutura correta
            try:
                df_meta_temp = pd.read_excel(FILE_PATH, sheet_name=df_meta.name if hasattr(df_meta, 'name') else nome_meta.replace('META_', 'META(').replace('_', ''), header=2)
                
                # Para cada tipo de projeto, tentar extrair os campos
                for tipo in tipos_projeto:
                    # Procurar seções relacionadas ao tipo de projeto
                    # Os campos podem estar em colunas que contêm o nome do tipo de projeto
                    colunas_tipo = [col for col in df_meta_temp.columns if tipo.upper() in str(col).upper()]
                    
                    if colunas_tipo:
                        print(f"  ✅ Encontradas {len(colunas_tipo)} colunas para {tipo}")
                        # Extrair campos usando a função
                        campos = extrair_campos_licenca_meta(df_meta_temp, tipo_projeto=tipo)
                        campos_extraidos[nome_meta][tipo] = campos
                    else:
                        print(f"  ⚠️ Nenhuma coluna encontrada para {tipo}")
                        
            except Exception as e:
                print(f"  ❌ Erro ao processar {nome_meta}: {e}")
        else:
            print(f"⚠️ {nome_meta} não está disponível")
    
    return campos_extraidos

# Executar o processamento (comentado para não executar automaticamente)
# campos_licenca_metas = processar_campos_licenca_todas_metas()
print("✅ Função processar_campos_licenca_todas_metas criada!")
print("💡 Use 'campos_licenca_metas = processar_campos_licenca_todas_metas()' para executar")


In [ ]:
# Extrair campos de licença diretamente das planilhas de META
# Esta célula lê as planilhas de META e extrai os campos solicitados

def extrair_campos_licenca_da_meta(sheet_name, tipos_projeto=['PAVIMENTAÇÃO', 'DUPLICAÇÃO', 'OAE', 'CONTORNO', 'FX ADICIONAL', 'TERCEIRA FAIXA']):
    """
    Extrai os campos de licença diretamente de uma planilha de META.
    
    Parâmetros:
    - sheet_name: Nome da planilha (ex: 'META(2024)')
    - tipos_projeto: Lista dos tipos de projeto a processar
    
    Retorna:
    - DataFrame com todos os campos extraídos
    """
    try:
        # Ler a planilha sem header para ver todas as linhas
        df_raw = pd.read_excel(FILE_PATH, sheet_name=sheet_name, header=None)
        
        # Procurar pelos campos nas primeiras linhas (cabeçalhos)
        campos_encontrados = {}
        
        # Lista de campos a procurar
        campos_procurados = [
            'Nº PROCESSO',
            'DATA DO PROTOCOLO', 
            'LICENÇA',
            'SITUAÇÃO GERAL',
            'Data do último documento encaminhado',
            'Foi solicitado licença para o segmento?',
            'O pedido de licença desse segmento está dentro de um pedido de trecho maior?',
            'Se positivo, especificar o trecho completo requisitado',
            'Colocar aqui observações gerais'
        ]
        
        # Procurar nas primeiras 5 linhas pelos cabeçalhos
        for linha_idx in range(min(5, len(df_raw))):
            linha = df_raw.iloc[linha_idx]
            for campo in campos_procurados:
                for col_idx, valor in enumerate(linha):
                    if pd.notna(valor) and campo.upper() in str(valor).upper():
                        if campo not in campos_encontrados:
                            campos_encontrados[campo] = col_idx
                            print(f"✅ Encontrado '{campo}' na coluna {col_idx} (linha {linha_idx})")
        
        # Se encontrou campos, ler novamente com header apropriado
        if campos_encontrados:
            # Tentar ler com header=2 (padrão usado no notebook)
            df_meta = pd.read_excel(FILE_PATH, sheet_name=sheet_name, header=2)
            
            # Criar DataFrame com os campos encontrados
            df_campos = pd.DataFrame()
            for campo, col_idx in campos_encontrados.items():
                if col_idx < len(df_meta.columns):
                    col_name = df_meta.columns[col_idx]
                    df_campos[campo] = df_meta[col_name]
            
            return df_campos
        else:
            print(f"⚠️ Nenhum campo encontrado na planilha {sheet_name}")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"❌ Erro ao processar {sheet_name}: {e}")
        return pd.DataFrame()

# Exemplo de uso (comentado):
# df_campos_meta_24 = extrair_campos_licenca_da_meta('META(2024)')
# print(f"\nCampos extraídos: {list(df_campos_meta_24.columns)}")

print("✅ Função extrair_campos_licenca_da_meta criada!")
print("💡 Use 'df_campos = extrair_campos_licenca_da_meta(\"META(2024)\")' para extrair campos de uma meta específica")


In [ ]:
# ==============================================================================
# EXEMPLO PRÁTICO: Extrair e adicionar campos de licença às metas
# ==============================================================================

# Descomente as linhas abaixo para testar a extração de campos

# Exemplo 1: Extrair campos da META 2024
# df_campos_meta_24 = extrair_campos_licenca_da_meta('META(2024)')
# print("\n" + "="*60)
# print("Campos extraídos da META 2024:")
# print("="*60)
# if not df_campos_meta_24.empty:
#     print(f"Colunas: {list(df_campos_meta_24.columns)}")
#     print(f"Shape: {df_campos_meta_24.shape}")
#     print("\nPrimeiras linhas:")
#     print(df_campos_meta_24.head())
# else:
#     print("Nenhum campo encontrado")

# Exemplo 2: Extrair campos de todas as metas
# metas_para_processar = ['META(1) 2023', 'META(2024)', 'META(2025)', 'META(2026)', 'META(2027)']
# campos_todas_metas = {}
# for meta in metas_para_processar:
#     print(f"\n{'='*60}")
#     print(f"Processando {meta}...")
#     print('='*60)
#     campos_todas_metas[meta] = extrair_campos_licenca_da_meta(meta)
#     if not campos_todas_metas[meta].empty:
#         print(f"✅ {len(campos_todas_metas[meta].columns)} campos extraídos")
#     else:
#         print("⚠️ Nenhum campo encontrado")

# Exemplo 3: Adicionar campos a uma meta já processada (após o processamento normal)
# Após processar df_meta_24_copy normalmente, você pode fazer:
# df_meta_24_copy_com_campos = adicionar_campos_licenca_a_meta(df_meta_24_copy, 'META(2024)')

print("💡 Descomente as linhas acima para testar a extração de campos")


In [ ]:
# Função para adicionar campos de licença às metas processadas
def adicionar_campos_licenca_a_meta(df_meta_processado, sheet_name_original):
    """
    Adiciona os campos de licença extraídos à meta já processada.
    
    Parâmetros:
    - df_meta_processado: DataFrame da meta já processada
    - sheet_name_original: Nome da planilha original (ex: 'META(2024)')
    
    Retorna:
    - DataFrame com os campos de licença adicionados
    """
    # Extrair os campos da planilha original
    df_campos = extrair_campos_licenca_da_meta(sheet_name_original)
    
    if df_campos.empty:
        print(f"⚠️ Nenhum campo de licença encontrado para {sheet_name_original}")
        return df_meta_processado
    
    # Garantir que os índices sejam compatíveis
    if len(df_campos) != len(df_meta_processado):
        print(f"⚠️ Tamanhos diferentes: campos={len(df_campos)}, meta={len(df_meta_processado)}")
        # Tentar alinhar pelos índices
        df_campos = df_campos.reindex(df_meta_processado.index, fill_value=None)
    
    # Adicionar os campos ao DataFrame processado
    df_resultado = pd.concat([df_meta_processado, df_campos], axis=1)
    
    print(f"✅ {len(df_campos.columns)} campos de licença adicionados à meta {sheet_name_original}")
    print(f"   Campos adicionados: {list(df_campos.columns)}")
    
    return df_resultado

# Exemplo de uso para META 2024 (comentado):
# df_meta_24_com_campos = adicionar_campos_licenca_a_meta(df_meta_24_copy, 'META(2024)')

print("✅ Função adicionar_campos_licenca_a_meta criada!")
print("💡 Use esta função após processar cada meta para adicionar os campos de licença")


In [ ]:
file_path = 'Planilha.xlsx'

In [ ]:
# Carregar as planilhas
df_informacoes = pd.read_excel(FILE_PATH, sheet_name='INFORMAÇÕES (1)')
df_per = pd.read_excel(FILE_PATH, sheet_name='PER (1)')
df_plan_exec_ano_ant = pd.read_excel(FILE_PATH, sheet_name='PLAN_EXEC ATÉ ANO ANTERIOR (22)')
df_meta_23 = pd.read_excel(FILE_PATH, sheet_name='META(1) 2023')
df_meta_24 = pd.read_excel(FILE_PATH, sheet_name='META(2024)')
df_meta_25 = pd.read_excel(FILE_PATH, sheet_name='META(2025)')
df_meta_26 = pd.read_excel(FILE_PATH, sheet_name='META(2026)')
df_meta_27 = pd.read_excel(FILE_PATH, sheet_name='META(2027)')
df_inexec = pd.read_excel(FILE_PATH, sheet_name='iNEXECUTADO (pendências até 24)')
df_a_executar = pd.read_excel(FILE_PATH, sheet_name='À EXECUTAR (2026 DIANTE)')

<h1><b><i>INFORMAÇÕES (1)</b></i></h1>

In [ ]:
#CHECKPOINT
df_informacoes_copy = df_informacoes.copy()

In [ ]:
df_informacoes_copy = pd.read_excel(FILE_PATH, sheet_name='INFORMAÇÕES (1)', header=2)
df_informacoes_copy.columns

In [ ]:
df_informacoes_copy

In [ ]:
df_informacoes_copy = df_informacoes_copy.drop([0, 1, 2], axis = 0)

In [ ]:
df_informacoes_copy = df_informacoes_copy.dropna(subset=['BR'])

In [ ]:
df_informacoes_copy['Etapa'] = None
df_informacoes_copy['Descrição do Trecho'] = None
etapa_atual = None

for index, row in df_informacoes_copy.iterrows():
    if 'ETAPA' in str(row['TRECHO']):
        etapa_atual = row['TRECHO']
    df_informacoes_copy.at[index, 'Etapa'] = etapa_atual
    if pd.notna(row['TRECHO']) and 'ETAPA' not in str(row['TRECHO']):
        df_informacoes_copy.at[index, 'Descrição do Trecho'] = row['TRECHO']

In [ ]:
df_informacoes_copy.drop('TRECHO', axis=1, inplace=True)

In [ ]:
# Updating the 'Região' column based on the 'UF' mapping
region_mapping = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste',
    'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MS': 'Centro-Oeste', 'MT': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'SC': 'Sul', 'RS': 'Sul'
}
df_informacoes_copy['Região'] = df_informacoes_copy['UF'].map(region_mapping)

In [ ]:
columns_to_fill = ['ESTADO/LOTE', 'EMPREENDIMENTO', 'PROPONENTE', 'EXECUTOR             (Grupo Controlador)',
       'ESTRUTURADOR DO PROJETO', 'ANO LEILÃO', 'DATA DE INÍCIO']
df_informacoes_copy[columns_to_fill] = df_informacoes_copy.groupby((df_informacoes_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [ ]:
df_informacoes_copy['CAPEX (BI)'].unique()

In [ ]:
df_informacoes_copy

In [ ]:
df_informacoes_copy['Situação'] = df_informacoes_copy['KM Inicial'].apply(lambda x: x if str(x) in ['ATIVO', 'CADUCIDADE', 'JUDICIAL', 'RELICITAÇÃO', 'RENEGOCIAÇÃO'] else pd.NA)

# Agora preenchemos para baixo até encontrar o próximo valor não-NaN (isto é, a próxima 'Situação')
df_informacoes_copy['Situação'] = df_informacoes_copy['Situação'].fillna(method='ffill')

# Agora, separemos os valores numéricos para uma nova coluna 'KM Inicial Num'
df_informacoes_copy['km (i)'] = df_informacoes_copy['KM Inicial'].apply(lambda x: x if isinstance(x, float) else pd.NA)

In [ ]:
df_informacoes_copy.drop('KM Inicial', axis=1, inplace=True)

In [ ]:
# Renomear Coluna ['EXECUTOR...']
# DROP de Colunas e Realocação.

df_informacoes_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR (Grupo Controlador)'}, inplace=True)
df_informacoes_copy.rename(columns={'EXTENSÃO\n (km)': 'EXTENSÃO (km)'}, inplace=True)
df_informacoes_copy.rename(columns={'KM Final': 'km (f)'}, inplace=True)

df_informacoes_copy = df_informacoes_copy[['Região', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO',
       'ANO LEILÃO', 'DATA DE INÍCIO', 'ANO DA CONCESSÃO',
       'PRAZO (anos)', 'CAPEX (BI)', 'OPEX (BI)', 'INVESTIMENTO TOTAL (BI)', 
       'Etapa', 'Descrição do Trecho',
       'Situação', 'km (i)', 'km (f)', 'EXTENSÃO (km)']]

In [ ]:
df_informacoes_copy['ESTRUTURADOR DO PROJETO'].unique()

In [ ]:
df_informacoes_copy['PROPONENTE'].unique()

In [ ]:
df_informacoes_copy.columns

In [ ]:
# Converter colunas para 'category'
df_informacoes_copy['SETOR'] = df_informacoes_copy['SETOR'].astype('category')
df_informacoes_copy['UF'] = df_informacoes_copy['UF'].astype('category')
df_informacoes_copy['ESTADO/LOTE'] = df_informacoes_copy['ESTADO/LOTE'].astype('category')
df_informacoes_copy['PROPONENTE'] = df_informacoes_copy['PROPONENTE'].astype('category')
df_informacoes_copy['EXECUTOR (Grupo Controlador)'] = df_informacoes_copy['EXECUTOR (Grupo Controlador)'].astype('category')
df_informacoes_copy['ESTRUTURADOR DO PROJETO'] = df_informacoes_copy['ESTRUTURADOR DO PROJETO'].astype('category')

# Converter colunas para 'int'
df_informacoes_copy['ANO DA CONCESSÃO'] = pd.to_numeric(df_informacoes_copy['ANO DA CONCESSÃO'], errors='coerce').fillna(0).astype(int)
df_informacoes_copy['PRAZO (anos)'] = pd.to_numeric(df_informacoes_copy['PRAZO (anos)'], errors='coerce').fillna(0).astype(int)

# Converter colunas para 'float'
df_informacoes_copy['CAPEX (BI)'] = pd.to_numeric(df_informacoes_copy['CAPEX (BI)'], errors='coerce')
df_informacoes_copy['OPEX (BI)'] = pd.to_numeric(df_informacoes_copy['OPEX (BI)'], errors='coerce')
df_informacoes_copy['INVESTIMENTO TOTAL (BI)'] = df_informacoes_copy['INVESTIMENTO TOTAL (BI)'].replace('[\$]', '', regex=True).replace(',', '.', regex=True)
df_informacoes_copy['INVESTIMENTO TOTAL (BI)'] = pd.to_numeric(df_informacoes_copy['INVESTIMENTO TOTAL (BI)'], errors='coerce')

df_informacoes_copy['km (i)'] = pd.to_numeric(df_informacoes_copy['km (i)'], errors='coerce')
df_informacoes_copy['km (f)'] = pd.to_numeric(df_informacoes_copy['km (f)'], errors='coerce')
df_informacoes_copy['EXTENSÃO (km)'] = pd.to_numeric(df_informacoes_copy['EXTENSÃO (km)'], errors='coerce')

# Converter colunas para 'string'
df_informacoes_copy['EMPREENDIMENTO'] = df_informacoes_copy['EMPREENDIMENTO'].astype('string') 


In [ ]:
# Função para limpar e converter as datas para o formato datetime
def limpar_e_converter_data(data):
    # Tratar dados como string para evitar problemas com diferentes tipos
    data = str(data)
    # Substituir '//'' por '/'
    data = data.replace('//', '/')
    # Substituir '/' por '-'
    data = data.replace('/', '-')
    # Tentar converter para datetime
    return pd.to_datetime(data, errors='coerce', dayfirst=True)

# Aplicar a função no DataFrame
# Substitua 'df_informacoes_copy' pelo nome do seu DataFrame e 'DATA DE INÍCIO' pelo nome da coluna que contém as datas
df_informacoes_copy['DATA DE INÍCIO'] = df_informacoes_copy['DATA DE INÍCIO'].apply(limpar_e_converter_data)

In [ ]:
columns_to_fill = ['Região', 'ESTADO/LOTE', 'EMPREENDIMENTO', 'EXECUTOR (Grupo Controlador)']
df_informacoes_copy[columns_to_fill] = df_informacoes_copy.groupby((df_informacoes_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [ ]:
df_informacoes_copy.index = range(1, len(df_informacoes_copy) + 1)

In [ ]:
df_informacoes_copy.index.name = 'ID-ÚNICO'

In [ ]:
df_informacoes_copy

In [ ]:
df_informacoes_copy['UF'].unique()

---

In [ ]:
df_informacoes_copy = df_informacoes_copy[df_informacoes_copy['BR'] == 1]

In [ ]:
columns_to_drop = ['SETOR', 'Região', 'Descrição do Trecho', 'km (i)', 'km (f)']

# Removendo as colunas extras do DataFrame df_informacoes_copy
df_informacoes_copy = df_informacoes_copy.drop(columns=columns_to_drop)

In [ ]:
# Dicionário de renomeação
rename_dict = {
    'UF': 'Região',
    'Etapa': 'ETAPA',
    'Situação': 'SITUAÇÃO'
}

# Renomear as colunas usando o dicionário
df_informacoes_copy = df_informacoes_copy.rename(columns=rename_dict)

In [ ]:
df_informacoes_copy = df_informacoes_copy[['Região', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO',
       'ANO LEILÃO', 'DATA DE INÍCIO', 'ANO DA CONCESSÃO',
       'PRAZO (anos)', 'CAPEX (BI)', 'OPEX (BI)', 'INVESTIMENTO TOTAL (BI)',
       'ETAPA', 'SITUAÇÃO', 'EXTENSÃO (km)', 'ESTADO/LOTE']]

In [ ]:
# Realiza a substituição na coluna 'EMPREENDIMENTO'
df_informacoes_copy['EMPREENDIMENTO'] = df_informacoes_copy['EMPREENDIMENTO'].replace('CONCESSÃO VIA SUL', 'VIA SUL')

In [ ]:
df_informacoes_copy.to_excel('Dados Gerados/INFORMAÇÕES (1).xlsx', index=False, sheet_name='INFORMAÇÕES (1)')

<h1><b><i>PER (1)</b></i></h1>

In [ ]:
#CHECKPOINT
df_per_copy = df_per.copy()

In [ ]:
df_per_copy = pd.read_excel(FILE_PATH, sheet_name='PER (1)', header=2)

In [ ]:
df_per_copy

In [ ]:
df_per_copy = df_per_copy.drop([0, 1, 2], axis = 0)

In [ ]:
df_per_copy = df_per_copy.loc[df_per_copy['BR'] != 1]
df_per_copy = df_per_copy.dropna(subset=['BR'])

In [ ]:
df_per_copy = df_per_copy.dropna(subset=['ESTADO/LOTE'])

In [ ]:
df_per_copy['ESTADO/LOTE'] = df_per_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS',
       'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO',
       'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA',
       'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)

# Agora preenchemos para baixo até encontrar o próximo valor não-NaN (isto é, a próxima 'Situação')
df_per_copy['ESTADO/LOTE'] = df_per_copy['ESTADO/LOTE'].fillna(method='ffill')

In [ ]:
df_per_copy

In [ ]:
df_per_copy = df_per_copy[~df_per_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]

In [ ]:
# Renomear Coluna ['EXECUTOR...']

df_per_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR (Grupo Controlador)'}, inplace=True)
df_per_copy.rename(columns={'PAVIMENTAÇÃO': 'PAVIMENTAÇÃO - < 2022 - Descrição'}, inplace=True)
df_per_copy.rename(columns={'DUPLICAÇÃO': 'DUPLICAÇÃO - < 2022 - Descrição'}, inplace=True)
df_per_copy.rename(columns={'OAE': 'OAE - < 2022 - Descrição'}, inplace=True)
df_per_copy.rename(columns={'CONTORNO': 'CONTORNO - < 2022 - Descrição'}, inplace=True)
df_per_copy.rename(columns={'FX ADICIONAL': 'FX ADICIONAL - < 2022 - Descrição'}, inplace=True)
df_per_copy.rename(columns={'TERCEIRA FAIXA': 'TERCEIRA FAIXA - < 2022 - Descrição'}, inplace=True)

In [ ]:
df_per_copy

In [ ]:
new_column_names = ['PAVIMENTAÇÃO - < 2022 - km (i)', 
                    'PAVIMENTAÇÃO - < 2022 - km (f)', 
                    'PAVIMENTAÇÃO - < 2022 - Ext. (km)', 
                    'PAVIMENTAÇÃO - < 2022 - PERCENTUAL (%)', 
                    'PAVIMENTAÇÃO - < 2022 - (km)%', 
                    'PAVIMENTAÇÃO - < 2022 - FINANCEIRO (R$)', 'PAVIMENTAÇÃO - 2023 - Descrição', 
                    'PAVIMENTAÇÃO - 2023 - km (i)', 'PAVIMENTAÇÃO - 2023 - km (f)', 'PAVIMENTAÇÃO - 2023 - Ext. (km)', 
                    'PAVIMENTAÇÃO - 2023 - PERCENTUAL (%)', 'PAVIMENTAÇÃO - 2023 - (km)%', 'PAVIMENTAÇÃO - 2023 - FINANCEIRO (R$)', 
                    'PAVIMENTAÇÃO - 2024 - Descrição', 'PAVIMENTAÇÃO - 2024 - km (i)', 'PAVIMENTAÇÃO - 2024 - km (f)', 
                    'PAVIMENTAÇÃO - 2024 - Ext. (km)', 'PAVIMENTAÇÃO - 2024 - PERCENTUAL (%)', 'PAVIMENTAÇÃO - 2024 - (km)%', 
                    'PAVIMENTAÇÃO - 2024 - FINANCEIRO (R$)', 'PAVIMENTAÇÃO - 2025 - Descrição', 'PAVIMENTAÇÃO - 2025 - km (i)', 
                    'PAVIMENTAÇÃO - 2025 - km (f)', 'PAVIMENTAÇÃO - 2025 - Ext. (km)', 'PAVIMENTAÇÃO - 2025 - PERCENTUAL (%)', 
                    'PAVIMENTAÇÃO - 2025 - (km)%', 'PAVIMENTAÇÃO - 2025 - FINANCEIRO (R$)', 'PAVIMENTAÇÃO - 2026 - Descrição', 
                    'PAVIMENTAÇÃO - 2026 - km (i)', 'PAVIMENTAÇÃO - 2026 - km (f)', 'PAVIMENTAÇÃO - 2026 - Ext. (km)', 
                    'PAVIMENTAÇÃO - 2026 - PERCENTUAL (%)', 'PAVIMENTAÇÃO - 2026 - (km)%', 'PAVIMENTAÇÃO - 2026 - FINANCEIRO (R$)', 
                    'PAVIMENTAÇÃO - Pós 2026 - Descrição', 'PAVIMENTAÇÃO - Pós 2026 - km (i)', 'PAVIMENTAÇÃO - Pós 2026 - km (f)', 
                    'PAVIMENTAÇÃO - Pós 2026 - Ext. (km)', 'PAVIMENTAÇÃO - Pós 2026 - PERCENTUAL (%)', 'PAVIMENTAÇÃO - Pós 2026 - (km)%', 
                    'PAVIMENTAÇÃO - Pós 2026 - FINANCEIRO (R$)', 'PAVIMENTAÇÃO - Pós 2026 - REL. FÍSICO (km)', 
                    'PAVIMENTAÇÃO - Pós 2026 - REL.FINANCEIRO (R$)', 'DUPLICAÇÃO - < 2022 - Descrição', 'DUPLICAÇÃO - < 2022 - km (i)', 
                    'DUPLICAÇÃO - < 2022 - km (f)', 'DUPLICAÇÃO - < 2022 - Ext. (km)', 'DUPLICAÇÃO - < 2022 - PERCENTUAL (%)', 
                    'DUPLICAÇÃO - < 2022 - (km)%', 'DUPLICAÇÃO - < 2022 - FINANCEIRO (R$)', 'DUPLICAÇÃO - 2023 - Descrição', 
                    'DUPLICAÇÃO - 2023 - km (i)', 'DUPLICAÇÃO - 2023 - km (f)', 'DUPLICAÇÃO - 2023 - Ext. (km)', 
                    'DUPLICAÇÃO - 2023 - PERCENTUAL (%)', 'DUPLICAÇÃO - 2023 - (km)%', 'DUPLICAÇÃO - 2023 - FINANCEIRO (R$)', 
                    'DUPLICAÇÃO - 2024 - Descrição', 'DUPLICAÇÃO - 2024 - km (i)', 'DUPLICAÇÃO - 2024 - km (f)', 
                    'DUPLICAÇÃO - 2024 - Ext. (km)', 'DUPLICAÇÃO - 2024 - PERCENTUAL (%)', 'DUPLICAÇÃO - 2024 - (km)%', 
                    'DUPLICAÇÃO - 2024 - FINANCEIRO (R$)', 'DUPLICAÇÃO - 2025 - Descrição', 'DUPLICAÇÃO - 2025 - km (i)', 
                    'DUPLICAÇÃO - 2025 - km (f)', 'DUPLICAÇÃO - 2025 - Ext. (km)', 'DUPLICAÇÃO - 2025 - PERCENTUAL (%)', 
                    'DUPLICAÇÃO - 2025 - (km)%', 'DUPLICAÇÃO - 2025 - FINANCEIRO (R$)', 'DUPLICAÇÃO - 2026 - Descrição', 
                    'DUPLICAÇÃO - 2026 - km (i)', 'DUPLICAÇÃO - 2026 - km (f)', 'DUPLICAÇÃO - 2026 - Ext. (km)', 
                    'DUPLICAÇÃO - 2026 - PERCENTUAL (%)', 'DUPLICAÇÃO - 2026 - (km)%', 'DUPLICAÇÃO - 2026 - FINANCEIRO (R$)', 
                    'DUPLICAÇÃO - Pós 2026 - Descrição', 'DUPLICAÇÃO - Pós 2026 - km (i)', 'DUPLICAÇÃO - Pós 2026 - km (f)', 
                    'DUPLICAÇÃO - Pós 2026 - Ext. (km)', 'DUPLICAÇÃO - Pós 2026 - PERCENTUAL (%)', 'DUPLICAÇÃO - Pós 2026 - (km)%', 
                    'DUPLICAÇÃO - Pós 2026 - FINANCEIRO (R$)', 'DUPLICAÇÃO - Pós 2026 - REL. FÍSICO (km)', 'DUPLICAÇÃO - Pós 2026 - REL.FINANCEIRO (R$)', 
                    'OAE - < 2022 - Descrição', 'OAE - < 2022 - km (i)', 'OAE - < 2022 - km (f)', 'OAE - < 2022 - Ext. (km)', 
                    'OAE - < 2022 - PERCENTUAL (%)', 'OAE - < 2022 - (km)%', 'OAE - < 2022 - FINANCEIRO (R$)', 'OAE - 2023 - Descrição', 
                    'OAE - 2023 - km (i)', 'OAE - 2023 - km (f)', 'OAE - 2023 - Ext. (km)', 'OAE - 2023 - PERCENTUAL (%)', 
                    'OAE - 2023 - (km)%', 'OAE - 2023 - FINANCEIRO (R$)', 'OAE - 2024 - Descrição', 'OAE - 2024 - km (i)', 
                    'OAE - 2024 - km (f)', 'OAE - 2024 - Ext. (km)', 'OAE - 2024 - PERCENTUAL (%)', 'OAE - 2024 - (km)%', 
                    'OAE - 2024 - FINANCEIRO (R$)', 'OAE - 2025 - Descrição', 'OAE - 2025 - km (i)', 'OAE - 2025 - km (f)', 
                    'OAE - 2025 - Ext. (km)', 'OAE - 2025 - PERCENTUAL (%)', 'OAE - 2025 - (km)%', 'OAE - 2025 - FINANCEIRO (R$)', 
                    'OAE - 2026 - Descrição', 'OAE - 2026 - km (i)', 'OAE - 2026 - km (f)', 'OAE - 2026 - Ext. (km)', 
                    'OAE - 2026 - PERCENTUAL (%)', 'OAE - 2026 - (km)%', 'OAE - 2026 - FINANCEIRO (R$)', 'OAE - Pós 2026 - Descrição', 
                    'OAE - Pós 2026 - km (i)', 'OAE - Pós 2026 - km (f)', 'OAE - Pós 2026 - Ext. (km)', 'OAE - Pós 2026 - PERCENTUAL (%)', 
                    'OAE - Pós 2026 - (km)%', 'OAE - Pós 2026 - FINANCEIRO (R$)', 'OAE - Pós 2026 - REL. FÍSICO (km)', 'OAE - Pós 2026 - REL.FINANCEIRO (R$)', 
                    'CONTORNO - < 2022 - Descrição', 'CONTORNO - < 2022 - km (i)', 'CONTORNO - < 2022 - km (f)', 'CONTORNO - < 2022 - Ext. (km)', 
                    'CONTORNO - < 2022 - PERCENTUAL (%)', 'CONTORNO - < 2022 - (km)%', 'CONTORNO - < 2022 - FINANCEIRO (R$)', 'CONTORNO - 2023 - Descrição', 
                    'CONTORNO - 2023 - km (i)', 'CONTORNO - 2023 - km (f)', 'CONTORNO - 2023 - Ext. (km)', 'CONTORNO - 2023 - PERCENTUAL (%)', 
                    'CONTORNO - 2023 - (km)%', 'CONTORNO - 2023 - FINANCEIRO (R$)', 'CONTORNO - 2024 - Descrição', 'CONTORNO - 2024 - km (i)', 
                    'CONTORNO - 2024 - km (f)', 'CONTORNO - 2024 - Ext. (km)', 'CONTORNO - 2024 - PERCENTUAL (%)', 'CONTORNO - 2024 - (km)%', 
                    'CONTORNO - 2024 - FINANCEIRO (R$)', 'CONTORNO - 2025 - Descrição', 'CONTORNO - 2025 - km (i)', 'CONTORNO - 2025 - km (f)', 
                    'CONTORNO - 2025 - Ext. (km)', 'CONTORNO - 2025 - PERCENTUAL (%)', 'CONTORNO - 2025 - (km)%', 'CONTORNO - 2025 - FINANCEIRO (R$)', 
                    'CONTORNO - 2026 - Descrição', 'CONTORNO - 2026 - km (i)', 'CONTORNO - 2026 - km (f)', 'CONTORNO - 2026 - Ext. (km)', 
                    'CONTORNO - 2026 - PERCENTUAL (%)', 'CONTORNO - 2026 - (km)%', 'CONTORNO - 2026 - FINANCEIRO (R$)', 'CONTORNO - Pós 2026 - Descrição', 
                    'CONTORNO - Pós 2026 - km (i)', 'CONTORNO - Pós 2026 - km (f)', 'CONTORNO - Pós 2026 - Ext. (km)', 'CONTORNO - Pós 2026 - PERCENTUAL (%)', 
                    'CONTORNO - Pós 2026 - (km)%', 'CONTORNO - Pós 2026 - FINANCEIRO (R$)', 'CONTORNO - Pós 2026 - REL. FÍSICO (km)', 'CONTORNO - Pós 2026 - REL.FINANCEIRO (R$)', 
                    'FX ADICIONAL - < 2022 - Descrição', 'FX ADICIONAL - < 2022 - km (i)', 'FX ADICIONAL - < 2022 - km (f)', 'FX ADICIONAL - < 2022 - Ext. (km)', 
                    'FX ADICIONAL - < 2022 - PERCENTUAL (%)', 'FX ADICIONAL - < 2022 - (km)%', 'FX ADICIONAL - < 2022 - FINANCEIRO (R$)', 'FX ADICIONAL - 2023 - Descrição', 
                    'FX ADICIONAL - 2023 - km (i)', 'FX ADICIONAL - 2023 - km (f)', 'FX ADICIONAL - 2023 - Ext. (km)', 'FX ADICIONAL - 2023 - PERCENTUAL (%)', 
                    'FX ADICIONAL - 2023 - (km)%', 'FX ADICIONAL - 2023 - FINANCEIRO (R$)', 'FX ADICIONAL - 2024 - Descrição', 'FX ADICIONAL - 2024 - km (i)', 
                    'FX ADICIONAL - 2024 - km (f)', 'FX ADICIONAL - 2024 - Ext. (km)', 'FX ADICIONAL - 2024 - PERCENTUAL (%)', 'FX ADICIONAL - 2024 - (km)%', 
                    'FX ADICIONAL - 2024 - FINANCEIRO (R$)', 'FX ADICIONAL - 2025 - Descrição', 'FX ADICIONAL - 2025 - km (i)', 'FX ADICIONAL - 2025 - km (f)', 
                    'FX ADICIONAL - 2025 - Ext. (km)', 'FX ADICIONAL - 2025 - PERCENTUAL (%)', 'FX ADICIONAL - 2025 - (km)%', 'FX ADICIONAL - 2025 - FINANCEIRO (R$)', 
                    'FX ADICIONAL - 2026 - Descrição', 'FX ADICIONAL - 2026 - km (i)', 'FX ADICIONAL - 2026 - km (f)', 'FX ADICIONAL - 2026 - Ext. (km)', 
                    'FX ADICIONAL - 2026 - PERCENTUAL (%)', 'FX ADICIONAL - 2026 - (km)%', 'FX ADICIONAL - 2026 - FINANCEIRO (R$)', 
                    'FX ADICIONAL - Pós 2026 - Descrição', 'FX ADICIONAL - Pós 2026 - km (i)', 'FX ADICIONAL - Pós 2026 - km (f)', 
                    'FX ADICIONAL - Pós 2026 - Ext. (km)', 'FX ADICIONAL - Pós 2026 - PERCENTUAL (%)', 'FX ADICIONAL - Pós 2026 - (km)%', 
                    'FX ADICIONAL - Pós 2026 - FINANCEIRO (R$)', 'FX ADICIONAL - Pós 2026 - REL. FÍSICO (km)', 'FX ADICIONAL - Pós 2026 - REL.FINANCEIRO (R$)', 
                    'TERCEIRA FAIXA - < 2022 - Descrição', 'TERCEIRA FAIXA - < 2022 - km (i)', 'TERCEIRA FAIXA - < 2022 - km (f)', 'TERCEIRA FAIXA - < 2022 - Ext. (km)', 
                    'TERCEIRA FAIXA - < 2022 - PERCENTUAL (%)', 'TERCEIRA FAIXA - < 2022 - (km)%', 'TERCEIRA FAIXA - < 2022 - FINANCEIRO (R$)', 'TERCEIRA FAIXA - 2023 - Descrição', 
                    'TERCEIRA FAIXA - 2023 - km (i)', 'TERCEIRA FAIXA - 2023 - km (f)', 'TERCEIRA FAIXA - 2023 - Ext. (km)', 'TERCEIRA FAIXA - 2023 - PERCENTUAL (%)', 
                    'TERCEIRA FAIXA - 2023 - (km)%', 'TERCEIRA FAIXA - 2023 - FINANCEIRO (R$)', 'TERCEIRA FAIXA - 2024 - Descrição', 'TERCEIRA FAIXA - 2024 - km (i)', 
                    'TERCEIRA FAIXA - 2024 - km (f)', 'TERCEIRA FAIXA - 2024 - Ext. (km)', 'TERCEIRA FAIXA - 2024 - PERCENTUAL (%)', 'TERCEIRA FAIXA - 2024 - (km)%', 
                    'TERCEIRA FAIXA - 2024 - FINANCEIRO (R$)', 'TERCEIRA FAIXA - 2025 - Descrição', 'TERCEIRA FAIXA - 2025 - km (i)', 'TERCEIRA FAIXA - 2025 - km (f)', 
                    'TERCEIRA FAIXA - 2025 - Ext. (km)', 'TERCEIRA FAIXA - 2025 - PERCENTUAL (%)', 'TERCEIRA FAIXA - 2025 - (km)%', 'TERCEIRA FAIXA - 2025 - FINANCEIRO (R$)', 
                    'TERCEIRA FAIXA - 2026 - Descrição', 'TERCEIRA FAIXA - 2026 - km (i)', 'TERCEIRA FAIXA - 2026 - km (f)', 'TERCEIRA FAIXA - 2026 - Ext. (km)', 
                    'TERCEIRA FAIXA - 2026 - PERCENTUAL (%)', 'TERCEIRA FAIXA - 2026 - (km)%', 'TERCEIRA FAIXA - 2026 - FINANCEIRO (R$)', 
                    'TERCEIRA FAIXA - Pós 2026 - Descrição', 'TERCEIRA FAIXA - Pós 2026 - km (i)', 'TERCEIRA FAIXA - Pós 2026 - km (f)', 
                    'TERCEIRA FAIXA - Pós 2026 - Ext. (km)', 'TERCEIRA FAIXA - Pós 2026 - PERCENTUAL (%)', 'TERCEIRA FAIXA - Pós 2026 - (km)%', 
                    'TERCEIRA FAIXA - Pós 2026 - FINANCEIRO (R$)', 'TERCEIRA FAIXA - Pós 2026 - REL. FÍSICO (km)', 'TERCEIRA FAIXA - Pós 2026 - REL.FINANCEIRO (R$)']

In [ ]:
df_meta_27_copy = df_meta_27.copy()
# Criar um dicionário que mapeia qualquer coluna a partir do índice 9 para os novos nomes
rename_dict = {df_meta_27_copy.columns[i]: name for i, name in enumerate(new_column_names, start=9)}

# Usa o método rename para renomear as colunas, substituindo os nomes existentes
df_meta_27_copy.rename(columns=rename_dict, inplace=True)

In [ ]:
df_per_copy['ESTRUTURADOR DO PROJETO'].unique()

In [ ]:
df_per_copy['PROPONENTE'].unique()

In [ ]:
df_per_copy['ESTRUTURADOR DO PROJETO'] = 'EPL'

In [ ]:
df_per_copy['PROPONENTE'] = 'ANTT'

In [ ]:
df_per_copy.index = range(1, len(df_per_copy) + 1)
df_per_copy.index.name = 'ID-ÚNICO'
df_per_copy = df_per_copy.reset_index()

In [ ]:
df_per_copy

In [ ]:
df_per_raw = df_per_copy

In [ ]:
# df_per_copy.to_excel('Dados Gerados/RAW_PER (1).xlsx', index=False)

In [ ]:
colunas_manter = [
    'ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
    'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
]

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_per_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_per_copy = df_per_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [ ]:
df_per_copy

In [ ]:
df_per_copy = df_per_copy.loc[df_per_copy['Valor'] != 0]
df_per_copy = df_per_copy.dropna(subset=['Valor'])

In [ ]:
df_per_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_per_copy['Atributo'].str.split(' - ', expand=True)


In [ ]:
df_per_copy = df_per_copy[['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
    'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO', 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [ ]:
'''# Converter colunas para 'category'
df_per_copy['SETOR'] =  df_per_copy['SETOR'].astype('category')
df_per_copy['UF'] =  df_per_copy['UF'].astype('category')
df_per_copy['PROPONENTE'] =  df_per_copy['PROPONENTE'].astype('category')
df_per_copy['EXECUTOR (Grupo Controlador)'] =  df_per_copy['EXECUTOR (Grupo Controlador)'].astype('category')
df_per_copy['ESTRUTURADOR DO PROJETO'] =  df_per_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_per_copy['Atributo.1'] =  df_per_copy['Atributo.1'].astype('category')
df_per_copy['Atributo.2'] =  df_per_copy['Atributo.2'].astype('category')

# Converter colunas para 'string'
df_per_copy['EMPREENDIMENTO'] =  df_per_copy['EMPREENDIMENTO'].astype('string') 
'''

In [ ]:
df_per_copy

In [ ]:
df_per_copy.to_excel('Dados Gerados/PER (1).xlsx', index=False)

---

In [ ]:
df_per_raw

In [ ]:
# Drop na coluna 'Região' que não é usada.
df_per_raw.drop(columns=['ID-ÚNICO'], inplace=True)

In [ ]:
df_per_raw.index = range(1, len(df_per_raw) + 1)
df_per_raw.index.name = 'ID-ÚNICO'
df_per_raw = df_per_raw.reset_index()

In [ ]:
df_per_raw = df_per_raw.sort_values(by='ID-ÚNICO', ascending=True)

In [ ]:
# Lista de strings que devem estar no nome das colunas para serem mantidas
keywords_to_keep = [
    'ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO', 
    'Descrição', 'Ext. (km)', 'FINANCEIRO (R$)', 
    '(km)%'
]

# Identifica colunas que possuem qualquer uma das strings na lista de keywords
columns_to_keep = [col for col in df_per_raw.columns if any(keyword in col for keyword in keywords_to_keep)]

In [ ]:
# Mantém apenas as colunas identificadas
df_per_raw = df_per_raw[columns_to_keep]

In [ ]:
df_per_raw.columns

In [ ]:
# Seleciona colunas da posição em diante
cols_a_verificar = df_per_raw.iloc[:, 3:]

# Remove linhas onde todas essas colunas estão vazias
df_per_raw = df_per_raw.dropna(subset=cols_a_verificar.columns, how='all')

In [ ]:
# Primeiro, crie df_meta_25_raw se ele não existir
if 'df_meta_25' in locals() or 'df_meta_25' in globals():
    df_meta_25_raw = df_meta_25.copy()
    print("df_meta_25_raw criado a partir de df_meta_25")
else:
    # Ou carregue do arquivo
    df_meta_25_raw = pd.read_excel(FILE_PATH, sheet_name='META(2025)')
    print("df_meta_25_raw carregado do arquivo")
df_a_executar_copy = df_a_executar.copy()
# Agora execute o código de renomeação
rename_dict = {df_meta_25_raw.columns[i]: name for i, name in enumerate(new_column_names, start=9)}
df_meta_25_raw.rename(columns=rename_dict, inplace=True)
# Criar um dicionário que mapeia qualquer coluna a partir do índice 9 para os novos nomes
rename_dict = {df_meta_25_raw.columns[i]: name for i, name in enumerate(new_column_names, start=9)}

# Usa o método rename para renomear as colunas, substituindo os nomes existentes
df_a_executar_copy.rename(columns=rename_dict, inplace=True)

In [ ]:
# Identifica colunas que possuem a string 'OBSERVAÇÕES' em seu nome (preserva colunas de RISCOS)
columns_to_drop = [col for col in df_a_executar_copy.columns if 'OBSERVAÇÕES' in col]
if columns_to_drop:
    df_a_executar_copy = df_a_executar_copy.drop(columns=columns_to_drop)

In [ ]:
df_a_executar_copy

In [ ]:
# 1. Recarregar a planilha com os parâmetros corretos
df_a_executar = pd.read_excel(FILE_PATH, sheet_name='À EXECUTAR (2026 DIANTE)', header=2)

# 2. Fazer uma cópia para processamento
df_a_executar_copy = df_a_executar.copy()

# 3. Verificar se carregou corretamente
print("✅ DataFrame carregado corretamente")
print(f"Shape: {df_a_executar_copy.shape}")
print(f"\nPrimeiras colunas:")
for i, col in enumerate(df_a_executar_copy.columns[:10], 1):
    print(f"{i:2}. '{col}'")

# 4. Verificar se a coluna 'ESTADO/LOTE' existe
if 'ESTADO/LOTE' in df_a_executar_copy.columns:
    print(f"\n✅ Coluna 'ESTADO/LOTE' encontrada!")
    print(f"Valores únicos: {df_a_executar_copy['ESTADO/LOTE'].unique()[:10]}")  # Primeiros 10 valores únicos
else:
    print("\n❌ Coluna 'ESTADO/LOTE' NÃO encontrada!")

In [ ]:
columns_to_fill = ['EMPREENDIMENTO']
df_a_executar_copy[columns_to_fill] = df_a_executar_copy.groupby((df_a_executar_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [ ]:
df_per_raw1 = df_per_raw[['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO']]

In [ ]:
df_a_executar_copy = df_a_executar_copy[~df_a_executar_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]

In [ ]:
colunas_manter = ['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO']

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_per_raw.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_per_raw = df_per_raw.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [ ]:
df_per_raw['SETOR'] = 'Rodoviário'

In [ ]:
df_per_raw

In [ ]:
df_per_raw[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_per_raw['Atributo'].str.split(' - ', expand=True)

In [ ]:
df_per_raw = df_per_raw[['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO', 'Atributo.1', 'Atributo.2', 'Valor']]

In [ ]:
df_per_raw.columns

In [ ]:
df_per_raw

In [ ]:
df_per_raw.dropna(subset=['Valor'])

In [ ]:
# Pivotar o DataFrame
df_per_raw = df_per_raw.pivot_table(index=['ID-ÚNICO', 'Atributo.1'], 
                                            columns='Atributo.2', values='Valor', aggfunc='first').reset_index()

In [ ]:
df_per_raw.columns

In [ ]:
df_per_raw

In [ ]:
# Remove a coluna de índice indesejada, se existir
if 'Atributo.2' in df_per_raw.columns:
    df_per_raw = df_per_raw.drop(columns='Atributo.2')

In [ ]:
df_per_raw1

In [ ]:
# Faz o merge com base na coluna 'ID-ÚNICO'
df_per_raw = pd.merge(df_per_raw1, df_per_raw, on='ID-ÚNICO', how='inner')

In [ ]:
df_per_raw

In [ ]:
df_per_raw = df_per_raw.dropna(subset=['Atributo.1'])

In [ ]:
df_per_raw

In [ ]:
df_per_raw.columns

In [ ]:
df_per_raw = df_per_raw[['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO', 'Atributo.1', 'Descrição',
            'Ext. (km)', '(km)%', 'FINANCEIRO (R$)']]

In [ ]:
# Renomear Colunas
df_per_raw.rename(columns={'ID-ÚNICO': 'ID-ÚNICO1'}, inplace=True)
df_per_raw.rename(columns={'SETOR': 'SETOR2'}, inplace=True)
df_per_raw.rename(columns={'EMPREENDIMENTO': 'EMPREENDIMENTO2'}, inplace=True)
df_per_raw.rename(columns={'Descrição': 'Descrição2'}, inplace=True)
df_per_raw.rename(columns={'Ext. (km)': 'Ext.(km)2'}, inplace=True)
df_per_raw.rename(columns={'FINANCEIRO (R$)': 'FINANCEIRO(R$)2'}, inplace=True)
df_per_raw.rename(columns={'(km)%': '(km)%2'}, inplace=True)

In [ ]:
df_per_raw

In [ ]:
df_per_raw.to_excel('PER(2).xlsx', sheet_name= 'PER(2)', index= False)

<h1><b><i>PLAN_EXEC ATÉ ANO ANTERIOR (22) -> (24)** </b></i></h1>

In [ ]:
#CHECKPOINT
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant.copy()

In [ ]:
df_plan_exec_ano_ant_copy = pd.read_excel(file_path, sheet_name='PLAN_EXEC ATÉ ANO ANTERIOR (22)', header=2)

In [ ]:
df_plan_exec_ano_ant_copy

In [ ]:
df_plan_exec_ano_ant_copy.dropna(subset=['ESTADO/LOTE'])

In [ ]:
columns_to_fill = ['SETOR', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR             (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO']
df_plan_exec_ano_ant_copy[columns_to_fill] = df_plan_exec_ano_ant_copy.groupby((df_plan_exec_ano_ant_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [ ]:
df_plan_exec_ano_ant_copy['ESTADO/LOTE'] = df_plan_exec_ano_ant_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS',
       'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO',
       'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA',
       'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)

# Agora preenchemos para baixo até encontrar o próximo valor não-NaN (isto é, a próxima 'Situação')
df_plan_exec_ano_ant_copy['ESTADO/LOTE'] = df_plan_exec_ano_ant_copy['ESTADO/LOTE'].fillna(method='ffill')

In [ ]:
# Renomear Coluna ['EXECUTOR...']

df_plan_exec_ano_ant_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR (Grupo Controlador)'}, inplace=True)
df_plan_exec_ano_ant_copy.rename(columns={'PAVIMENTAÇÃO                                                                                                                                                                                                                                                                                                                           (Acumulada até 2022)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_plan_exec_ano_ant_copy.rename(columns={'DUPLICAÇÃO                                                                                                                                                                                                                                                                                                  (Acumulada até 2022)': 'DUPLICAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_plan_exec_ano_ant_copy.rename(columns={'OAE                                                                                                                                                                                                                                                                                                               (Acumulada até 2022)': 'OAE - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_plan_exec_ano_ant_copy.rename(columns={'CONTORNO                                                                                                                                                                                                                                                                                                          (Acumulada até 2022)': 'CONTORNO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_plan_exec_ano_ant_copy.rename(columns={'FX ADICIONAL                                                                                                                                                                                                                                                                (Acumulada até 2022)': 'FX ADICIONAL - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_plan_exec_ano_ant_copy.rename(columns={'TERCEIRA FAIXA                                                                                                                                                                                                                                                                       (Acumulada até 2022)': 'TERCEIRA FAIXA - (Acumulada até ano anterior) - Descrição'}, inplace=True)

In [ ]:
new_column_names = ['PAVIMENTAÇÃO - (Acumulada até ano anterior) - km (i)', 
                    'PAVIMENTAÇÃO - (Acumulada até ano anterior) - km (f)', 
                    'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Ext. (km)', 
                    'PAVIMENTAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
                    'PAVIMENTAÇÃO - (Acumulada até ano anterior) - (km)% PLAN', 
                    'PAVIMENTAÇÃO - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)',
                    'PAVIMENTAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC',
                    'PAVIMENTAÇÃO - (Acumulada até ano anterior) - (km)% EXEC',
                    'PAVIMENTAÇÃO - (Acumulada até ano anterior) - FINANCEIRO (R$)',
                    'DUPLICAÇÃO - (Acumulada até ano anterior) - Descrição',
                    'DUPLICAÇÃO - (Acumulada até ano anterior) - km (i)', 
                    'DUPLICAÇÃO - (Acumulada até ano anterior) - km (f)', 
                    'DUPLICAÇÃO - (Acumulada até ano anterior) - Ext. (km)', 
                    'DUPLICAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
                    'DUPLICAÇÃO - (Acumulada até ano anterior) - (km)% PLAN', 
                    'DUPLICAÇÃO - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)',
                    'DUPLICAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC',
                    'DUPLICAÇÃO - (Acumulada até ano anterior) - (km)% EXEC',
                    'DUPLICAÇÃO - (Acumulada até ano anterior) - FINANCEIRO (R$)',
                    'OAE - (Acumulada até ano anterior) - Descrição',
                    'OAE - (Acumulada até ano anterior) - km (i)', 
                    'OAE - (Acumulada até ano anterior) - km (f)', 
                    'OAE - (Acumulada até ano anterior) - Ext. (km)', 
                    'OAE - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
                    'OAE - (Acumulada até ano anterior) - (km)% PLAN', 
                    'OAE - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)',
                    'OAE - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC',
                    'OAE - (Acumulada até ano anterior) - (km)% EXEC',
                    'OAE - (Acumulada até ano anterior) - FINANCEIRO (R$)',
                    'CONTORNO - (Acumulada até ano anterior) - Descrição',
                    'CONTORNO - (Acumulada até ano anterior) - km (i)', 
                    'CONTORNO - (Acumulada até ano anterior) - km (f)', 
                    'CONTORNO - (Acumulada até ano anterior) - Ext. (km)', 
                    'CONTORNO - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
                    'CONTORNO - (Acumulada até ano anterior) - (km)% PLAN', 
                    'CONTORNO - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)',
                    'CONTORNO - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC',
                    'CONTORNO - (Acumulada até ano anterior) - (km)% EXEC',
                    'CONTORNO - (Acumulada até ano anterior) - FINANCEIRO (R$)',
                    'FX ADICIONAL - (Acumulada até ano anterior) - Descrição',
                    'FX ADICIONAL - (Acumulada até ano anterior) - km (i)', 
                    'FX ADICIONAL - (Acumulada até ano anterior) - km (f)', 
                    'FX ADICIONAL - (Acumulada até ano anterior) - Ext. (km)', 
                    'FX ADICIONAL - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
                    'FX ADICIONAL - (Acumulada até ano anterior) - (km)% PLAN', 
                    'FX ADICIONAL - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)',
                    'FX ADICIONAL - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC',
                    'FX ADICIONAL - (Acumulada até ano anterior) - (km)% EXEC',
                    'FX ADICIONAL - (Acumulada até ano anterior) - FINANCEIRO (R$)',
                    'TERCEIRA FAIXA - (Acumulada até ano anterior) - Descrição',
                    'TERCEIRA FAIXA - (Acumulada até ano anterior) - km (i)', 
                    'TERCEIRA FAIXA - (Acumulada até ano anterior) - km (f)', 
                    'TERCEIRA FAIXA - (Acumulada até ano anterior) - Ext. (km)', 
                    'TERCEIRA FAIXA - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
                    'TERCEIRA FAIXA - (Acumulada até ano anterior) - (km)% PLAN', 
                    'TERCEIRA FAIXA - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)',
                    'TERCEIRA FAIXA - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC',
                    'TERCEIRA FAIXA - (Acumulada até ano anterior) - (km)% EXEC',
                    'TERCEIRA FAIXA - (Acumulada até ano anterior) - FINANCEIRO (R$)']

In [ ]:
# Criar um dicionário que mapeia 'Unnamed: x' para os novos nomes
rename_dict = {'Unnamed: {}'.format(i): name for i, name in enumerate(new_column_names, start=9)}

# Usar o método rename para renomear as colunas
df_plan_exec_ano_ant_copy.rename(columns=rename_dict, inplace=True)

In [ ]:
# Encontrar o índice da coluna 'Unnamed: 69'
idx = df_plan_exec_ano_ant_copy.columns.get_loc("Unnamed: 69")

# Remover todas as colunas a partir deste índice
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.iloc[:, :idx]

In [ ]:
# Identifica colunas que possuem as strings 'RISCOS' ou 'OBSERVAÇÕES' em seu nome
columns_to_drop = [col for col in df_plan_exec_ano_ant_copy.columns if 'RISCOS' in col or 'OBSERVAÇÕES' in col]

# Remove as colunas identificadas
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.drop(columns=columns_to_drop)

In [ ]:
df_plan_exec_ano_ant_copy.columns

In [ ]:
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.drop([0, 1, 2], axis = 0)

In [ ]:
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy[~df_plan_exec_ano_ant_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]

In [ ]:
df_plan_exec_ano_ant_copy

In [ ]:
df_plan_exec_ano_ant_copy.columns

In [ ]:
df_plan_exec_ano_ant_copy.index = range(1, len(df_plan_exec_ano_ant_copy) + 1)
df_plan_exec_ano_ant_copy.index.name = 'ID-ÚNICO'
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.reset_index()

In [ ]:
df_plan_exec_ano_ant_raw = df_plan_exec_ano_ant_copy

In [ ]:
# df_plan_exec_ano_ant_copy.to_excel('Dados Gerados/RAW_PLAN_EXEC ATÉ ANO ANTERIOR (22).xlsx', index=False)

In [ ]:
colunas_manter = ['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
]

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_plan_exec_ano_ant_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [ ]:
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.loc[df_plan_exec_ano_ant_copy['Valor'] != 0]
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.dropna(subset=['Valor'])

In [ ]:
df_plan_exec_ano_ant_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_plan_exec_ano_ant_copy['Atributo'].str.split(' - ', expand=True)

In [ ]:
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy[['SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [ ]:
# Converter colunas para 'category'
df_plan_exec_ano_ant_copy['SETOR'] = df_plan_exec_ano_ant_copy['SETOR'].astype('category')
df_plan_exec_ano_ant_copy['UF'] = df_plan_exec_ano_ant_copy['UF'].astype('category')
df_plan_exec_ano_ant_copy['PROPONENTE'] = df_plan_exec_ano_ant_copy['PROPONENTE'].astype('category')
df_plan_exec_ano_ant_copy['EXECUTOR (Grupo Controlador)'] = df_plan_exec_ano_ant_copy['EXECUTOR (Grupo Controlador)'].astype('category')
df_plan_exec_ano_ant_copy['ESTRUTURADOR DO PROJETO'] = df_plan_exec_ano_ant_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_plan_exec_ano_ant_copy['Atributo.1'] = df_plan_exec_ano_ant_copy['Atributo.1'].astype('category')
df_plan_exec_ano_ant_copy['Atributo.2'] = df_plan_exec_ano_ant_copy['Atributo.2'].astype('category')

# Converter colunas para 'string'
df_plan_exec_ano_ant_copy['EMPREENDIMENTO'] = df_plan_exec_ano_ant_copy['EMPREENDIMENTO'].astype('string') 


---

<h1><b><i>'META(2023)'</b></i></h1>

In [ ]:
#CHECKPOINT
df_meta_23_copy = df_meta_23.copy()

In [ ]:
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)

In [ ]:
df_meta_23_copy

In [ ]:
df_meta_23_copy = df_meta_23_copy.drop([0, 1, 2], axis = 0)

In [ ]:
df_meta_23_copy.columns

In [ ]:
# Drop na coluna 'Região' que não é usada.
df_meta_23_copy.drop(columns=['Região'], inplace=True)

In [ ]:
# Removendo colunas
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)

In [ ]:
# Renomear Colunas

df_meta_23_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR (Grupo Controlador)'}, inplace=True)
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO                                                                                                                                                                                                                                                                                     (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_meta_23_copy.rename(columns={'DUPLICAÇÃO                                                                                                                                                                                                                                                                                               (Ano 2023)': 'DUPLICAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_meta_23_copy.rename(columns={'OAE                                                                                                                                                                                                                                                                                                                    (Ano 2023)': 'OAE - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_meta_23_copy.rename(columns={'CONTORNO                                                                                                                                                                                                                                                                                                             (Ano 2023)': 'CONTORNO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_meta_23_copy.rename(columns={'FX ADICIONAL                                                                                                                                                                                                                                                                                                     (Ano 2023)': 'FX ADICIONAL - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_meta_23_copy.rename(columns={'TERCEIRA FAIXA                                                                                                                                                                                                                                                                                         (Ano 2023)': 'TERCEIRA FAIXA - (Acumulada até ano anterior) - Descrição'}, inplace=True)

In [ ]:
new_column_names = ['PAVIMENTAÇÃO - (Acumulada até ano anterior) - km (i)', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - km (f)', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Ext. (km)', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - (km)% PLAN', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - (km)% EXEC', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - FINANCEIRO (R$)',
 'DUPLICAÇÃO - (Acumulada até ano anterior) - Descrição',
 'DUPLICAÇÃO - (Acumulada até ano anterior) - km (i)', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - km (f)', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - Ext. (km)', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - (km)% PLAN', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - (km)% EXEC', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - FINANCEIRO (R$)',
 'OAE - (Acumulada até ano anterior) - Descrição',
 'OAE - (Acumulada até ano anterior) - km (i)', 
 'OAE - (Acumulada até ano anterior) - km (f)', 
 'OAE - (Acumulada até ano anterior) - Ext. (km)', 
 'OAE - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'OAE - (Acumulada até ano anterior) - (km)% PLAN', 
 'OAE - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'OAE - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'OAE - (Acumulada até ano anterior) - (km)% EXEC', 
 'OAE - (Acumulada até ano anterior) - FINANCEIRO (R$)',
 'CONTORNO - (Acumulada até ano anterior) - Descrição',
 'CONTORNO - (Acumulada até ano anterior) - km (i)', 
 'CONTORNO - (Acumulada até ano anterior) - km (f)', 
 'CONTORNO - (Acumulada até ano anterior) - Ext. (km)', 
 'CONTORNO - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'CONTORNO - (Acumulada até ano anterior) - (km)% PLAN', 
 'CONTORNO - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'CONTORNO - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'CONTORNO - (Acumulada até ano anterior) - (km)% EXEC',
 'CONTORNO - (Acumulada até ano anterior) - FINANCEIRO (R$)',
 'FX ADICIONAL - (Acumulada até ano anterior) - Descrição',
 'FX ADICIONAL - (Acumulada até ano anterior) - km (i)',
 'FX ADICIONAL - (Acumulada até ano anterior) - km (f)', 
 'FX ADICIONAL - (Acumulada até ano anterior) - Ext. (km)', 
 'FX ADICIONAL - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'FX ADICIONAL - (Acumulada até ano anterior) - (km)% PLAN', 
 'FX ADICIONAL - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'FX ADICIONAL - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'FX ADICIONAL - (Acumulada até ano anterior) - (km)% EXEC', 
 'FX ADICIONAL - (Acumulada até ano anterior) - FINANCEIRO (R$)',
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - Descrição',
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - km (i)', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - km (f)', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - Ext. (km)', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - (km)% PLAN', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - (km)% EXEC', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - FINANCEIRO (R$)']


In [ ]:
# Criar um dicionário que mapeia 'Unnamed: x' para os novos nomes
rename_dict = {'Unnamed: {}'.format(i): name for i, name in enumerate(new_column_names, start=178)}

# Usar o método rename para renomear as colunas
df_meta_23_copy.rename(columns=rename_dict, inplace=True)

In [ ]:
# Identifica colunas que possuem a string 'OBSERVAÇÕES' em seu nome
columns_to_drop = [col for col in df_meta_23_copy.columns if 'OBSERVAÇÕES' in col]

# Remove as colunas identificadas
df_meta_23_copy = df_meta_23_copy.drop(columns=columns_to_drop)

In [ ]:
df_meta_23_copy.columns

In [ ]:
df_meta_23_copy.dropna(subset=['ESTADO/LOTE'])

In [ ]:
columns_to_fill = ['EMPREENDIMENTO']
df_meta_23_copy[columns_to_fill] = df_meta_23_copy.groupby((df_meta_23_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [ ]:
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS',
       'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO',
       'MATO GROSSO', 'PARÁ', 'RIO/SP', 'SÃO PAULO', 'PARANÁ', 'BAHIA',
       'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)

# Agora preenchemos para baixo até encontrar o próximo valor não-NaN (isto é, a próxima 'Situação')
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')

In [ ]:
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]

In [ ]:
df_meta_23_copy.columns

In [ ]:
df_meta_23_copy.index = range(1, len(df_meta_23_copy) + 1)
df_meta_23_copy.index.name = 'ID-ÚNICO'
df_meta_23_copy = df_meta_23_copy.reset_index()

In [ ]:
df_meta_23_raw = df_meta_23_copy

In [ ]:
# df_meta_23_copy.to_excel('Dados Gerados/RAW_META(2023).xlsx', index=False)

In [ ]:
colunas_manter = ['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO']

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_meta_23_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [ ]:
df_meta_23_copy['EMPREENDIMENTO'].unique()

In [ ]:
df_meta_23_copy = df_meta_23_copy.loc[df_meta_23_copy['Valor'] != 0]
df_meta_23_copy = df_meta_23_copy.dropna(subset=['Valor'])

In [ ]:
df_meta_23_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_meta_23_copy['Atributo'].str.split(' - ', expand=True)

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

In [ ]:
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_meta_23_copy = df_meta_23_copy[['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

In [ ]:
# Converter colunas para 'category'
df_meta_23_copy['SETOR'] = df_meta_23_copy['SETOR'].astype('category')
df_meta_23_copy['UF'] = df_meta_23_copy['UF'].astype('category')
df_meta_23_copy['PROPONENTE'] = df_meta_23_copy['PROPONENTE'].astype('category')
df_meta_23_copy['EXECUTOR (Grupo Controlador)'] = df_meta_23_copy['EXECUTOR (Grupo Controlador)'].astype('category')
df_meta_23_copy['ESTRUTURADOR DO PROJETO'] = df_meta_23_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_meta_23_copy['Atributo.1'] = df_meta_23_copy['Atributo.1'].astype('category')
df_meta_23_copy['Atributo.2'] = df_meta_23_copy['Atributo.2'].astype('category')

# Converter colunas para 'string'
df_meta_23_copy['EMPREENDIMENTO'] = df_meta_23_copy['EMPREENDIMENTO'].astype('string') 


In [ ]:
df_meta_23_copy.to_excel('Dados Gerados/META(2023).xlsx', index=False, sheet_name= 'META(23)')

---

<h1><b><i>'META(2024)'</b></i></h1>

In [ ]:
#CHECKPOINT
df_meta_24_copy = df_meta_24.copy()

In [ ]:
df_meta_24_copy = pd.read_excel(file_path, sheet_name='META(2024)', header=2)

In [ ]:
df_meta_24_copy

In [ ]:
df_meta_24_copy = df_meta_24_copy.drop([0, 1, 2], axis = 0)

In [ ]:
df_meta_24_copy.columns

In [ ]:
# Drop na coluna 'Região' que não é usada.
df_meta_24_copy.drop(columns=['Região'], inplace=True)

In [ ]:
# Removendo colunas
df_meta_24_copy = df_meta_24_copy.drop(df_meta_24_copy.columns[8:176], axis=1)

In [ ]:
# Renomear Colunas

df_meta_24_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR (Grupo Controlador)'}, inplace=True)
df_meta_24_copy.rename(columns={'PAVIMENTAÇÃO                                                                                                                                                                                                                                                                                     (Ano 2024)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_meta_24_copy.rename(columns={'DUPLICAÇÃO                                                                                                                                                                                                                                                                                               (Ano 2024)': 'DUPLICAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_meta_24_copy.rename(columns={'OAE                                                                                                                                                                                                                                                                                                               (Ano 2024)': 'OAE - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_meta_24_copy.rename(columns={'CONTORNO                                                                                                                                                                                                                                                                                                                   (Ano 2024)': 'CONTORNO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_meta_24_copy.rename(columns={'FX ADICIONAL                                                                                                                                                                                                                                                                                                     (Ano 2024)': 'FX ADICIONAL - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_meta_24_copy.rename(columns={'TERCEIRA FAIXA                                                                                                                                                                                                                                                                                         (Ano 2024)': 'TERCEIRA FAIXA - (Acumulada até ano anterior) - Descrição'}, inplace=True)

df_meta_24_copy.rename(columns={'RISCOS (Pavimentação)       ': 'RISCOS (Pavimentação) - SITUAÇÃO'}, inplace=True)
df_meta_24_copy.rename(columns={'RISCOS (Duplicação)       ': 'RISCOS (Duplicação) - SITUAÇÃO'}, inplace=True)
df_meta_24_copy.rename(columns={'RISCOS (OAE)       ': 'RISCOS (OAE) - SITUAÇÃO'}, inplace=True)
df_meta_24_copy.rename(columns={'RISCOS (Contorno)       ': 'RISCOS (Contorno) - SITUAÇÃO'}, inplace=True)
df_meta_24_copy.rename(columns={'RISCOS (FX Adicional)       ': 'RISCOS (Faixa Adicional) - SITUAÇÃO'}, inplace=True)
df_meta_24_copy.rename(columns={'RISCOS (Terceira Faixa)       ': 'RISCOS (Terceira Faixa) - SITUAÇÃO'}, inplace=True)

In [ ]:
new_column_names = ['PAVIMENTAÇÃO - (Acumulada até ano anterior) - km (i)', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - km (f)', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Ext. (km)', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - (km)% PLAN', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - (km)% EXEC', 
 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - FINANCEIRO (R$)',
 'RISCOS (Pavimentação) - SITUAÇÃO',
 'RISCOS (Pavimentação) - INTERFERÊNCIA',
 'RISCOS (Pavimentação) - PRINCIPAL',
 'RISCOS (Pavimentação) - NÍVEL DE ALERTA',
 'DUPLICAÇÃO - (Acumulada até ano anterior) - Descrição',
 'DUPLICAÇÃO - (Acumulada até ano anterior) - km (i)', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - km (f)', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - Ext. (km)', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - (km)% PLAN', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - (km)% EXEC', 
 'DUPLICAÇÃO - (Acumulada até ano anterior) - FINANCEIRO (R$)',
 'RISCOS (Duplicação) - SITUAÇÃO',
 'RISCOS (Duplicação) - INTERFERÊNCIA',
 'RISCOS (Duplicação) - PRINCIPAL',
 'RISCOS (Duplicação) - NÍVEL DE ALERTA',
 'OAE - (Acumulada até ano anterior) - Descrição',
 'OAE - (Acumulada até ano anterior) - km (i)', 
 'OAE - (Acumulada até ano anterior) - km (f)', 
 'OAE - (Acumulada até ano anterior) - Ext. (km)', 
 'OAE - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'OAE - (Acumulada até ano anterior) - (km)% PLAN', 
 'OAE - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'OAE - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'OAE - (Acumulada até ano anterior) - (km)% EXEC', 
 'OAE - (Acumulada até ano anterior) - FINANCEIRO (R$)',
 'RISCOS (OAE) - SITUAÇÃO',
 'RISCOS (OAE) - INTERFERÊNCIA',
 'RISCOS (OAE) - PRINCIPAL',
 'RISCOS (OAE) - NÍVEL DE ALERTA',
 'CONTORNO - (Acumulada até ano anterior) - Descrição',
 'CONTORNO - (Acumulada até ano anterior) - km (i)', 
 'CONTORNO - (Acumulada até ano anterior) - km (f)', 
 'CONTORNO - (Acumulada até ano anterior) - Ext. (km)', 
 'CONTORNO - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'CONTORNO - (Acumulada até ano anterior) - (km)% PLAN', 
 'CONTORNO - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'CONTORNO - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'CONTORNO - (Acumulada até ano anterior) - (km)% EXEC',
 'CONTORNO - (Acumulada até ano anterior) - FINANCEIRO (R$)',
 'RISCOS (Contorno) - SITUAÇÃO',
 'RISCOS (Contorno) - INTERFERÊNCIA',
 'RISCOS (Contorno) - PRINCIPAL',
 'RISCOS (Contorno) - NÍVEL DE ALERTA',
 'FX ADICIONAL - (Acumulada até ano anterior) - Descrição',
 'FX ADICIONAL - (Acumulada até ano anterior) - km (i)',
 'FX ADICIONAL - (Acumulada até ano anterior) - km (f)', 
 'FX ADICIONAL - (Acumulada até ano anterior) - Ext. (km)', 
 'FX ADICIONAL - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'FX ADICIONAL - (Acumulada até ano anterior) - (km)% PLAN', 
 'FX ADICIONAL - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'FX ADICIONAL - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'FX ADICIONAL - (Acumulada até ano anterior) - (km)% EXEC', 
 'FX ADICIONAL - (Acumulada até ano anterior) - FINANCEIRO (R$)',
 'FX ADICIONAL - (Acumulada até ano anterior) - Descrição',
 'RISCOS (Faixa Adicional) - SITUAÇÃO',
 'RISCOS (Faixa Adicional) - INTERFERÊNCIA',
 'RISCOS (Faixa Adicional) - PRINCIPAL',
 'RISCOS (Faixa Adicional) - NÍVEL DE ALERTA',
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - km (i)', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - km (f)', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - Ext. (km)', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - PERCENTUAL (%) PLAN', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - (km)% PLAN', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - FINANCEIRO PLAN (R$)', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - PERCENTUAL (%) EXEC', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - (km)% EXEC', 
 'TERCEIRA FAIXA - (Acumulada até ano anterior) - FINANCEIRO (R$)',
 'RISCOS (Terceira Faixa) - SITUAÇÃO',
 'RISCOS (Terceira Faixa) - INTERFERÊNCIA',
 'RISCOS (Terceira Faixa) - PRINCIPAL',
 'RISCOS (Terceira Faixa) - NÍVEL DE ALERTA']


In [ ]:
# Criar um dicionário que mapeia 'Unnamed: x' para os novos nomes
rename_dict = {'Unnamed: {}'.format(i): name for i, name in enumerate(new_column_names, start=178)}

# Usar o método rename para renomear as colunas
df_meta_24_copy.rename(columns=rename_dict, inplace=True)

In [ ]:
# Identifica colunas que possuem as strings 'RISCOS' ou 'OBSERVAÇÕES' em seu nome
columns_to_drop = [col for col in df_meta_24_copy.columns if 'RISCOS' in col or 'OBSERVAÇÕES' in col]

# Remove as colunas identificadas
df_meta_24_copy = df_meta_24_copy.drop(columns=columns_to_drop)

In [ ]:
df_meta_24_copy.columns

In [ ]:
df_meta_24_copy.dropna(subset=['ESTADO/LOTE'])

In [ ]:
columns_to_fill = ['EMPREENDIMENTO']
df_meta_24_copy[columns_to_fill] = df_meta_24_copy.groupby((df_meta_24_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [ ]:
df_meta_24_copy['ESTADO/LOTE'] = df_meta_24_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS',
       'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO',
       'MATO GROSSO', 'PARÁ', 'RIO/SP', 'SÃO PAULO', 'PARANÁ', 'BAHIA',
       'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)

# Agora preenchemos para baixo até encontrar o próximo valor não-NaN (isto é, a próxima 'Situação')
df_meta_24_copy['ESTADO/LOTE'] = df_meta_24_copy['ESTADO/LOTE'].fillna(method='ffill')

In [ ]:
df_meta_24_copy = df_meta_24_copy[~df_meta_24_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]

In [ ]:
df_meta_24_copy.columns

In [ ]:
df_meta_24_copy.index = range(1, len(df_meta_24_copy) + 1)
df_meta_24_copy.index.name = 'ID-ÚNICO'
df_meta_24_copy = df_meta_24_copy.reset_index()

In [ ]:
df_meta_24_raw = df_meta_24_copy

In [ ]:
# df_meta_24_copy.to_excel('Dados Gerados/RAW_META(2024).xlsx', index=False)

In [ ]:
colunas_manter = ['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO']

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_meta_24_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_meta_24_copy = df_meta_24_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [ ]:
df_meta_24_copy['EMPREENDIMENTO'].unique()

In [ ]:
df_meta_24_copy = df_meta_24_copy.loc[df_meta_24_copy['Valor'] != 0]
df_meta_24_copy = df_meta_24_copy.dropna(subset=['Valor'])

In [ ]:
df_meta_24_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_meta_24_copy['Atributo'].str.split(' - ', expand=True)

In [ ]:
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_meta_24_copy = df_meta_24_copy[['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [ ]:
df_meta_24_copy

In [ ]:
'''df_meta_24_copy['Atributo.1'] = df_meta_24_copy['Atributo.1'].apply(
    lambda x: x.replace('CONTORNO (Ano Vigente)', 'CONTORNO')
              .replace('DUPLICAÇÃO (Ano Vigente)', 'DUPLICAÇÃO')
              .replace('FX ADICIONAL (Ano Vigente)', 'FX ADICIONAL')
              .replace('OAE (Ano Vigente)', 'OAE')
              .replace('PAVIMENTAÇÃO (Ano Vigente)', 'PAVIMENTAÇÃO')
              .replace('TERCEIRA FAIXA (Ano Vigente)', 'TERCEIRA FAIXA')
)'''

In [ ]:
# Converter colunas para 'category'
df_meta_24_copy['SETOR'] = df_meta_24_copy['SETOR'].astype('category')
df_meta_24_copy['UF'] = df_meta_24_copy['UF'].astype('category')
df_meta_24_copy['PROPONENTE'] = df_meta_24_copy['PROPONENTE'].astype('category')
df_meta_24_copy['EXECUTOR (Grupo Controlador)'] = df_meta_24_copy['EXECUTOR (Grupo Controlador)'].astype('category')
df_meta_24_copy['ESTRUTURADOR DO PROJETO'] = df_meta_24_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_meta_24_copy['Atributo.1'] = df_meta_24_copy['Atributo.1'].astype('category')
df_meta_24_copy['Atributo.2'] = df_meta_24_copy['Atributo.2'].astype('category')

# Converter colunas para 'string'
df_meta_24_copy['EMPREENDIMENTO'] = df_meta_24_copy['EMPREENDIMENTO'].astype('string') 


In [ ]:
df_meta_24_copy = df_meta_24_copy.sort_values(by='ID-ÚNICO', ascending=True)

In [ ]:
df_meta_24_copy.to_excel('Dados Gerados/META(2024).xlsx', index=False, sheet_name= 'META')

---

# Exec(22) + Meta(23) + Meta(24)

In [ ]:
# Suponha que df1 e df2 sejam seus DataFrames
df_22_23_24 = pd.concat([df_plan_exec_ano_ant_copy, df_meta_23_copy, df_meta_24_copy], axis=0)

# Resetando o índice do DataFrame resultante para evitar índices duplicados
df_22_23_24 = df_22_23_24.reset_index(drop=True)

In [ ]:
df_22_23_24 = df_22_23_24.sort_values(by='ID-ÚNICO', ascending=True)

In [ ]:
df_22_23_24.to_excel('Dados Gerados/PLAN_EXEC ATÉ ANO ANTERIOR (24).xlsx', index=False)

---

In [ ]:
def process_and_concat_dfs(df_plan_exec_ano_ant_raw, df_meta_23_raw, df_meta_24_raw):
    def process_df(df):
        # Drop na coluna 'ID-ÚNICO'
        if 'ID-ÚNICO' in df.columns:
            df.drop(columns=['ID-ÚNICO'], inplace=True)
        
        # Reset index e renomear índice
        df.index = range(1, len(df) + 1)
        df.index.name = 'ID-ÚNICO'
        df = df.reset_index()
        df = df.sort_values(by='ID-ÚNICO', ascending=True)

        # Definir os novos nomes das colunas a partir do índice 9
        new_column_names = ['PAVIMENTAÇÃO - Descrição',
                            'PAVIMENTAÇÃO - km(i)', 
                            'PAVIMENTAÇÃO - km(f)', 
                            'PAVIMENTAÇÃO - Ext.(km)', 
                            'PAVIMENTAÇÃO - PERCENTUAL (%) PLAN', 
                            'PAVIMENTAÇÃO - (km)% PLAN', 
                            'PAVIMENTAÇÃO - FINANCEIRO PLAN (R$)', 
                            'PAVIMENTAÇÃO - PERCENTUAL (%) EXEC', 
                            'PAVIMENTAÇÃO - (km)% EXEC', 
                            'PAVIMENTAÇÃO - FINANCEIRO(R$)',
                            'DUPLICAÇÃO - Descrição',
                            'DUPLICAÇÃO - km(i)', 
                            'DUPLICAÇÃO - km(f)', 
                            'DUPLICAÇÃO - Ext.(km)', 
                            'DUPLICAÇÃO - PERCENTUAL (%) PLAN', 
                            'DUPLICAÇÃO - (km)% PLAN', 
                            'DUPLICAÇÃO - FINANCEIRO PLAN (R$)', 
                            'DUPLICAÇÃO - PERCENTUAL (%) EXEC', 
                            'DUPLICAÇÃO - (km)% EXEC', 
                            'DUPLICAÇÃO - FINANCEIRO(R$)',
                            'OAE - Descrição',
                            'OAE - km(i)', 
                            'OAE - km(f)', 
                            'OAE - Ext.(km)', 
                            'OAE - PERCENTUAL (%) PLAN', 
                            'OAE - (km)% PLAN', 
                            'OAE - FINANCEIRO PLAN (R$)', 
                            'OAE - PERCENTUAL (%) EXEC', 
                            'OAE - (km)% EXEC', 
                            'OAE - FINANCEIRO(R$)',
                            'CONTORNO - Descrição',
                            'CONTORNO - km(i)', 
                            'CONTORNO - km(f)', 
                            'CONTORNO - Ext.(km)', 
                            'CONTORNO - PERCENTUAL (%) PLAN', 
                            'CONTORNO - (km)% PLAN', 
                            'CONTORNO - FINANCEIRO PLAN (R$)', 
                            'CONTORNO - PERCENTUAL (%) EXEC', 
                            'CONTORNO - (km)% EXEC',
                            'CONTORNO - FINANCEIRO(R$)',
                            'FX ADICIONAL - Descrição',
                            'FX ADICIONAL - km(i)',
                            'FX ADICIONAL - km(f)', 
                            'FX ADICIONAL - Ext.(km)', 
                            'FX ADICIONAL - PERCENTUAL (%) PLAN', 
                            'FX ADICIONAL - (km)% PLAN', 
                            'FX ADICIONAL - FINANCEIRO PLAN (R$)', 
                            'FX ADICIONAL - PERCENTUAL (%) EXEC', 
                            'FX ADICIONAL - (km)% EXEC', 
                            'FX ADICIONAL - FINANCEIRO(R$)',
                            'TERCEIRA FAIXA - Descrição',
                            'TERCEIRA FAIXA - km(i)', 
                            'TERCEIRA FAIXA - km(f)', 
                            'TERCEIRA FAIXA - Ext.(km)', 
                            'TERCEIRA FAIXA - PERCENTUAL (%) PLAN', 
                            'TERCEIRA FAIXA - (km)% PLAN', 
                            'TERCEIRA FAIXA - FINANCEIRO PLAN (R$)', 
                            'TERCEIRA FAIXA - PERCENTUAL (%) EXEC', 
                            'TERCEIRA FAIXA - (km)% EXEC', 
                            'TERCEIRA FAIXA - FINANCEIRO(R$)']

        # Renomear colunas a partir do índice 9
        rename_dict = {df.columns[i]: name for i, name in enumerate(new_column_names, start=9) if i < len(df.columns)}
        df.rename(columns=rename_dict, inplace=True)

        # Manter colunas que contêm estas keywords
        keywords_to_keep = [
            'ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO', 
            'Descrição', 'Ext.(km)', 'FINANCEIRO PLAN (R$)', 'FINANCEIRO(R$)', 
            '(km)% PLAN', '(km)% EXEC'
        ]
        columns_to_keep = [col for col in df.columns if any(keyword in col for keyword in keywords_to_keep)]
        df = df[columns_to_keep]

        # Remover linhas onde todas as colunas a partir da posição 3 estão vazias
        cols_a_verificar = df.iloc[:, 3:]
        df = df.dropna(subset=cols_a_verificar.columns, how='all')

        # Preparar para unpivot
        df1 = df[['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO']]
        colunas_manter = ['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO']
        colunas_unpivot = df.columns.difference(colunas_manter)

        # Unpivot (melt)
        df = df.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                     var_name='Atributo', value_name='Valor')

        # Adicionar coluna 'SETOR' fixa (como no exemplo)
        df['SETOR'] = 'Rodoviário'

        # Dividir a coluna 'Atributo' em duas
        df[['Atributo.1', 'Atributo.2']] = df['Atributo'].str.split(' - ', expand=True)

        # Selecionar colunas relevantes
        df = df[['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO', 'Atributo.1', 'Atributo.2', 'Valor']]

        # Remover valores nulos e zeros em 'Valor'
        df = df.dropna(subset=['Valor'])
        df = df.loc[df['Valor'] != 0]

        # Pivotar novamente
        df = df.pivot_table(index=['ID-ÚNICO', 'Atributo.1'],
                            columns='Atributo.2', values='Valor', aggfunc='first').reset_index()

        # Remover coluna de índice indesejada, se existir
        if 'Atributo.2' in df.columns:
            df = df.drop(columns='Atributo.2')

        # Merge com df1 para recuperar colunas originais
        df = pd.merge(df1, df, on='ID-ÚNICO', how='inner')

        # Remover linhas com valores nulos em 'Atributo.1'
        df = df.dropna(subset=['Atributo.1'])

        # Selecionar e ordenar colunas finais
        final_cols = ['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO', 'Atributo.1', 
                      'Descrição', 'Ext.(km)', 'FINANCEIRO PLAN (R$)', 'FINANCEIRO(R$)', 
                      '(km)% PLAN', '(km)% EXEC']
        df = df[final_cols]

        # Renomear colunas para evitar conflitos futuros
        df.rename(columns={
            'ID-ÚNICO': 'ID-ÚNICO2',
            'SETOR': 'SETOR2',
            'EMPREENDIMENTO': 'EMPREENDIMENTO2',
            'Descrição': 'Descrição2',
            'Ext.(km)': 'Ext.(km)2',
            'FINANCEIRO PLAN (R$)': 'FINANCEIRO PLAN (R$)2',
            'FINANCEIRO(R$)': 'FINANCEIRO(R$)2',
            '(km)% PLAN': '(km)% PLAN2',
            '(km)% EXEC': '(km)% EXEC2'
        }, inplace=True)

        return df

    # Processar cada df
    df1_processed = process_df(df_plan_exec_ano_ant_raw.copy())
    df2_processed = process_df(df_meta_23_raw.copy())
    df3_processed = process_df(df_meta_24_raw.copy())

    # Concatenar os DataFrames processados
    df_concat = pd.concat([df1_processed, df2_processed, df3_processed], axis=0)
    df_concat = df_concat.reset_index(drop=True)
    df_concat = df_concat.sort_values(by='ID-ÚNICO2', ascending=True)

    # Salvar resultado
    df_concat.to_excel('PLAN_EXEC ATÉ ANO ANTERIOR (24)(2).xlsx', index=False)

    return df_concat

In [ ]:
df_meta_24_raw

In [ ]:
print("Processando a planilha 'META(2025)' para gerar o arquivo '2025(2).xlsx'...")

# ==============================================================================
# 1. CARREGAMENTO E LIMPEZA INICIAL
# ==============================================================================

# Carrega os dados da planilha pulando as linhas de cabeçalho
df_meta_25_copy = pd.read_excel(file_path, sheet_name='META(2025)', header=2)
# Remove as primeiras linhas que são sub-cabeçalhos ou vazias
df_meta_25_copy = df_meta_25_copy.drop([0, 1, 2], axis=0)
# Remove colunas 'Unnamed' que são criadas para colunas sem nome no Excel
df_meta_25_raw = df_meta_25_copy.loc[:, ~df_meta_25_copy.columns.str.contains('^Unnamed')]
# Remove linhas de totais que podem atrapalhar a análise
df_meta_25_raw = df_meta_25_raw[~df_meta_25_raw.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA', case=False, na=False)).any(axis=1)]
# Remove quaisquer linhas que estejam completamente vazias
df_meta_25_raw.dropna(how='all', inplace=True)


In [ ]:
# Cria um ID único para cada linha para podermos rastreá-las após as transformações
df_meta_25_raw.index = range(1, len(df_meta_25_raw) + 1)
df_meta_25_raw.index.name = 'ID-ÚNICO'
df_meta_25_raw = df_meta_25_raw.reset_index()

# Guarda as colunas de identificação que usaremos no final para o merge
info_cols_df = df_meta_25_raw[['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO']].copy()

# Identifica as colunas de ID de forma automática para a transformação
id_vars = [col for col in df_meta_25_raw.columns if not any(keyword in str(col) for keyword in ['PAVIMENTAÇÃO', 'DUPLICAÇÃO', 'OAE', 'CONTORNO', 'FX ADICIONAL', 'TERCEIRA FAIXA'])]

# Transforma a tabela "larga" em "longa". Este passo é crucial para a robustez.
df_long = df_meta_25_raw.melt(id_vars=id_vars, var_name='AtributoOriginal', value_name='Valor')

In [ ]:
# Se houver dados para processar, continua
if not df_long.empty:
    # Extrai de forma inteligente o "Tipo de Obra" e a "Métrica" do nome da coluna original
    df_long[['Atributo.1', 'Atributo.2']] = df_long['AtributoOriginal'].str.extract(r'([A-Z\s\/]+[A-Z])\s*\((.+)\)')
    df_long['Atributo.1'] = df_long['Atributo.1'].str.strip()
    df_long['Atributo.2'] = df_long['Atributo.2'].str.strip()

    # Transforma a tabela de volta para um formato com as métricas como colunas
    df_pivot = df_long.pivot_table(
        index=['ID-ÚNICO', 'Atributo.1'],
        columns='Atributo.2',
        values='Valor',
        aggfunc='first'
    ).reset_index()

    # Junta as informações de identificação de volta no DataFrame final
    df_final = pd.merge(info_cols_df, df_pivot, on='ID-ÚNICO', how='inner')

    # ==============================================================================
    # 4. SELEÇÃO FINAL, RENOMEAÇÃO E EXPORTAÇÃO
    # ==============================================================================

    # Seleciona de forma segura apenas as colunas que você quer no resultado final
    # Isso evita o erro 'KeyError' se uma coluna não tiver dados e não for criada
    colunas_desejadas = [
        'ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO', 'Atributo.1',
        'Descrição', 'Ext.(km)', 'FINANCEIRO PLAN (R$)', 'FINANCEIRO(R$)',
        '(km)% PLAN', '(km)% EXEC'
    ]
    colunas_finais_existentes = [col for col in colunas_desejadas if col in df_final.columns]
    df_final = df_final[colunas_finais_existentes]

    # Renomeia as colunas com o sufixo "2", como no seu código original
    rename_final = {col: col.replace('ID-ÚNICO', 'ID-ÚNICO2')
                         .replace('SETOR', 'SETOR2')
                         .replace('EMPREENDIMENTO', 'EMPREENDIMENTO2')
                         .replace('Descrição', 'Descrição2')
                         .replace('Ext.(km)', 'Ext.(km)2')
                         .replace('FINANCEIRO PLAN (R$)', 'FINANCEIRO PLAN (R$)2')
                         .replace('FINANCEIRO(R$)', 'FINANCEIRO(R$)2')
                         .replace('(km)% PLAN', '(km)% PLAN2')
                         .replace('(km)% EXEC', '(km)% EXEC2') 
                   for col in df_final.columns}
    df_final.rename(columns=rename_final, inplace=True)

    # Salva o arquivo final com o nome desejado
    output_filename = '2025(2).xlsx'
    df_final.to_excel(output_filename, sheet_name='2025(2)', index=False)
    print(f"\nProcesso concluído! Arquivo '{output_filename}' foi gerado com sucesso.")

else:
    print("\nAVISO: Nenhum dado válido encontrado em 'META(2025)' para gerar o arquivo final.")


<h1><b><i>'META(2025)'</b></i></h1>

In [ ]:
# Primeiro cria um alias
df_meta_25 = df_meta_2025

#CHECKPOINT
df_meta_25_copy = df_meta_25.copy()

In [ ]:
df_meta_25_copy = pd.read_excel(file_path, sheet_name='META(2025)', header=2)

In [ ]:
df_meta_25_copy

In [ ]:
df_meta_25_copy = df_meta_25_copy.drop([0, 1, 2], axis = 0)

In [ ]:
df_meta_25_copy.columns

In [ ]:
# Drop na coluna 'Região' que não é usada.
df_meta_25_copy.drop(columns=['Região'], inplace=True)

In [ ]:
# Removendo colunas
df_meta_25_copy = df_meta_25_copy.drop(df_meta_25_copy.columns[8:176], axis=1)

In [ ]:
# Renomear Colunas

df_meta_25_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR (Grupo Controlador)'}, inplace=True)
df_meta_25_copy.rename(columns={'PAVIMENTAÇÃO                                                                                                                                                                                                                                                                                     (Ano Vigente)': 'PAVIMENTAÇÃO - (Ano Vigente) - Descrição'}, inplace=True)
df_meta_25_copy.rename(columns={'DUPLICAÇÃO                                                                                                                                                                                                                                                                                               (Ano Vigente)': 'DUPLICAÇÃO - (Ano Vigente) - Descrição'}, inplace=True)
df_meta_25_copy.rename(columns={'OAE                                                                                                                                                                                                                                                                                                               (Ano Vigente)': 'OAE - (Ano Vigente) - Descrição'}, inplace=True)
df_meta_25_copy.rename(columns={'CONTORNO                                                                                                                                                                                                                                                                                                                   (Ano Vigente)': 'CONTORNO - (Ano Vigente) - Descrição'}, inplace=True)
df_meta_25_copy.rename(columns={'FX ADICIONAL                                                                                                                                                                                                                                                                                                     (Ano Vigente)': 'FX ADICIONAL - (Ano Vigente) - Descrição'}, inplace=True)
df_meta_25_copy.rename(columns={'TERCEIRA FAIXA                                                                                                                                                                                                                                                                                         (Ano Vigente)': 'TERCEIRA FAIXA - (Ano Vigente) - Descrição'}, inplace=True)

df_meta_25_copy.rename(columns={'RISCOS (Pavimentação)       ': 'RISCOS (Pavimentação) - SITUAÇÃO'}, inplace=True)
df_meta_25_copy.rename(columns={'RISCOS (Duplicação)       ': 'RISCOS (Duplicação) - SITUAÇÃO'}, inplace=True)
df_meta_25_copy.rename(columns={'RISCOS (OAE)       ': 'RISCOS (OAE) - SITUAÇÃO'}, inplace=True)
df_meta_25_copy.rename(columns={'RISCOS (Contorno)       ': 'RISCOS (Contorno) - SITUAÇÃO'}, inplace=True)
df_meta_25_copy.rename(columns={'RISCOS (FX Adicional)       ': 'RISCOS (Faixa Adicional) - SITUAÇÃO'}, inplace=True)
df_meta_25_copy.rename(columns={'RISCOS (Terceira Faixa)       ': 'RISCOS (Terceira Faixa) - SITUAÇÃO'}, inplace=True)

In [ ]:
new_column_names = ['PAVIMENTAÇÃO - (Ano Vigente) - km (i)', 
 'PAVIMENTAÇÃO - (Ano Vigente) - km (f)', 
 'PAVIMENTAÇÃO - (Ano Vigente) - Ext. (km)', 
 'PAVIMENTAÇÃO - (Ano Vigente) - PERCENTUAL (%) PLAN', 
 'PAVIMENTAÇÃO - (Ano Vigente) - (km)% PLAN', 
 'PAVIMENTAÇÃO - (Ano Vigente) - FINANCEIRO PLAN (R$)', 
 'PAVIMENTAÇÃO - (Ano Vigente) - PERCENTUAL (%) EXEC', 
 'PAVIMENTAÇÃO - (Ano Vigente) - (km)% EXEC', 
 'PAVIMENTAÇÃO - (Ano Vigente) - FINANCEIRO (R$)',
 'RISCOS (Pavimentação) - SITUAÇÃO',
 'RISCOS (Pavimentação) - INTERFERÊNCIA',
 'RISCOS (Pavimentação) - PRINCIPAL',
 'RISCOS (Pavimentação) - NÍVEL DE ALERTA',
 'RISCOS (Pavimentação) - Nº PROCESSO',
 'RISCOS (Pavimentação) - DATA DO PROTOCOLO',
 'RISCOS (Pavimentação) - LICENÇA',
 'RISCOS (Pavimentação) - SITUAÇÃO GERAL',
 'DUPLICAÇÃO - (Ano Vigente) - Descrição',
 'DUPLICAÇÃO - (Ano Vigente) - km (i)', 
 'DUPLICAÇÃO - (Ano Vigente) - km (f)', 
 'DUPLICAÇÃO - (Ano Vigente) - Ext. (km)', 
 'DUPLICAÇÃO - (Ano Vigente) - PERCENTUAL (%) PLAN', 
 'DUPLICAÇÃO - (Ano Vigente) - (km)% PLAN', 
 'DUPLICAÇÃO - (Ano Vigente) - FINANCEIRO PLAN (R$)', 
 'DUPLICAÇÃO - (Ano Vigente) - PERCENTUAL (%) EXEC', 
 'DUPLICAÇÃO - (Ano Vigente) - (km)% EXEC', 
 'DUPLICAÇÃO - (Ano Vigente) - FINANCEIRO (R$)',
 'RISCOS (Duplicação) - SITUAÇÃO',
 'RISCOS (Duplicação) - INTERFERÊNCIA',
 'RISCOS (Duplicação) - PRINCIPAL',
 'RISCOS (Duplicação) - NÍVEL DE ALERTA',
 'RISCOS (Duplicação) - Nº PROCESSO',
 'RISCOS (Duplicação) - DATA DO PROTOCOLO',
 'RISCOS (Duplicação) - LICENÇA',
 'RISCOS (Duplicação) - SITUAÇÃO GERAL',
 'OAE - (Ano Vigente) - Descrição',
 'OAE - (Ano Vigente) - km (i)', 
 'OAE - (Ano Vigente) - km (f)', 
 'OAE - (Ano Vigente) - Ext. (km)', 
 'OAE - (Ano Vigente) - PERCENTUAL (%) PLAN', 
 'OAE - (Ano Vigente) - (km)% PLAN', 
 'OAE - (Ano Vigente) - FINANCEIRO PLAN (R$)', 
 'OAE - (Ano Vigente) - PERCENTUAL (%) EXEC', 
 'OAE - (Ano Vigente) - (km)% EXEC', 
 'OAE - (Ano Vigente) - FINANCEIRO (R$)',
 'RISCOS (OAE) - SITUAÇÃO',
 'RISCOS (OAE) - INTERFERÊNCIA',
 'RISCOS (OAE) - PRINCIPAL',
 'RISCOS (OAE) - NÍVEL DE ALERTA',
 'CONTORNO - (Ano Vigente) - Descrição',
 'CONTORNO - (Ano Vigente) - km (i)', 
 'CONTORNO - (Ano Vigente) - km (f)', 
 'CONTORNO - (Ano Vigente) - Ext. (km)', 
 'CONTORNO - (Ano Vigente) - PERCENTUAL (%) PLAN', 
 'CONTORNO - (Ano Vigente) - (km)% PLAN', 
 'CONTORNO - (Ano Vigente) - FINANCEIRO PLAN (R$)', 
 'CONTORNO - (Ano Vigente) - PERCENTUAL (%) EXEC', 
 'CONTORNO - (Ano Vigente) - (km)% EXEC',
 'CONTORNO - (Ano Vigente) - FINANCEIRO (R$)',
 'RISCOS (Contorno) - SITUAÇÃO',
 'RISCOS (Contorno) - INTERFERÊNCIA',
 'RISCOS (Contorno) - PRINCIPAL',
 'RISCOS (Contorno) - NÍVEL DE ALERTA',
 'RISCOS (Contorno) - Nº PROCESSO',
 'RISCOS (Contorno) - DATA DO PROTOCOLO',
 'RISCOS (Contorno) - LICENÇA',
 'RISCOS (Contorno) - SITUAÇÃO GERAL',
 'FX ADICIONAL - (Ano Vigente) - Descrição',
 'FX ADICIONAL - (Ano Vigente) - km (i)',
 'FX ADICIONAL - (Ano Vigente) - km (f)', 
 'FX ADICIONAL - (Ano Vigente) - Ext. (km)', 
 'FX ADICIONAL - (Ano Vigente) - PERCENTUAL (%) PLAN', 
 'FX ADICIONAL - (Ano Vigente) - (km)% PLAN', 
 'FX ADICIONAL - (Ano Vigente) - FINANCEIRO PLAN (R$)', 
 'FX ADICIONAL - (Ano Vigente) - PERCENTUAL (%) EXEC', 
 'FX ADICIONAL - (Ano Vigente) - (km)% EXEC', 
 'FX ADICIONAL - (Ano Vigente) - FINANCEIRO (R$)',
 'FX ADICIONAL - (Ano Vigente) - Descrição',
 'RISCOS (Faixa Adicional) - SITUAÇÃO',
 'RISCOS (Faixa Adicional) - INTERFERÊNCIA',
 'RISCOS (Faixa Adicional) - PRINCIPAL',
 'RISCOS (Faixa Adicional) - NÍVEL DE ALERTA',
 'RISCOS (Faixa Adicional) - Nº PROCESSO',
 'RISCOS (Faixa Adicional) - DATA DO PROTOCOLO',
 'RISCOS (Faixa Adicional) - LICENÇA',
 'RISCOS (Faixa Adicional) - SITUAÇÃO GERAL',
 'TERCEIRA FAIXA - (Ano Vigente) - km (i)', 
 'TERCEIRA FAIXA - (Ano Vigente) - km (f)', 
 'TERCEIRA FAIXA - (Ano Vigente) - Ext. (km)', 
 'TERCEIRA FAIXA - (Ano Vigente) - PERCENTUAL (%) PLAN', 
 'TERCEIRA FAIXA - (Ano Vigente) - (km)% PLAN', 
 'TERCEIRA FAIXA - (Ano Vigente) - FINANCEIRO PLAN (R$)', 
 'TERCEIRA FAIXA - (Ano Vigente) - PERCENTUAL (%) EXEC', 
 'TERCEIRA FAIXA - (Ano Vigente) - (km)% EXEC', 
 'TERCEIRA FAIXA - (Ano Vigente) - FINANCEIRO (R$)',
 'RISCOS (Terceira Faixa) - SITUAÇÃO',
 'RISCOS (Terceira Faixa) - INTERFERÊNCIA',
 'RISCOS (Terceira Faixa) - PRINCIPAL',
 'RISCOS (Terceira Faixa) - NÍVEL DE ALERTA'
 'RISCOS (Terceira Faixa) - Nº PROCESSO',
 'RISCOS (Terceira Faixa) - DATA DO PROTOCOLO',
 'RISCOS (Terceira Faixa) - LICENÇA',
 'RISCOS (Terceira Faixa) - SITUAÇÃO GERAL'
 'OBSERVAÇÕES - SITUAÇÃO GERAL']  


In [ ]:
# Criar um dicionário que mapeia 'Unnamed: x' para os novos nomes
rename_dict = {'Unnamed: {}'.format(i): name for i, name in enumerate(new_column_names, start=178)}

# Usar o método rename para renomear as colunas
df_meta_25_copy.rename(columns=rename_dict, inplace=True)

In [ ]:
# Identifica colunas que possuem as strings 'RISCOS' ou 'OBSERVAÇÕES' em seu nome
columns_to_drop = [col for col in df_meta_25_copy.columns if 'RISCOS' in col or 'OBSERVAÇÕES' in col]

# Remove as colunas identificadas
df_meta_25_copy = df_meta_25_copy.drop(columns=columns_to_drop)

In [ ]:
df_meta_25_copy.columns

In [ ]:
df_meta_25_copy.dropna(subset=['ESTADO/LOTE'])

In [ ]:
columns_to_fill = ['EMPREENDIMENTO']
df_meta_25_copy[columns_to_fill] = df_meta_25_copy.groupby((df_meta_25_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [ ]:
df_meta_25_copy['ESTADO/LOTE'] = df_meta_25_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS',
       'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO',
       'MATO GROSSO', 'PARÁ', 'RIO/SP', 'SÃO PAULO', 'PARANÁ', 'BAHIA',
       'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)

# Agora preenchemos para baixo até encontrar o próximo valor não-NaN (isto é, a próxima 'Situação')
df_meta_25_copy['ESTADO/LOTE'] = df_meta_25_copy['ESTADO/LOTE'].fillna(method='ffill')

In [ ]:
df_meta_25_copy = df_meta_25_copy[~df_meta_25_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]

In [ ]:
df_meta_25_copy.columns

In [ ]:
df_meta_25_copy.index = range(1, len(df_meta_25_copy) + 1)
df_meta_25_copy.index.name = 'ID-ÚNICO'
df_meta_25_copy = df_meta_25_copy.reset_index()

In [ ]:
df_meta_25_raw = df_meta_25_copy

In [ ]:
# df_meta_25_copy.to_excel('Dados Gerados/RAW_META(2025).xlsx', index=False)

In [ ]:
colunas_manter = ['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO']

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_meta_25_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_meta_25_copy = df_meta_25_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [ ]:
df_meta_25_copy['EMPREENDIMENTO'].unique()

In [ ]:
df_meta_25_copy = df_meta_25_copy.loc[df_meta_25_copy['Valor'] != 0]
df_meta_25_copy = df_meta_25_copy.dropna(subset=['Valor'])

In [ ]:
df_meta_25_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_meta_25_copy['Atributo'].str.split(' - ', expand=True)

In [ ]:
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_meta_25_copy = df_meta_25_copy[['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [ ]:
# Converter colunas para 'category'
df_meta_25_copy['SETOR'] = df_meta_25_copy['SETOR'].astype('category')
df_meta_25_copy['UF'] = df_meta_25_copy['UF'].astype('category')
df_meta_25_copy['PROPONENTE'] = df_meta_25_copy['PROPONENTE'].astype('category')
df_meta_25_copy['EXECUTOR (Grupo Controlador)'] = df_meta_25_copy['EXECUTOR (Grupo Controlador)'].astype('category')
df_meta_25_copy['ESTRUTURADOR DO PROJETO'] = df_meta_25_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_meta_25_copy['Atributo.1'] = df_meta_25_copy['Atributo.1'].astype('category')
df_meta_25_copy['Atributo.2'] = df_meta_25_copy['Atributo.2'].astype('category')

# Converter colunas para 'string'
df_meta_25_copy['EMPREENDIMENTO'] = df_meta_25_copy['EMPREENDIMENTO'].astype('string') 


In [ ]:
df_meta_25_copy = df_meta_25_copy.sort_values(by='ID-ÚNICO', ascending=True)

In [ ]:
df_meta_25_copy.to_excel('Dados Gerados/META(2025).xlsx', index=False, sheet_name= 'META')

---

In [ ]:
df_meta_25_raw

In [ ]:
# Drop na coluna 'Região' que não é usada.
df_meta_25_raw.drop(columns=['ID-ÚNICO'], inplace=True)

In [ ]:
df_meta_25_raw.index = range(1, len(df_meta_25_raw) + 1)
df_meta_25_raw.index.name = 'ID-ÚNICO'
df_meta_25_raw = df_meta_25_raw.reset_index()

In [ ]:
df_meta_25_raw = df_meta_25_raw.sort_values(by='ID-ÚNICO', ascending=True)

In [ ]:
new_column_names = ['PAVIMENTAÇÃO - Descrição',
 'PAVIMENTAÇÃO - km(i)', 
 'PAVIMENTAÇÃO - km(f)', 
 'PAVIMENTAÇÃO - Ext.(km)', 
 'PAVIMENTAÇÃO - PERCENTUAL (%) PLAN', 
 'PAVIMENTAÇÃO - (km)% PLAN', 
 'PAVIMENTAÇÃO - FINANCEIRO PLAN (R$)', 
 'PAVIMENTAÇÃO - PERCENTUAL (%) EXEC', 
 'PAVIMENTAÇÃO - (km)% EXEC', 
 'PAVIMENTAÇÃO - FINANCEIRO(R$)',
 'DUPLICAÇÃO - Descrição',
 'DUPLICAÇÃO - km(i)', 
 'DUPLICAÇÃO - km(f)', 
 'DUPLICAÇÃO - Ext.(km)', 
 'DUPLICAÇÃO - PERCENTUAL (%) PLAN', 
 'DUPLICAÇÃO - (km)% PLAN', 
 'DUPLICAÇÃO - FINANCEIRO PLAN (R$)', 
 'DUPLICAÇÃO - PERCENTUAL (%) EXEC', 
 'DUPLICAÇÃO - (km)% EXEC', 
 'DUPLICAÇÃO - FINANCEIRO(R$)',
 'OAE - Descrição',
 'OAE - km(i)', 
 'OAE - km(f)', 
 'OAE - Ext.(km)', 
 'OAE - PERCENTUAL (%) PLAN', 
 'OAE - (km)% PLAN', 
 'OAE - FINANCEIRO PLAN (R$)', 
 'OAE - PERCENTUAL (%) EXEC', 
 'OAE - (km)% EXEC', 
 'OAE - FINANCEIRO(R$)',
 'CONTORNO - Descrição',
 'CONTORNO - km(i)', 
 'CONTORNO - km(f)', 
 'CONTORNO - Ext.(km)', 
 'CONTORNO - PERCENTUAL (%) PLAN', 
 'CONTORNO - (km)% PLAN', 
 'CONTORNO - FINANCEIRO PLAN (R$)', 
 'CONTORNO - PERCENTUAL (%) EXEC', 
 'CONTORNO - (km)% EXEC',
 'CONTORNO - FINANCEIRO(R$)',
 'FX ADICIONAL - Descrição',
 'FX ADICIONAL - km(i)',
 'FX ADICIONAL - km(f)', 
 'FX ADICIONAL - Ext.(km)', 
 'FX ADICIONAL - PERCENTUAL (%) PLAN', 
 'FX ADICIONAL - (km)% PLAN', 
 'FX ADICIONAL - FINANCEIRO PLAN (R$)', 
 'FX ADICIONAL - PERCENTUAL (%) EXEC', 
 'FX ADICIONAL - (km)% EXEC', 
 'FX ADICIONAL - FINANCEIRO(R$)',
 'TERCEIRA FAIXA - Descrição',
 'TERCEIRA FAIXA - km(i)', 
 'TERCEIRA FAIXA - km(f)', 
 'TERCEIRA FAIXA - Ext.(km)', 
 'TERCEIRA FAIXA - PERCENTUAL (%) PLAN', 
 'TERCEIRA FAIXA - (km)% PLAN', 
 'TERCEIRA FAIXA - FINANCEIRO PLAN (R$)', 
 'TERCEIRA FAIXA - PERCENTUAL (%) EXEC', 
 'TERCEIRA FAIXA - (km)% EXEC', 
 'TERCEIRA FAIXA - FINANCEIRO(R$)']

In [ ]:
# Cria um dicionário que mapeia qualquer coluna a partir do índice 9 para os novos nomes
rename_dict = {df_meta_25_raw.columns[i]: name for i, name in enumerate(new_column_names, start=9)}

# Usa o método rename para renomear as colunas, substituindo os nomes existentes
df_meta_25_raw.rename(columns=rename_dict, inplace=True)

In [ ]:
# Identifica colunas que possuem a string 'OBSERVAÇÕES' em seu nome (preserva colunas de RISCOS)
columns_to_drop = [col for col in df_meta_27_copy.columns if 'OBSERVAÇÕES' in col]
if columns_to_drop:
    df_meta_27_copy = df_meta_27_copy.drop(columns=columns_to_drop)

In [ ]:

# Mantém apenas as colunas identificadas
df_meta_25_raw = df_meta_25_raw[columns_to_keep]

In [ ]:
df_meta_25_raw.columns

In [ ]:
df_meta_25_raw

In [ ]:
# Seleciona colunas da posição em diante
cols_a_verificar = df_meta_25_raw.iloc[:, 3:]

# Remove linhas onde todas essas colunas estão vazias
df_meta_25_raw = df_meta_25_raw.dropna(subset=cols_a_verificar.columns, how='all')

In [ ]:
df_meta_25_raw1 = df_meta_25_raw[['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO']]

In [ ]:
colunas_manter = ['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO']

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_meta_25_raw.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_meta_25_raw = df_meta_25_raw.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [ ]:
df_meta_25_raw['SETOR'] = 'Rodoviário'

In [ ]:
df_meta_25_raw[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_meta_25_raw['Atributo'].str.split(' - ', expand=True)

In [ ]:
df_meta_25_raw.columns

In [ ]:
df_meta_25_raw = df_meta_25_raw[['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO',
       'Atributo.1', 'Atributo.2', 'Valor']]

In [ ]:
df_meta_25_raw.dropna(subset=['Valor'])
df_meta_25_raw = df_meta_25_raw.loc[df_meta_25_raw['Valor'] != 0]

In [ ]:
# Pivotar o DataFrame
df_meta_25_raw = df_meta_25_raw.pivot_table(index=['ID-ÚNICO', 'Atributo.1'], 
                                            columns='Atributo.2', values='Valor', aggfunc='first').reset_index()

In [ ]:
df_meta_25_raw.columns

In [ ]:
df_meta_25_raw

In [ ]:
# Remove a coluna de índice indesejada, se existir
if 'Atributo.2' in df_meta_25_raw.columns:
    df_meta_25_raw = df_meta_25_raw.drop(columns='Atributo.2')

In [ ]:
df_meta_25_raw1

In [ ]:
# Faz o merge com base na coluna 'ID-ÚNICO'
df_meta_25_raw = pd.merge(df_meta_25_raw1, df_meta_25_raw, on='ID-ÚNICO', how='inner')

In [ ]:
df_meta_25_raw

In [ ]:
df_meta_25_raw1

In [ ]:
df_meta_25_raw = df_meta_25_raw.dropna(subset=['Atributo.1'])

In [ ]:
df_meta_25_raw

In [ ]:
df_meta_25_raw.columns

In [ ]:
# Filtra apenas as colunas que existem
colunas_existentes = [col for col in ['ID-ÚNICO', 'SETOR', 'EMPREENDIMENTO',
                                      'Atributo.1', 'Descrição', 'Ext.(km)', 
                                      'FINANCEIRO PLAN (R$)', 'FINANCEIRO(R$)', 
                                      '(km)% PLAN', '(km)% EXEC'] 
                      if col in df_meta_25_raw.columns]

print(f"Colunas que existem: {colunas_existentes}")
df_meta_25_raw = df_meta_25_raw[colunas_existentes]

print(f"Shape final: {df_meta_25_raw.shape}")
print(df_meta_25_raw.head())

In [ ]:
# Renomear Colunas
df_meta_25_raw.rename(columns={'ID-ÚNICO': 'ID-ÚNICO2'}, inplace=True)
df_meta_25_raw.rename(columns={'SETOR': 'SETOR2'}, inplace=True)
df_meta_25_raw.rename(columns={'EMPREENDIMENTO': 'EMPREENDIMENTO2'}, inplace=True)
df_meta_25_raw.rename(columns={'Descrição': 'Descrição2'}, inplace=True)
df_meta_25_raw.rename(columns={'Ext.(km)': 'Ext.(km)2'}, inplace=True)
df_meta_25_raw.rename(columns={'FINANCEIRO PLAN (R$)': 'FINANCEIRO PLAN (R$)2'}, inplace=True)
df_meta_25_raw.rename(columns={'FINANCEIRO(R$)': 'FINANCEIRO(R$)2'}, inplace=True)
df_meta_25_raw.rename(columns={'(km)% PLAN': '(km)% PLAN2'}, inplace=True)
df_meta_25_raw.rename(columns={'(km)% EXEC': '(km)% EXEC2'}, inplace=True)

In [ ]:
df_meta_25_raw

In [ ]:
df_meta_25_raw.to_excel('2025(2).xlsx', sheet_name= '2025(2)', index= False)

<h1><b><i>'META(2026)'</b></i></h1>

In [ ]:
#CHECKPOINT
# Primeiro cria um alias
df_meta_26 = df_meta_2026

df_meta_26_copy = df_meta_26.copy()

In [ ]:
df_meta_26_copy = pd.read_excel(file_path, sheet_name='META(2026)', header=2)

In [ ]:
df_meta_26_copy

In [ ]:
df_meta_26_copy = df_meta_26_copy.drop([0, 1, 2], axis = 0)

In [ ]:
df_meta_26_copy.columns

In [ ]:
# Drop na coluna 'Região' que não é usada.
df_meta_26_copy.drop(columns=['Região'], inplace=True)

In [ ]:
# Removendo colunas
df_meta_26_copy = df_meta_26_copy.drop(df_meta_26_copy.columns[8:176], axis=1)

In [ ]:
# Renomear Colunas

df_meta_26_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR (Grupo Controlador)'}, inplace=True)
df_meta_26_copy.rename(columns={'PAVIMENTAÇÃO                                                                                                                                                                                                                                                                                     (Ano Vigente)': 'PAVIMENTAÇÃO - 2026 - Descrição'}, inplace=True)
df_meta_26_copy.rename(columns={'DUPLICAÇÃO                                                                                                                                                                                                                                                                                               (Ano Vigente)': 'DUPLICAÇÃO - 2026 - Descrição'}, inplace=True)
df_meta_26_copy.rename(columns={'OAE                                                                                                                                                                                                                                                                                                               (Ano Vigente)': 'OAE - 2026 - Descrição'}, inplace=True)
df_meta_26_copy.rename(columns={'CONTORNO                                                                                                                                                                                                                                                                                                                   (Ano Vigente)': 'CONTORNO - 2026 - Descrição'}, inplace=True)
df_meta_26_copy.rename(columns={'FX ADICIONAL                                                                                                                                                                                                                                                                                                     (Ano Vigente)': 'FX ADICIONAL - 2026 - Descrição'}, inplace=True)
df_meta_26_copy.rename(columns={'TERCEIRA FAIXA                                                                                                                                                                                                                                                                                         (Ano Vigente)': 'TERCEIRA FAIXA - 2026 - Descrição'}, inplace=True)

df_meta_26_copy.rename(columns={'RISCOS (Pavimentação)       ': 'RISCOS (Pavimentação) - SITUAÇÃO'}, inplace=True)
df_meta_26_copy.rename(columns={'RISCOS (Duplicação)       ': 'RISCOS (Duplicação) - SITUAÇÃO'}, inplace=True)
df_meta_26_copy.rename(columns={'RISCOS (OAE)       ': 'RISCOS (OAE) - SITUAÇÃO'}, inplace=True)
df_meta_26_copy.rename(columns={'RISCOS (Contorno)       ': 'RISCOS (Contorno) - SITUAÇÃO'}, inplace=True)
df_meta_26_copy.rename(columns={'RISCOS (FX Adicional)       ': 'RISCOS (Faixa Adicional) - SITUAÇÃO'}, inplace=True)
df_meta_26_copy.rename(columns={'RISCOS (Terceira Faixa)       ': 'RISCOS (Terceira Faixa) - SITUAÇÃO'}, inplace=True)

In [ ]:
new_column_names = ['PAVIMENTAÇÃO - 2026 - km (i)', 
 'PAVIMENTAÇÃO - 2026 - km (f)', 
 'PAVIMENTAÇÃO - 2026 - Ext. (km)', 
 'PAVIMENTAÇÃO - 2026 - PERCENTUAL (%) PLAN', 
 'PAVIMENTAÇÃO - 2026 - (km)% PLAN', 
 'PAVIMENTAÇÃO - 2026 - FINANCEIRO PLAN (R$)', 
 'PAVIMENTAÇÃO - 2026 - PERCENTUAL (%) EXEC', 
 'PAVIMENTAÇÃO - 2026 - (km)% EXEC', 
 'PAVIMENTAÇÃO - 2026 - FINANCEIRO (R$)',
 'RISCOS (Pavimentação) - SITUAÇÃO',
 'RISCOS (Pavimentação) - INTERFERÊNCIA',
 'RISCOS (Pavimentação) - PRINCIPAL',
 'RISCOS (Pavimentação) - NÍVEL DE ALERTA',
 'RISCOS (Pavimentação) - Nº PROCESSO',
 'RISCOS (Pavimentação) - DATA DO PROTOCOLO',
 'RISCOS (Pavimentação) - LICENÇA',
 'RISCOS (Pavimentação) - SITUAÇÃO GERAL',
 'DUPLICAÇÃO - 2026 - Descrição',
 'DUPLICAÇÃO - 2026 - km (i)', 
 'DUPLICAÇÃO - 2026 - km (f)', 
 'DUPLICAÇÃO - 2026 - Ext. (km)', 
 'DUPLICAÇÃO - 2026 - PERCENTUAL (%) PLAN', 
 'DUPLICAÇÃO - 2026 - (km)% PLAN', 
 'DUPLICAÇÃO - 2026 - FINANCEIRO PLAN (R$)', 
 'DUPLICAÇÃO - 2026 - PERCENTUAL (%) EXEC', 
 'DUPLICAÇÃO - 2026 - (km)% EXEC', 
 'DUPLICAÇÃO - 2026 - FINANCEIRO (R$)',
 'RISCOS (Duplicação) - SITUAÇÃO',
 'RISCOS (Duplicação) - INTERFERÊNCIA',
 'RISCOS (Duplicação) - PRINCIPAL',
 'RISCOS (Duplicação) - NÍVEL DE ALERTA',
 'RISCOS (Duplicação) - Nº PROCESSO',
 'RISCOS (Duplicação) - DATA DO PROTOCOLO',
 'RISCOS (Duplicação) - LICENÇA',
 'RISCOS (Duplicação) - SITUAÇÃO GERAL',
 'OAE - 2026 - Descrição',
 'OAE - 2026 - km (i)', 
 'OAE - 2026 - km (f)', 
 'OAE - 2026 - Ext. (km)', 
 'OAE - 2026 - PERCENTUAL (%) PLAN', 
 'OAE - 2026 - (km)% PLAN', 
 'OAE - 2026 - FINANCEIRO PLAN (R$)', 
 'OAE - 2026 - PERCENTUAL (%) EXEC', 
 'OAE - 2026 - (km)% EXEC', 
 'OAE - 2026 - FINANCEIRO (R$)',
 'RISCOS (OAE) - SITUAÇÃO',
 'RISCOS (OAE) - INTERFERÊNCIA',
 'RISCOS (OAE) - PRINCIPAL',
 'RISCOS (OAE) - NÍVEL DE ALERTA',
 'RISCOS (OAE) - Nº PROCESSO',
 'RISCOS (OAE) - DATA DO PROTOCOLO',
 'RISCOS (OAE) - LICENÇA',
 'RISCOS (OAE) - SITUAÇÃO GERAL',
 'CONTORNO - 2026 - Descrição',
 'CONTORNO - 2026 - km (i)', 
 'CONTORNO - 2026 - km (f)', 
 'CONTORNO - 2026 - Ext. (km)', 
 'CONTORNO - 2026 - PERCENTUAL (%) PLAN', 
 'CONTORNO - 2026 - (km)% PLAN', 
 'CONTORNO - 2026 - FINANCEIRO PLAN (R$)', 
 'CONTORNO - 2026 - PERCENTUAL (%) EXEC', 
 'CONTORNO - 2026 - (km)% EXEC',
 'CONTORNO - 2026 - FINANCEIRO (R$)',
 'RISCOS (Contorno) - SITUAÇÃO',
 'RISCOS (Contorno) - INTERFERÊNCIA',
 'RISCOS (Contorno) - PRINCIPAL',
 'RISCOS (Contorno) - NÍVEL DE ALERTA',
 'RISCOS (Contorno) - Nº PROCESSO',
 'RISCOS (Contorno) - DATA DO PROTOCOLO',
 'RISCOS (Contorno) - LICENÇA',
 'RISCOS (Contorno) - SITUAÇÃO GERAL',
 'FX ADICIONAL - 2026 - Descrição',
 'FX ADICIONAL - 2026 - km (i)',
 'FX ADICIONAL - 2026 - km (f)', 
 'FX ADICIONAL - 2026 - Ext. (km)', 
 'FX ADICIONAL - 2026 - PERCENTUAL (%) PLAN', 
 'FX ADICIONAL - 2026 - (km)% PLAN', 
 'FX ADICIONAL - 2026 - FINANCEIRO PLAN (R$)', 
 'FX ADICIONAL - 2026 - PERCENTUAL (%) EXEC', 
 'FX ADICIONAL - 2026 - (km)% EXEC', 
 'FX ADICIONAL - 2026 - FINANCEIRO (R$)',
 'FX ADICIONAL - 2026 - Descrição',
 'RISCOS (Faixa Adicional) - SITUAÇÃO',
 'RISCOS (Faixa Adicional) - INTERFERÊNCIA',
 'RISCOS (Faixa Adicional) - PRINCIPAL',
 'RISCOS (Faixa Adicional) - NÍVEL DE ALERTA',
 'RISCOS (Faixa Adicional) - Nº PROCESSO',
 'RISCOS (Faixa Adicional) - DATA DO PROTOCOLO',
 'RISCOS (Faixa Adicional) - LICENÇA',
 'RISCOS (Faixa Adicional) - SITUAÇÃO GERAL',
 'TERCEIRA FAIXA - 2026 - Descrição',
 'TERCEIRA FAIXA - 2026 - km (i)', 
 'TERCEIRA FAIXA - 2026 - km (f)', 
 'TERCEIRA FAIXA - 2026 - Ext. (km)', 
 'TERCEIRA FAIXA - 2026 - PERCENTUAL (%) PLAN', 
 'TERCEIRA FAIXA - 2026 - (km)% PLAN', 
 'TERCEIRA FAIXA - 2026 - FINANCEIRO PLAN (R$)', 
 'TERCEIRA FAIXA - 2026 - PERCENTUAL (%) EXEC', 
 'TERCEIRA FAIXA - 2026 - (km)% EXEC', 
 'TERCEIRA FAIXA - 2026 - FINANCEIRO (R$)',
 'RISCOS (Terceira Faixa) - SITUAÇÃO',
 'RISCOS (Terceira Faixa) - INTERFERÊNCIA',
 'RISCOS (Terceira Faixa) - PRINCIPAL',
 'RISCOS (Terceira Faixa) - NÍVEL DE ALERTA'
 'RISCOS (Terceira Faixa) - Nº PROCESSO',
 'RISCOS (Terceira Faixa) - DATA DO PROTOCOLO',
 'RISCOS (Terceira Faixa) - LICENÇA',
 'RISCOS (Terceira Faixa) - SITUAÇÃO GERAL',
 'OBSERVAÇÕES - SITUAÇÃO GERAL']


In [ ]:
# Criar um dicionário que mapeia 'Unnamed: x' para os novos nomes
rename_dict = {'Unnamed: {}'.format(i): name for i, name in enumerate(new_column_names, start=178)}

# Usar o método rename para renomear as colunas
df_meta_26_copy.rename(columns=rename_dict, inplace=True)

In [ ]:
# Identifica colunas que possuem a string 'RISCOS' em seu nome
columns_to_drop = [col for col in df_meta_26_copy.columns if 'RISCOS' in col or 'OBSERVAÇÕES' in col]

# Remove as colunas identificadas
df_meta_26_copy = df_meta_26_copy.drop(columns=columns_to_drop)

In [ ]:
df_meta_26_copy.columns

In [ ]:
df_meta_26_copy.dropna(subset=['ESTADO/LOTE'])

In [ ]:
columns_to_fill = ['EMPREENDIMENTO']
df_meta_26_copy[columns_to_fill] = df_meta_26_copy.groupby((df_meta_26_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [ ]:
df_meta_26_copy['ESTADO/LOTE'] = df_meta_26_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS',
       'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO',
       'MATO GROSSO', 'PARÁ', 'RIO/SP', 'SÃO PAULO', 'PARANÁ', 'BAHIA',
       'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)

# Agora preenchemos para baixo até encontrar o próximo valor não-NaN (isto é, a próxima 'Situação')
df_meta_26_copy['ESTADO/LOTE'] = df_meta_26_copy['ESTADO/LOTE'].fillna(method='ffill')

In [ ]:
df_meta_26_copy = df_meta_26_copy[~df_meta_26_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]

In [ ]:
df_meta_26_copy.columns

In [ ]:
df_meta_26_copy.index = range(1, len(df_meta_26_copy) + 1)
df_meta_26_copy.index.name = 'ID-ÚNICO'
df_meta_26_copy = df_meta_26_copy.reset_index()

In [ ]:
df_meta_26_raw = df_meta_26_copy

In [ ]:
# df_meta_26_copy.to_excel('Dados Gerados/RAW_META(2026).xlsx', index=False)

In [ ]:
colunas_manter = ['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO']

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_meta_26_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_meta_26_copy = df_meta_26_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [ ]:
df_meta_26_copy['EMPREENDIMENTO'].unique()

In [ ]:
df_meta_26_copy = df_meta_26_copy.loc[df_meta_26_copy['Valor'] != 0]
df_meta_26_copy = df_meta_26_copy.dropna(subset=['Valor'])

In [ ]:
df_meta_26_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_meta_26_copy['Atributo'].str.split(' - ', expand=True)

In [ ]:
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_meta_26_copy = df_meta_26_copy[['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [ ]:
# Converter colunas para 'category'
df_meta_26_copy['SETOR'] = df_meta_26_copy['SETOR'].astype('category')
df_meta_26_copy['UF'] = df_meta_26_copy['UF'].astype('category')
df_meta_26_copy['PROPONENTE'] = df_meta_26_copy['PROPONENTE'].astype('category')
df_meta_26_copy['EXECUTOR (Grupo Controlador)'] = df_meta_26_copy['EXECUTOR (Grupo Controlador)'].astype('category')
df_meta_26_copy['ESTRUTURADOR DO PROJETO'] = df_meta_26_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_meta_26_copy['Atributo.1'] = df_meta_26_copy['Atributo.1'].astype('category')
df_meta_26_copy['Atributo.2'] = df_meta_26_copy['Atributo.2'].astype('category')

# Converter colunas para 'string'
df_meta_26_copy['EMPREENDIMENTO'] = df_meta_26_copy['EMPREENDIMENTO'].astype('string') 


In [ ]:
df_meta_26_copy = df_meta_26_copy.sort_values(by='ID-ÚNICO', ascending=True)

In [ ]:
df_meta_26_copy.to_excel('Dados Gerados/META(2026).xlsx', index=False, sheet_name= 'META')

<h1><b><i>'META(2027)'</b></i></h1>

In [ ]:
#CHECKPOINT
# Primeiro cria um alias
df_meta_27 = df_meta_2027

df_meta_27_copy = df_meta_27.copy()


In [ ]:
df_meta_27_copy = pd.read_excel(file_path, sheet_name='META(2027)', header=2)

In [ ]:
df_meta_27_copy

In [ ]:
df_meta_27_copy = df_meta_27_copy.drop([0, 1, 2], axis = 0)

In [ ]:
df_meta_27_copy.columns

In [ ]:
# Drop na coluna 'Região' que não é usada.
df_meta_27_copy.drop(columns=['Região'], inplace=True)

In [ ]:
# Removendo colunas
df_meta_27_copy = df_meta_27_copy.drop(df_meta_27_copy.columns[8:176], axis=1)

In [ ]:
# Renomear Colunas

df_meta_27_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR (Grupo Controlador)'}, inplace=True)
df_meta_27_copy.rename(columns={'PAVIMENTAÇÃO                                                                                                                                                                                                                                                                                     (Ano Vigente)': 'PAVIMENTAÇÃO - 2027 - Descrição'}, inplace=True)
df_meta_27_copy.rename(columns={'DUPLICAÇÃO                                                                                                                                                                                                                                                                                               (Ano Vigente)': 'DUPLICAÇÃO - 2027 - Descrição'}, inplace=True)
df_meta_27_copy.rename(columns={'OAE                                                                                                                                                                                                                                                                                                               (Ano Vigente)': 'OAE - 2027 - Descrição'}, inplace=True)
df_meta_27_copy.rename(columns={'CONTORNO                                                                                                                                                                                                                                                                                                                   (Ano Vigente)': 'CONTORNO - 2027 - Descrição'}, inplace=True)
df_meta_27_copy.rename(columns={'FX ADICIONAL                                                                                                                                                                                                                                                                                                     (Ano Vigente)': 'FX ADICIONAL - 2027 - Descrição'}, inplace=True)
df_meta_27_copy.rename(columns={'TERCEIRA FAIXA                                                                                                                                                                                                                                                                                         (Ano Vigente)': 'TERCEIRA FAIXA - 2027 - Descrição'}, inplace=True)

df_meta_27_copy.rename(columns={'RISCOS (Pavimentação)       ': 'RISCOS (Pavimentação) - SITUAÇÃO'}, inplace=True)
df_meta_27_copy.rename(columns={'RISCOS (Duplicação)       ': 'RISCOS (Duplicação) - SITUAÇÃO'}, inplace=True)
df_meta_27_copy.rename(columns={'RISCOS (OAE)       ': 'RISCOS (OAE) - SITUAÇÃO'}, inplace=True)
df_meta_27_copy.rename(columns={'RISCOS (Contorno)       ': 'RISCOS (Contorno) - SITUAÇÃO'}, inplace=True)
df_meta_27_copy.rename(columns={'RISCOS (FX Adicional)       ': 'RISCOS (Faixa Adicional) - SITUAÇÃO'}, inplace=True)
df_meta_27_copy.rename(columns={'RISCOS (Terceira Faixa)       ': 'RISCOS (Terceira Faixa) - SITUAÇÃO'}, inplace=True)

In [ ]:
new_column_names = ['PAVIMENTAÇÃO - 2027 - km (i)', 
 'PAVIMENTAÇÃO - 2027 - km (f)', 
 'PAVIMENTAÇÃO - 2027 - Ext. (km)', 
 'PAVIMENTAÇÃO - 2027 - PERCENTUAL (%) PLAN', 
 'PAVIMENTAÇÃO - 2027 - (km)% PLAN', 
 'PAVIMENTAÇÃO - 2027 - FINANCEIRO PLAN (R$)', 
 'PAVIMENTAÇÃO - 2027 - PERCENTUAL (%) EXEC', 
 'PAVIMENTAÇÃO - 2027 - (km)% EXEC', 
 'PAVIMENTAÇÃO - 2027 - FINANCEIRO (R$)',
 'RISCOS (Pavimentação) - SITUAÇÃO',
 'RISCOS (Pavimentação) - INTERFERÊNCIA',
 'RISCOS (Pavimentação) - PRINCIPAL',
 'RISCOS (Pavimentação) - NÍVEL DE ALERTA',
 'RISCOS (Pavimentação) - Nº PROCESSO',
 'RISCOS (Pavimentação) - DATA DO PROTOCOLO',
 'RISCOS (Pavimentação) - LICENÇA',
 'RISCOS (Pavimentação) - SITUAÇÃO GERAL',
 'DUPLICAÇÃO - 2027 - Descrição',
 'DUPLICAÇÃO - 2027 - km (i)', 
 'DUPLICAÇÃO - 2027 - km (f)', 
 'DUPLICAÇÃO - 2027 - Ext. (km)', 
 'DUPLICAÇÃO - 2027 - PERCENTUAL (%) PLAN', 
 'DUPLICAÇÃO - 2027 - (km)% PLAN', 
 'DUPLICAÇÃO - 2027 - FINANCEIRO PLAN (R$)', 
 'DUPLICAÇÃO - 2027 - PERCENTUAL (%) EXEC', 
 'DUPLICAÇÃO - 2027 - (km)% EXEC', 
 'DUPLICAÇÃO - 2027 - FINANCEIRO (R$)',
 'RISCOS (Duplicação) - SITUAÇÃO',
 'RISCOS (Duplicação) - INTERFERÊNCIA',
 'RISCOS (Duplicação) - PRINCIPAL',
 'RISCOS (Duplicação) - NÍVEL DE ALERTA',
 'RISCOS (Duplicação) - Nº PROCESSO',
 'RISCOS (Duplicação) - DATA DO PROTOCOLO',
 'RISCOS (Duplicação) - LICENÇA',
 'RISCOS (Duplicação) - SITUAÇÃO GERAL',
 'OAE - 2027 - Descrição',
 'OAE - 2027 - km (i)', 
 'OAE - 2027 - km (f)', 
 'OAE - 2027 - Ext. (km)', 
 'OAE - 2027 - PERCENTUAL (%) PLAN', 
 'OAE - 2027 - (km)% PLAN', 
 'OAE - 2027 - FINANCEIRO PLAN (R$)', 
 'OAE - 2027 - PERCENTUAL (%) EXEC', 
 'OAE - 2027 - (km)% EXEC', 
 'OAE - 2027 - FINANCEIRO (R$)',
 'RISCOS (OAE) - SITUAÇÃO',
 'RISCOS (OAE) - INTERFERÊNCIA',
 'RISCOS (OAE) - PRINCIPAL',
 'RISCOS (OAE) - NÍVEL DE ALERTA',
 'RISCOS (OAE) - Nº PROCESSO',
 'RISCOS (OAE) - DATA DO PROTOCOLO',
 'RISCOS (OAE) - LICENÇA',
 'RISCOS (OAE) - SITUAÇÃO GERAL',
 'CONTORNO - 2027 - Descrição',
 'CONTORNO - 2027 - km (i)', 
 'CONTORNO - 2027 - km (f)', 
 'CONTORNO - 2027 - Ext. (km)', 
 'CONTORNO - 2027 - PERCENTUAL (%) PLAN', 
 'CONTORNO - 2027 - (km)% PLAN', 
 'CONTORNO - 2027 - FINANCEIRO PLAN (R$)', 
 'CONTORNO - 2027 - PERCENTUAL (%) EXEC', 
 'CONTORNO - 2027 - (km)% EXEC',
 'CONTORNO - 2027 - FINANCEIRO (R$)',
 'RISCOS (Contorno) - SITUAÇÃO',
 'RISCOS (Contorno) - INTERFERÊNCIA',
 'RISCOS (Contorno) - PRINCIPAL',
 'RISCOS (Contorno) - NÍVEL DE ALERTA',
 'RISCOS (Contorno) - Nº PROCESSO',
 'RISCOS (Contorno) - DATA DO PROTOCOLO',
 'RISCOS (Contorno) - LICENÇA',
 'RISCOS (Contorno) - SITUAÇÃO GERAL',
 'FX ADICIONAL - 2027 - Descrição',
 'FX ADICIONAL - 2027 - km (i)',
 'FX ADICIONAL - 2027 - km (f)', 
 'FX ADICIONAL - 2027 - Ext. (km)', 
 'FX ADICIONAL - 2027 - PERCENTUAL (%) PLAN', 
 'FX ADICIONAL - 2027 - (km)% PLAN', 
 'FX ADICIONAL - 2027 - FINANCEIRO PLAN (R$)', 
 'FX ADICIONAL - 2027 - PERCENTUAL (%) EXEC', 
 'FX ADICIONAL - 2027 - (km)% EXEC', 
 'FX ADICIONAL - 2027 - FINANCEIRO (R$)',
 'FX ADICIONAL - 2027 - Descrição',
 'RISCOS (Faixa Adicional) - SITUAÇÃO',
 'RISCOS (Faixa Adicional) - INTERFERÊNCIA',
 'RISCOS (Faixa Adicional) - PRINCIPAL',
 'RISCOS (Faixa Adicional) - NÍVEL DE ALERTA',
 'RISCOS (Faixa Adicional) - Nº PROCESSO',
 'RISCOS (Faixa Adicional) - DATA DO PROTOCOLO',
 'RISCOS (Faixa Adicional) - LICENÇA',
 'RISCOS (Faixa Adicional) - SITUAÇÃO GERAL',
 'TERCEIRA FAIXA - 2027 - km (i)', 
 'TERCEIRA FAIXA - 2027 - km (f)', 
 'TERCEIRA FAIXA - 2027 - Ext. (km)', 
 'TERCEIRA FAIXA - 2027 - PERCENTUAL (%) PLAN', 
 'TERCEIRA FAIXA - 2027 - (km)% PLAN', 
 'TERCEIRA FAIXA - 2027 - FINANCEIRO PLAN (R$)', 
 'TERCEIRA FAIXA - 2027 - PERCENTUAL (%) EXEC', 
 'TERCEIRA FAIXA - 2027 - (km)% EXEC', 
 'TERCEIRA FAIXA - 2027 - FINANCEIRO (R$)',
 'RISCOS (Terceira Faixa) - SITUAÇÃO',
 'RISCOS (Terceira Faixa) - INTERFERÊNCIA',
 'RISCOS (Terceira Faixa) - PRINCIPAL',
 'RISCOS (Terceira Faixa) - NÍVEL DE ALERTA'
 'RISCOS (Terceira Faixa) - Nº PROCESSO',
 'RISCOS (Terceira Faixa) - DATA DO PROTOCOLO',
 'RISCOS (Terceira Faixa) - LICENÇA',
 'RISCOS (Terceira Faixa) - SITUAÇÃO GERAL',
 'OBSERVAÇÕES - SITUAÇÃO GERAL']


In [ ]:
# Criar um dicionário que mapeia 'Unnamed: x' para os novos nomes
rename_dict = {'Unnamed: {}'.format(i): name for i, name in enumerate(new_column_names, start=178)}

# Usar o método rename para renomear as colunas
df_meta_27_copy.rename(columns=rename_dict, inplace=True)

In [ ]:
# Identifica colunas que possuem a string 'RISCOS' em seu nome
columns_to_drop = [col for col in df_meta_27_copy.columns if 'RISCOS' in col]

# Remove as colunas identificadas
df_meta_27_copy = df_meta_27_copy.drop(columns=columns_to_drop)

In [ ]:
df_meta_27_copy

In [ ]:
df_meta_27_copy.dropna(subset=['ESTADO/LOTE'])

In [ ]:
columns_to_fill = ['EMPREENDIMENTO']
df_meta_27_copy[columns_to_fill] = df_meta_27_copy.groupby((df_meta_27_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [ ]:
df_meta_27_copy['ESTADO/LOTE'] = df_meta_27_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS',
       'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO',
       'MATO GROSSO', 'PARÁ', 'RIO/SP', 'SÃO PAULO', 'PARANÁ', 'BAHIA',
       'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)

# Agora preenchemos para baixo até encontrar o próximo valor não-NaN (isto é, a próxima 'Situação')
df_meta_27_copy['ESTADO/LOTE'] = df_meta_27_copy['ESTADO/LOTE'].fillna(method='ffill')

In [ ]:
df_meta_27_copy = df_meta_27_copy[~df_meta_27_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]

In [ ]:
df_meta_27_copy.columns

In [ ]:
df_meta_27_copy.dropna(subset=['ESTADO/LOTE'])

In [ ]:
df_meta_27_raw = df_meta_27_copy

In [ ]:
# df_meta_27_copy.to_excel('Dados Gerados/RAW_META(2027).xlsx', index=False)

In [ ]:
colunas_manter = ['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO']

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_meta_27_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_meta_27_copy = df_meta_27_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [ ]:
df_meta_27_copy['EMPREENDIMENTO'].unique()

In [ ]:
df_meta_27_copy = df_meta_27_copy.loc[df_meta_27_copy['Valor'] != 0]
df_meta_27_copy = df_meta_27_copy.dropna(subset=['Valor'])

In [ ]:
df_meta_27_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_meta_27_copy['Atributo'].str.split(' - ', expand=True)

In [ ]:
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_meta_27_copy = df_meta_27_copy[['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [ ]:
# Converter colunas para 'category'
df_meta_27_copy['SETOR'] = df_meta_27_copy['SETOR'].astype('category')
df_meta_27_copy['UF'] = df_meta_27_copy['UF'].astype('category')
df_meta_27_copy['PROPONENTE'] = df_meta_27_copy['PROPONENTE'].astype('category')
df_meta_27_copy['EXECUTOR (Grupo Controlador)'] = df_meta_27_copy['EXECUTOR (Grupo Controlador)'].astype('category')
df_meta_27_copy['ESTRUTURADOR DO PROJETO'] = df_meta_27_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_meta_27_copy['Atributo.1'] = df_meta_27_copy['Atributo.1'].astype('category')
df_meta_27_copy['Atributo.2'] = df_meta_27_copy['Atributo.2'].astype('category')

# Converter colunas para 'string'
df_meta_27_copy['EMPREENDIMENTO'] = df_meta_27_copy['EMPREENDIMENTO'].astype('string') 


In [ ]:
df_meta_27_copy = df_meta_27_copy.sort_values(by='ID-ÚNICO', ascending=True)

In [ ]:
df_meta_27_copy.to_excel('Dados Gerados/META(2027).xlsx', index=False, sheet_name= 'META')

<h1><b><i>'iNEXECUTADO (pendências até 24)'</b></i></h1>

In [ ]:
#CHECKPOINT
df_inexec_copy = df_inexec.copy()

In [ ]:
df_inexec_copy =  pd.read_excel(file_path, sheet_name='iNEXECUTADO (pendências até 24)', header=2)
df_inexec_copy.columns

In [ ]:
df_inexec_copy

In [ ]:
df_inexec_copy = df_inexec_copy.drop([0, 1, 2], axis = 0)

In [ ]:
# Renomear Coluna ['EXECUTOR...']

df_inexec_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR (Grupo Controlador)'}, inplace=True)
df_inexec_copy.rename(columns={'PAVIMENTAÇÃO (até 2024)': 'PAVIMENTAÇÃO - (até 2024) - Descrição'}, inplace=True)
df_inexec_copy.rename(columns={'RISCOS (Pavimentação)       ': 'RISCOS (Pavimentação) - SITUAÇÃO'}, inplace=True)
df_inexec_copy.rename(columns={'DUPLICAÇÃO (até 2024)': 'DUPLICAÇÃO - (até 2024) - Descrição'}, inplace=True)
df_inexec_copy.rename(columns={'RISCOS (Duplicação)       ': 'RISCOS (Duplicação) - SITUAÇÃO'}, inplace=True)
df_inexec_copy.rename(columns={'OAE  (até 2024)': 'OAE - (até 2024) - Descrição'}, inplace=True)
df_inexec_copy.rename(columns={'RISCOS (OAE)       ': 'RISCOS (OAE) - SITUAÇÃO'}, inplace=True)
df_inexec_copy.rename(columns={'CONTORNO  (até 2024)': 'CONTORNO - (até 2024) - Descrição'}, inplace=True)
df_inexec_copy.rename(columns={'RISCOS (Contorno)       ': 'RISCOS (Contorno) - SITUAÇÃO'}, inplace=True)
df_inexec_copy.rename(columns={'FX ADICIONAL (até 2024)': 'FX ADICIONAL - (até 2024) - Descrição'}, inplace=True)
df_inexec_copy.rename(columns={'RISCOS (FX Adicional)       ': 'RISCOS (Faixa Adicional) - SITUAÇÃO'}, inplace=True)
df_inexec_copy.rename(columns={'TERCEIRA FAIXA (até 2024)': 'TERCEIRA FAIXA - (até 2024) - Descrição'}, inplace=True)
df_inexec_copy.rename(columns={'RISCOS (Terceira Faixa)       ': 'RISCOS (Terceira Faixa) - SITUAÇÃO'}, inplace=True)

In [ ]:
new_column_names = ['PAVIMENTAÇÃO - (até 2024) - km (i)', 
 'PAVIMENTAÇÃO - (até 2024) - km (f)', 
 'PAVIMENTAÇÃO - (até 2024) - Ext. (km)', 
 'PAVIMENTAÇÃO - (até 2024) - PERCENTUAL (%) PLAN', 
 'PAVIMENTAÇÃO - (até 2024) - (km)% PLAN', 
 'PAVIMENTAÇÃO - (até 2024) - FINANCEIRO PLAN (R$)', 
 'PAVIMENTAÇÃO - (até 2024) - PERCENTUAL (%) EXEC', 
 'PAVIMENTAÇÃO - (até 2024) - (km)% EXEC', 
 'PAVIMENTAÇÃO - (até 2024) - FINANCEIRO (R$)',
 'RISCOS (Pavimentação) - SITUAÇÃO',
 'RISCOS (Pavimentação) - INTERFERÊNCIA',
 'RISCOS (Pavimentação) - PRINCIPAL',
 'RISCOS (Pavimentação) - NÍVEL DE ALERTA',
 'DUPLICAÇÃO - (até 2024) - Descrição',
 'DUPLICAÇÃO - (até 2024) - km (i)', 
 'DUPLICAÇÃO - (até 2024) - km (f)', 
 'DUPLICAÇÃO - (até 2024) - Ext. (km)', 
 'DUPLICAÇÃO - (até 2024) - PERCENTUAL (%) PLAN', 
 'DUPLICAÇÃO - (até 2024) - (km)% PLAN', 
 'DUPLICAÇÃO - (até 2024) - FINANCEIRO PLAN (R$)', 
 'DUPLICAÇÃO - (até 2024) - PERCENTUAL (%) EXEC', 
 'DUPLICAÇÃO - (até 2024) - (km)% EXEC', 
 'DUPLICAÇÃO - (até 2024) - FINANCEIRO (R$)',
 'RISCOS (Duplicação) - SITUAÇÃO',
 'RISCOS (Duplicação) - INTERFERÊNCIA',
 'RISCOS (Duplicação) - PRINCIPAL',
 'RISCOS (Duplicação) - NÍVEL DE ALERTA',
 'OAE - (até 2024) - Descrição',
 'OAE - (até 2024) - km (i)', 
 'OAE - (até 2024) - km (f)', 
 'OAE - (até 2024) - Ext. (km)', 
 'OAE - (até 2024) - PERCENTUAL (%) PLAN', 
 'OAE - (até 2024) - (km)% PLAN', 
 'OAE - (até 2024) - FINANCEIRO PLAN (R$)', 
 'OAE - (até 2024) - PERCENTUAL (%) EXEC', 
 'OAE - (até 2024) - (km)% EXEC', 
 'OAE - (até 2024) - FINANCEIRO (R$)',
 'RISCOS (OAE) - SITUAÇÃO',
 'RISCOS (OAE) - INTERFERÊNCIA',
 'RISCOS (OAE) - PRINCIPAL',
 'RISCOS (OAE) - NÍVEL DE ALERTA',
 'CONTORNO - (até 2024) - Descrição',
 'CONTORNO - (até 2024) - km (i)', 
 'CONTORNO - (até 2024) - km (f)', 
 'CONTORNO - (até 2024) - Ext. (km)', 
 'CONTORNO - (até 2024) - PERCENTUAL (%) PLAN', 
 'CONTORNO - (até 2024) - (km)% PLAN', 
 'CONTORNO - (até 2024) - FINANCEIRO PLAN (R$)', 
 'CONTORNO - (até 2024) - PERCENTUAL (%) EXEC', 
 'CONTORNO - (até 2024) - (km)% EXEC',
 'CONTORNO - (até 2024) - FINANCEIRO (R$)',
 'FX ADICIONAL - (até 2024) - Descrição',
 'FX ADICIONAL - (até 2024) - km (i)',
 'FX ADICIONAL - (até 2024) - km (f)', 
 'FX ADICIONAL - (até 2024) - Ext. (km)', 
 'FX ADICIONAL - (até 2024) - PERCENTUAL (%) PLAN', 
 'FX ADICIONAL - (até 2024) - (km)% PLAN', 
 'FX ADICIONAL - (até 2024) - FINANCEIRO PLAN (R$)', 
 'FX ADICIONAL - (até 2024) - PERCENTUAL (%) EXEC', 
 'FX ADICIONAL - (até 2024) - (km)% EXEC', 
 'FX ADICIONAL - (até 2024) - FINANCEIRO (R$)',
 'TERCEIRA FAIXA - (até 2024) - km (i)', 
 'TERCEIRA FAIXA - (até 2024) - km (f)', 
 'TERCEIRA FAIXA - (até 2024) - Ext. (km)', 
 'TERCEIRA FAIXA - (até 2024) - PERCENTUAL (%) PLAN', 
 'TERCEIRA FAIXA - (até 2024) - (km)% PLAN', 
 'TERCEIRA FAIXA - (até 2024) - FINANCEIRO PLAN (R$)', 
 'TERCEIRA FAIXA - (até 2024) - PERCENTUAL (%) EXEC', 
 'TERCEIRA FAIXA - (até 2024) - (km)% EXEC', 
 'TERCEIRA FAIXA - (até 2024) - FINANCEIRO (R$)']

In [ ]:
# Criar um dicionário que mapeia 'Unnamed: x' para os novos nomes
rename_dict = {'Unnamed: {}'.format(i): name for i, name in enumerate(new_column_names, start=9)}

# Usar o método rename para renomear as colunas
df_inexec_copy.rename(columns=rename_dict, inplace=True)

In [ ]:

# Identifica colunas que possuem a string 'RISCOS' em seu nome
columns_to_drop = [col for col in df_inexec_copy.columns if 'RISCOS' in col or 'OBSERVAÇÕES' in col]

# Remove as colunas identificadas
df_inexec_copy = df_inexec_copy.drop(columns=columns_to_drop)

In [ ]:
df_inexec_copy

In [ ]:
df_inexec_copy.dropna(subset=['ESTADO/LOTE'])

In [ ]:
columns_to_fill = ['EMPREENDIMENTO']
df_inexec_copy[columns_to_fill] = df_inexec_copy.groupby((df_inexec_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [ ]:
df_inexec_copy['ESTADO/LOTE'] = df_inexec_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS',
       'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO',
       'MATO GROSSO', 'PARÁ', 'RIO/SP', 'SÃO PAULO', 'PARANÁ', 'BAHIA',
       'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)

# Agora preenchemos para baixo até encontrar o próximo valor não-NaN (isto é, a próxima 'Situação')
df_inexec_copy['ESTADO/LOTE'] = df_inexec_copy['ESTADO/LOTE'].fillna(method='ffill')

In [ ]:
df_inexec_copy = df_inexec_copy[~df_inexec_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]

In [ ]:
df_meta_27_copy.columns

In [ ]:
df_meta_27_copy.index = range(1, len(df_meta_27_copy) + 1)
df_meta_27_copy.index.name = 'ID-ÚNICO'
df_meta_27_copy = df_meta_27_copy.reset_index()

In [ ]:
df_inexec_raw = df_inexec_copy

In [ ]:
# df_inexec_copy.to_excel('Dados Gerados/RAW_iNEXECUTADO (pendências até 24).xlsx', index=False)

In [ ]:
colunas_manter = ['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO']

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_inexec_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_inexec_copy = df_inexec_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [ ]:
df_inexec_copy['EMPREENDIMENTO'].unique()

In [ ]:
df_inexec_copy = df_inexec_copy.loc[df_inexec_copy['Valor'] != 0]
df_inexec_copy = df_inexec_copy.dropna(subset=['Valor'])

In [ ]:
df_inexec_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_inexec_copy['Atributo'].str.split(' - ', expand=True)

In [ ]:
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_inexec_copy = df_inexec_copy[['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [ ]:
'''df_inexec_copy['Atributo.1'] = df_inexec_copy['Atributo.1'].apply(
    lambda x: x.replace('CONTORNO (até 2023)', 'CONTORNO')
              .replace('DUPLICAÇÃO (até 2023)', 'DUPLICAÇÃO')
              .replace('FX ADICIONAL (até 2023)', 'FX ADICIONAL')
              .replace('OAE (até 2023)', 'OAE')
              .replace('PAVIMENTAÇÃO (até 2023)', 'PAVIMENTAÇÃO')
              .replace('TERCEIRA FAIXA (até 2023)', 'TERCEIRA FAIXA')
)'''

In [ ]:
# Converter colunas para 'category'
df_inexec_copy['SETOR'] = df_inexec_copy['SETOR'].astype('category')
df_inexec_copy['UF'] = df_inexec_copy['UF'].astype('category')
df_inexec_copy['PROPONENTE'] = df_inexec_copy['PROPONENTE'].astype('category')
df_inexec_copy['EXECUTOR (Grupo Controlador)'] = df_inexec_copy['EXECUTOR (Grupo Controlador)'].astype('category')
df_inexec_copy['ESTRUTURADOR DO PROJETO'] = df_inexec_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_inexec_copy['Atributo.1'] = df_inexec_copy['Atributo.1'].astype('category')
df_inexec_copy['Atributo.2'] = df_inexec_copy['Atributo.2'].astype('category')

# Converter colunas para 'string'
df_inexec_copy['EMPREENDIMENTO'] = df_inexec_copy['EMPREENDIMENTO'].astype('string') 


In [ ]:
df_inexec_copy = df_inexec_copy.sort_values(by='ID-ÚNICO', ascending=True)

In [ ]:
df_inexec_copy

In [ ]:
df_inexec_copy.to_excel('Dados Gerados/iNEXECUTADO (pendências até 24).xlsx', index=False)

<h1><b><i>'À EXECUTAR (2026 DIANTE)'</b></i></h1>

In [ ]:
#CHECKPOINT
df_a_executar_copy = df_a_executar.copy()

In [ ]:
df_a_executar_copy = pd.read_excel(file_path, sheet_name='À EXECUTAR (2026 DIANTE)', header=2)

In [ ]:
df_a_executar_copy

In [ ]:
df_a_executar_copy = df_a_executar_copy.drop([0, 1, 2], axis = 0)

In [ ]:
# Renomear Coluna ['EXECUTOR...']
df_a_executar_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR (Grupo Controlador)'}, inplace=True)

df_a_executar_copy.rename(columns={'PAVIMENTAÇÃO': 'PAVIMENTAÇÃO - 2026 - Descrição'}, inplace=True)
df_a_executar_copy.rename(columns={'DUPLICAÇÃO': 'DUPLICAÇÃO - 2026 - Descrição'}, inplace=True)
df_a_executar_copy.rename(columns={'OAE': 'OAE - 2026 - Descrição'}, inplace=True)
df_a_executar_copy.rename(columns={'CONTORNO': 'CONTORNO - 2026 - Descrição'}, inplace=True)
df_a_executar_copy.rename(columns={'FX ADICIONAL': 'FX ADICIONAL - 2026 - Descrição'}, inplace=True)
df_a_executar_copy.rename(columns={'TERCEIRA FAIXA': 'TERCEIRA FAIXA - 2026 - Descrição'}, inplace=True)

In [ ]:
new_column_names = ['PAVIMENTAÇÃO - 2026 - km (i)', 
                    'PAVIMENTAÇÃO - 2026 - km (f)', 
                    'PAVIMENTAÇÃO - 2026 - Ext. (km)', 
                    'PAVIMENTAÇÃO - 2026 - PERCENTUAL (%)', 
                    'PAVIMENTAÇÃO - 2026 - (km)%', 
                    'PAVIMENTAÇÃO - 2026 - FINANCEIRO (R$)',
                    'PAVIMENTAÇÃO - 2026 - Descrição', 
                    'PAVIMENTAÇÃO - Pós 2026 - km (i)', 
                    'PAVIMENTAÇÃO - Pós 2026 - km (f)', 
                    'PAVIMENTAÇÃO - Pós 2026 - Ext. (km)', 
                    'PAVIMENTAÇÃO - Pós 2026 - PERCENTUAL (%)', 
                    'PAVIMENTAÇÃO - Pós 2026 - (km)%', 
                    'PAVIMENTAÇÃO - Pós 2026 - FINANCEIRO (R$)', 
                    'PAVIMENTAÇÃO - Pós 2026 - REL. FÍSICO (km)', 
                    'PAVIMENTAÇÃO - Pós 2026 - REL.FINANCEIRO (R$)',
                    'DUPLICAÇÃO - 2026 - Descrição', 
                    'DUPLICAÇÃO - 2026 - km (i)', 'DUPLICAÇÃO - 2026 - km (f)', 'DUPLICAÇÃO - 2026 - Ext. (km)', 
                    'DUPLICAÇÃO - 2026 - PERCENTUAL (%)', 'DUPLICAÇÃO - 2026 - (km)%', 'DUPLICAÇÃO - 2026 - FINANCEIRO (R$)', 
                    'DUPLICAÇÃO - Pós 2026 - Descrição', 'DUPLICAÇÃO - Pós 2026 - km (i)', 'DUPLICAÇÃO - Pós 2026 - km (f)', 
                    'DUPLICAÇÃO - Pós 2026 - Ext. (km)', 'DUPLICAÇÃO - Pós 2026 - PERCENTUAL (%)', 'DUPLICAÇÃO - Pós 2026 - (km)%', 
                    'DUPLICAÇÃO - Pós 2026 - FINANCEIRO (R$)', 'DUPLICAÇÃO - Pós 2026 - REL. FÍSICO (km)', 'DUPLICAÇÃO - Pós 2026 - REL.FINANCEIRO (R$)',
                    'OAE - 2026 - Descrição', 
                    'OAE - 2026 - km (i)', 'OAE - 2026 - km (f)', 'OAE - 2026 - Ext. (km)', 
                    'OAE - 2026 - PERCENTUAL (%)', 'OAE - 2026 - (km)%', 'OAE - 2026 - FINANCEIRO (R$)', 
                    'OAE - Pós 2026 - Descrição', 'OAE - Pós 2026 - km (i)', 'OAE - Pós 2026 - km (f)', 
                    'OAE - Pós 2026 - Ext. (km)', 'OAE - Pós 2026 - PERCENTUAL (%)', 'OAE - Pós 2026 - (km)%', 
                    'OAE - Pós 2026 - FINANCEIRO (R$)', 'OAE - Pós 2026 - REL. FÍSICO (km)', 'OAE - Pós 2026 - REL.FINANCEIRO (R$)',
                    'CONTORNO - 2026 - Descrição', 
                    'CONTORNO - 2026 - km (i)', 'CONTORNO - 2026 - km (f)', 'CONTORNO - 2026 - Ext. (km)', 
                    'CONTORNO - 2026 - PERCENTUAL (%)', 'CONTORNO - 2026 - (km)%', 'CONTORNO - 2026 - FINANCEIRO (R$)', 
                    'CONTORNO - Pós 2026 - Descrição', 'CONTORNO - Pós 2026 - km (i)', 'CONTORNO - Pós 2026 - km (f)', 
                    'CONTORNO - Pós 2026 - Ext. (km)', 'CONTORNO - Pós 2026 - PERCENTUAL (%)', 'CONTORNO - Pós 2026 - (km)%', 
                    'CONTORNO - Pós 2026 - FINANCEIRO (R$)', 'CONTORNO - Pós 2026 - REL. FÍSICO (km)', 'CONTORNO - Pós 2026 - REL.FINANCEIRO (R$)',
                    'FX ADICIONAL - 2026 - Descrição', 
                    'FX ADICIONAL - 2026 - km (i)', 'FX ADICIONAL - 2026 - km (f)', 'FX ADICIONAL - 2026 - Ext. (km)', 
                    'FX ADICIONAL - 2026 - PERCENTUAL (%)', 'FX ADICIONAL - 2026 - (km)%', 'FX ADICIONAL - 2026 - FINANCEIRO (R$)', 
                    'FX ADICIONAL - Pós 2026 - Descrição', 'FX ADICIONAL - Pós 2026 - km (i)', 'FX ADICIONAL - Pós 2026 - km (f)', 
                    'FX ADICIONAL - Pós 2026 - Ext. (km)', 'FX ADICIONAL - Pós 2026 - PERCENTUAL (%)', 'FX ADICIONAL - Pós 2026 - (km)%', 
                    'FX ADICIONAL - Pós 2026 - FINANCEIRO (R$)', 'FX ADICIONAL - Pós 2026 - REL. FÍSICO (km)', 'FX ADICIONAL - Pós 2026 - REL.FINANCEIRO (R$)',
                    'TERCEIRA FAIXA - 2026 - Descrição', 
                    'TERCEIRA FAIXA - 2026 - km (i)', 'TERCEIRA FAIXA - 2026 - km (f)', 'TERCEIRA FAIXA - 2026 - Ext. (km)', 
                    'TERCEIRA FAIXA - 2026 - PERCENTUAL (%)', 'TERCEIRA FAIXA - 2026 - (km)%', 'TERCEIRA FAIXA - 2026 - FINANCEIRO (R$)', 
                    'TERCEIRA FAIXA - Pós 2026 - Descrição', 'TERCEIRA FAIXA - Pós 2026 - km (i)', 'TERCEIRA FAIXA - Pós 2026 - km (f)', 
                    'TERCEIRA FAIXA - Pós 2026 - Ext. (km)', 'TERCEIRA FAIXA - Pós 2026 - PERCENTUAL (%)', 'TERCEIRA FAIXA - Pós 2026 - (km)%', 
                    'TERCEIRA FAIXA - Pós 2026 - FINANCEIRO (R$)', 'TERCEIRA FAIXA - Pós 2026 - REL. FÍSICO (km)', 'TERCEIRA FAIXA - Pós 2026 - REL.FINANCEIRO (R$)']

In [ ]:
# Criar um dicionário que mapeia 'Unnamed: x' para os novos nomes
rename_dict = {'Unnamed: {}'.format(i): name for i, name in enumerate(new_column_names, start=9)}

# Usar o método rename para renomear as colunas
df_a_executar_copy.rename(columns=rename_dict, inplace=True)

In [ ]:
df_a_executar_copy

In [ ]:
df_a_executar_copy.dropna(subset=['ESTADO/LOTE'])

In [ ]:
columns_to_fill = ['EMPREENDIMENTO']
df_a_executar_copy[columns_to_fill] = df_a_executar_copy.groupby((df_a_executar_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [ ]:
df_a_executar_copy['ESTADO/LOTE'] = df_a_executar_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS',
       'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO',
       'MATO GROSSO', 'PARÁ', 'RIO/SP', 'SÃO PAULO', 'PARANÁ', 'BAHIA',
       'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)

# Agora preenchemos para baixo até encontrar o próximo valor não-NaN (isto é, a próxima 'Situação')
df_a_executar_copy['ESTADO/LOTE'] = df_a_executar_copy['ESTADO/LOTE'].fillna(method='ffill')

In [ ]:
df_a_executar_copy = df_a_executar_copy[~df_a_executar_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]

In [ ]:
df_a_executar_copy.columns

In [ ]:
df_a_executar_copy.index = range(1, len(df_a_executar_copy) + 1)
df_a_executar_copy.index.name = 'ID-ÚNICO'
df_a_executar_copy = df_a_executar_copy.reset_index()

In [ ]:
df_a_executar_raw = df_a_executar_copy

In [ ]:
# df_a_executar_copy.to_excel('Dados Gerados/RAW_À EXECUTAR (2026 DIANTE).xlsx', index=False)

In [ ]:
colunas_manter = ['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO']

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_a_executar_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_a_executar_copy = df_a_executar_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [ ]:
df_a_executar_copy['EMPREENDIMENTO'].unique()

In [ ]:
df_a_executar_copy = df_a_executar_copy.loc[df_a_executar_copy['Valor'] != 0]
df_a_executar_copy = df_a_executar_copy.dropna(subset=['Valor'])

In [ ]:
df_a_executar_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_a_executar_copy['Atributo'].str.split(' - ', expand=True)

In [ ]:
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_a_executar_copy = df_a_executar_copy[['ID-ÚNICO', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [ ]:
df_a_executar_copy['Atributo.1'].unique()

In [ ]:
'''

df_a_executar_copy['Atributo.1'] = df_a_executar_copy['Atributo.1'].apply(
    lambda x: x.replace('CONTORNO ', 'CONTORNO')
              .replace('DUPLICAÇÃO ', 'DUPLICAÇÃO')
              .replace('FX ADICIONAL ', 'FX ADICIONAL')
              .replace('OAE ', 'OAE')
              .replace('PAVIMENTAÇÃO ', 'PAVIMENTAÇÃO')
              .replace('TERCEIRA FAIXA ', 'TERCEIRA FAIXA')
)
'''

In [ ]:
df_a_executar_copy['Atributo.2'].unique()

In [ ]:
# Converter colunas para 'category'
df_a_executar_copy['SETOR'] = df_a_executar_copy['SETOR'].astype('category')
df_a_executar_copy['UF'] = df_a_executar_copy['UF'].astype('category')
df_a_executar_copy['PROPONENTE'] = df_a_executar_copy['PROPONENTE'].astype('category')
df_a_executar_copy['EXECUTOR (Grupo Controlador)'] = df_a_executar_copy['EXECUTOR (Grupo Controlador)'].astype('category')
df_a_executar_copy['ESTRUTURADOR DO PROJETO'] = df_a_executar_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_a_executar_copy['Atributo.1'] = df_a_executar_copy['Atributo.1'].astype('category')
df_a_executar_copy['Atributo.2'] = df_a_executar_copy['Atributo.2'].astype('category')

# Converter colunas para 'string'
df_a_executar_copy['EMPREENDIMENTO'] = df_a_executar_copy['EMPREENDIMENTO'].astype('string') 


In [ ]:
df_a_executar_copy = df_a_executar_copy.sort_values(by='ID-ÚNICO', ascending=True)

In [ ]:
df_a_executar_copy

In [ ]:
df_a_executar_copy.to_excel('Dados Gerados/À EXECUTAR (2026 DIANTE).xlsx', index=False)

<h1><b><i>RISCOS - BI</b></i></h1>


In [ ]:
df_riscos = pd.read_excel('Planilha Monitoramento de Riscos_Modelo_RODOVIÁRIO_20_02_24.xlsx')

In [ ]:
file_path = 'Planilha Monitoramento de Riscos_Modelo_RODOVIÁRIO_20_02_24.xlsx'

In [ ]:
#CHECKPOINT
df_riscos_copy = df_riscos.copy()

In [ ]:
df_riscos_copy = pd.read_excel(file_path, sheet_name='RISCOS AMBIENTAIS', header=2)


In [ ]:
df_riscos_copy.columns

In [ ]:
df_riscos_copy = df_riscos_copy.rename(columns={'Unnamed: 3': 'Bin', 'Unnamed: 11': 'km(i)', 'Unnamed: 12': 'km(f)', 
                                                'Unnamed: 13': 'Ext.(km)', 'Unnamed: 14': 'FINANCEIRO(R$)',
                                                'Unnamed: 17': 'LI', 'Unnamed: 18': 'LO', 'Unnamed: 21': 'SISGLAF', 
                                                'Unnamed: 22': 'SEI', 'Unnamed: 27': 'Data do Atendimento', 
                                                'Unnamed: 28': 'NÍVEL DE ALERTA',
                                                'EXECUTOR             (Grupo Controlador)': 'EXECUTOR\n(Grupo Controlador)',
                                                'LOCALIZAÇÃO': 'Descrição', 
                                                'PER': 'PER\nANO(Execução)', 
                                                'LICENÇAS': 'LP', 
                                                'OBRAS': 'DATA DE INÍCIO', 'PROCESSO': 'SINAFLOR', 
                                                'INTERFERÊNCIAS.1': 'Risco Principal'})

In [ ]:
# Agora, modifique as colunas 'km(i)' e 'km(f)' para incluir o '+'
for col in ['km(i)', 'km(f)']:
    df_riscos_copy[col] = df_riscos_copy[col].astype(str).str.replace('(\d{3})(\d+)', r'\1+\2', regex=True)

In [ ]:
# Colunas que precisam ser preenchidas até encontrar '1' em 'BR'
colunas_para_preencher = ['EMPREENDIMENTO', 'EXECUTOR\n(Grupo Controlador)',
       'DATA DE INÍCIO DA CONCESSÃO', 'ANO DA CONCESSÃO']

# Iterar sobre as linhas do DataFrame
for coluna in colunas_para_preencher:
    valor_atual = None
    for idx, row in df_riscos_copy.iterrows():
        # Checa se 'BR' é '1' ou se a linha da coluna a ser preenchida não está vazia
        if row['BR'] == '1' or pd.notna(row[coluna]):
            valor_atual = row[coluna]
        df_riscos_copy.at[idx, coluna] = valor_atual

In [ ]:
df_riscos_copy = df_riscos_copy.drop([0, 1, 2], axis = 0)
df_riscos_copy = df_riscos_copy.reset_index(drop=True)

In [ ]:
df_riscos_copy = df_riscos_copy.loc[df_riscos_copy['BR'] != 1]

In [ ]:
df_riscos_copy['BR'].unique()

In [ ]:
df_riscos_copy = df_riscos_copy.reset_index(drop=True)

In [ ]:
# Função para limpar e converter as datas para o formato datetime
def limpar_e_converter_data(data):
    # Tratar dados como string para evitar problemas com diferentes tipos
    data = str(data)
    # Substituir '//'' por '/'
    data = data.replace('//', '/')
    # Substituir '/' por '-'
    data = data.replace('/', '-')
    # Tentar converter para datetime
    return pd.to_datetime(data, errors='coerce', dayfirst=True)

# Aplicar a função no DataFrame
# Substitua 'df_riscos_copy' pelo nome do seu DataFrame e 'DATA DE INÍCIO DA CONCESSÃO' pelo nome da coluna que contém as datas
df_riscos_copy['DATA DE INÍCIO DA CONCESSÃO'] = df_riscos_copy['DATA DE INÍCIO DA CONCESSÃO'].apply(limpar_e_converter_data)

In [ ]:
# Converter colunas para 'category'
df_riscos_copy['SETOR'] = df_riscos_copy['SETOR'].astype('object')
df_riscos_copy['UF'] = df_riscos_copy['UF'].astype('category')
df_riscos_copy['INTERFERÊNCIAS'] = df_riscos_copy['INTERFERÊNCIAS'].astype('category')
df_riscos_copy['INTERVENÇÃO AFETADA'] = df_riscos_copy['INTERVENÇÃO AFETADA'].astype('category')
df_riscos_copy['EXECUTOR\n(Grupo Controlador)'] = df_riscos_copy['EXECUTOR\n(Grupo Controlador)'].astype('category')
#df_riscos_copy['PROCESSO'] = df_riscos_copy['PROCESSO'].astype('category')
df_riscos_copy['Risco Principal'] = df_riscos_copy['Risco Principal'].astype('category')
#df_riscos_copy['Licenças'] = df_riscos_copy['Licenças'].astype('category')
#df_riscos_copy['Valor_Licenças'] = df_riscos_copy['Valor_Licenças'].astype('category')
df_riscos_copy['CRITICIDADE'] = df_riscos_copy['CRITICIDADE'].astype('category')
df_riscos_copy['NÍVEL DE ALERTA'] = df_riscos_copy['NÍVEL DE ALERTA'].astype('category')

# Converter colunas para 'int'
df_riscos_copy['FINANCEIRO(R$)'] = df_riscos_copy['FINANCEIRO(R$)'].replace('[\$]', '', regex=True).replace(',', '.', regex=True)
df_riscos_copy['FINANCEIRO(R$)'] = pd.to_numeric(df_riscos_copy['FINANCEIRO(R$)'], errors='coerce')

df_riscos_copy['ANO DA CONCESSÃO'] = pd.to_numeric(df_riscos_copy['ANO DA CONCESSÃO'], errors='coerce').fillna(0).astype(int)

# Converter colunas para 'float'
df_riscos_copy['Ext.(km)'] = pd.to_numeric(df_riscos_copy['Ext.(km)'], errors='coerce')

# Converter colunas para 'string'
df_riscos_copy['EMPREENDIMENTO'] = df_riscos_copy['EMPREENDIMENTO'].astype('string')
#df_riscos_copy['Nº PROCESSO'] = df_riscos_copy['Nº PROCESSO'].astype('string')
df_riscos_copy['Descrição'] = df_riscos_copy['Descrição'].astype('string')
df_riscos_copy['SITUAÇÃO GERAL'] = df_riscos_copy['SITUAÇÃO GERAL'].astype('string')  
df_riscos_copy['COMENTÁRIOS GERAIS'] = df_riscos_copy['COMENTÁRIOS GERAIS'].astype('string')

In [ ]:
df_riscos_copy.columns

In [ ]:
df_riscos_copy = df_riscos_copy[['SETOR', 'UF', 'BR', 'Bin', 'EMPREENDIMENTO',
       'EXECUTOR\n(Grupo Controlador)', 'DATA DE INÍCIO DA CONCESSÃO',
       'ANO DA CONCESSÃO', 'INTERFERÊNCIAS', 'INTERVENÇÃO AFETADA',
       'Descrição', 'km(i)', 'km(f)', 'Ext.(km)', 'FINANCEIRO(R$)',
       'PER\nANO(Execução)', 'LP', 'LI', 'LO', 'DATA DE INÍCIO', 'SINAFLOR',
       'SISGLAF', 'SEI', 'Risco Principal', 'CRITICIDADE', 'SITUAÇÃO GERAL',
       'COMENTÁRIOS GERAIS', 'Data do Atendimento', 'NÍVEL DE ALERTA']]

In [ ]:
#df_riscos_copy.to_excel('Planilha Monitoramento de Riscos_Final.xlsx', index=False)

---

<h1><b><i>ID - RISCO</b></i></h1>
Sheet


In [ ]:
df_id_riscos = df_riscos_copy[df_riscos_copy['Bin'] == 1]

In [ ]:
#df_id_riscos = pd.read_excel('Planilha Monitoramento de Riscos_Final.xlsx')

In [ ]:
(df_id_riscos['Bin'] == 1).sum()

In [ ]:
df_id_riscos = df_id_riscos.reset_index(drop=True)

In [ ]:
df_id_riscos.index = range(1, len(df_id_riscos) + 1)

In [ ]:
df_id_riscos.to_excel('RISCOS - LIMPO/ID - RISCOS.xlsx')

<h3><b><i>'Descrição'</b></i></h3>
Planilha 'Descrição'


In [ ]:
df_desc = pd.read_excel('RISCOS - LIMPO/ID - RISCOS.xlsx', sheet_name='Sheet1',
    usecols=['COMENTÁRIOS GERAIS'])

In [ ]:
df_desc_copy = df_desc.copy()

In [ ]:
df_desc_copy['COMENTÁRIOS GERAIS'].unique()

In [ ]:
# Inicializa as colunas para cada categoria, incluindo uma para "Outras Descrições"
categorias = ['Emitida', 'Em análise', 'Aguardando', 'Solicitada', 'Sem informação', 'Pendência', 'Outras Descrições']
for categoria in categorias:
       df_desc_copy[categoria] = None

# Iterando sobre as linhas do DataFrame
for index, row in df_desc_copy.iterrows():
    texto = row['COMENTÁRIOS GERAIS']
    if pd.isna(texto):  # Se a descrição for NaN, pula para a próxima iteração
        continue

    texto_inicial = texto.split()[0].lower()  # Considera apenas a primeira palavra para a categorização
    
    # Inicialmente classifica como "Outras Descrições"
    categoria = 'Outras Descrições'

    # Verifica o início da descrição para determinar a categoria
    if texto_inicial.startswith('emitida'):
        categoria = 'Emitida'
    elif texto_inicial.startswith('em') and 'análise' in texto.lower():
        categoria = 'Em análise'
    elif texto_inicial.startswith('aguardando'):
        categoria = 'Aguardando'
    elif texto_inicial.startswith('solicitada'):
        categoria = 'Solicitada'
    elif texto_inicial.startswith('sem') and 'informação' in texto.lower():
        categoria = 'Sem informação'
    elif texto_inicial.startswith('pendência'):
        categoria = 'Pendência'
    
    # Atribui a descrição à categoria identificada
    df_desc_copy.at[index, categoria] = texto

In [ ]:
df_desc_copy

In [ ]:
df_desc_copy = df_desc_copy.drop(columns=['COMENTÁRIOS GERAIS'])

In [ ]:
df_desc_copy.to_excel('RISCOS - LIMPO/Descrições - Risco.xlsx', index=False)

<h2><b><i>'ID-RISCO' + 'Descrição'</b></i></h2>
Script


In [ ]:
import pandas as pd

In [ ]:
df_riscos_concat = pd.read_excel('RISCOS - LIMPO/ID - RISCOS.xlsx')
df_desc_concat = pd.read_excel('RISCOS - LIMPO/Descrições - Risco.xlsx')

In [ ]:
df_desc_concat.fillna(0, inplace=True)

In [ ]:
df_final = pd.concat([df_riscos_concat, df_desc_concat], axis=1)

In [ ]:
df_final

In [ ]:
df_final.rename(columns={'Unnamed: 0': 'ID-ÚNICO'}, inplace=True)

In [ ]:
df_final

<h2><b><i>'Risco' + 'Lista Suspensa'</b></i></h2>
Add colunas de Interferência / Risco Principal


In [ ]:
valores_lista_suspensa = [
    "Ausência de licenças/autorizações ambientais para o empreendimento",
    "Ausência de licenças/autorizações ambientais para as áreas de uso",
    "Descumprimento de condicionantes ambientais",
    "Programas ambientais sem execução",
    "Descumprimento de exigências dos órgãos ambientais e intervenientes",
    "Descumprimento de exigências de órgãos de controle",
    "Medidas de proteção e recuperação ambiental previstas no projeto de engenharia não executadas",
    "RNC não solucionados",
    "Ausência de ASV",
    "Interferência com comunidades Quilombolas",
    "Não existe pendência ambiental",
    "OUTROS"
]

# Criando colunas para cada risco na lista suspensa
for risco in valores_lista_suspensa:
    nome_coluna = risco.replace(" ", "_").replace("/", "_").replace("ç", "c").replace("ã", "a").replace("/", "_").replace("(", "").replace(")", "").replace("ó", "o").replace("á", "a").replace("é", "e").replace(".", "")
    df_final[nome_coluna] = ""
    
    # Preenchendo a nova coluna com 'ID-Único' onde o risco corresponde
    df_final.loc[df_final['Risco Principal'] == risco, nome_coluna] = df_final['ID-ÚNICO']

In [ ]:
df_final.to_excel('DADOS - PRONTOS/Riscos - BI.xlsx', index=False, sheet_name= 'RISCOS')

---

<h1><b><i>Coluna FIN e KM novas</b></i></h1>
Sem Duplicadas.


In [ ]:
# Carregar o DataFrame
riscos_km_fin = pd.read_excel('DADOS - PRONTOS/Riscos - BI.xlsx')
riscos_km_fin_copy = riscos_km_fin.copy()

In [ ]:
def remove_duplicated_values(df):
    # Define as colunas para identificar duplicatas
    cols_to_check = ['INTERFERÊNCIAS', 'INTERVENÇÃO AFETADA', 'Descrição', 'km(i)', 'km(f)']
    
    # Marcar duplicatas, mantendo a primeira ocorrência de cada grupo
    duplicates_mask = df.duplicated(subset=cols_to_check, keep='first')
    
    # Criando as novas colunas 'Ext.(km)!' e 'FINANCEIRO(R$)!' onde não houver duplicatas
    df['Ext.(km)!'] = df['Ext.(km)'].where(~duplicates_mask, None)
    df['FINANCEIRO(R$)!'] = df['FINANCEIRO(R$)'].where(~duplicates_mask, None)
    
    return df

In [ ]:
# Aplicar a função ao DataFrame
riscos_km_fin = remove_duplicated_values(riscos_km_fin)

In [ ]:
# Inserir as novas colunas imediatamente após as colunas originais
idx_ext_km = riscos_km_fin.columns.get_loc('Ext.(km)') + 1
idx_fin = riscos_km_fin.columns.get_loc('FINANCEIRO(R$)') + 1

In [ ]:
riscos_km_fin.insert(idx_ext_km, 'Ext.(km)!', riscos_km_fin.pop('Ext.(km)!'))
riscos_km_fin.insert(idx_fin, 'FINANCEIRO(R$)!', riscos_km_fin.pop('FINANCEIRO(R$)!'))

In [ ]:
# Salvar o DataFrame atualizado de volta ao arquivo Excel original
riscos_km_fin.to_excel('RISCOS - LIMPO/Riscos - BI.xlsx', index=False, sheet_name= 'RISCOS')

---

<h1><b><i>Documentação do Processamento dos Dados</i></b></h1>

<p>Este documento descreve o processo de limpeza e transformação dos dados presentes no arquivo Excel <code>Planilha de Monitoramento_PPI_2025_25_03_25_PPI_R18.xlsx</code>, especificamente na planilha "INFORMAÇÕES (1)". O código realiza várias etapas de manipulação de dados utilizando a biblioteca <code>pandas</code> para organizar, limpar e transformar os dados em um formato mais apropriado para análises subsequentes.</p>

<h1><b><i>INFORMAÇÕES (1)</i></b></h1>

<h2><b><i>Etapas do Processo</i></b></h2>

<h3><b><i>1. Leitura do Arquivo Excel</i></b></h3>
<p>O código começa carregando o arquivo Excel e acessando a planilha "INFORMAÇÕES (1)" utilizando o método <code>pd.read_excel()</code>. Uma cópia dos dados é feita para evitar alterações no arquivo original.</p>
<pre><code>df_informacoes_copy = pd.read_excel(file_path, sheet_name='INFORMAÇÕES (1)', header=2)</code></pre>

<h3><b><i>2. Remoção de Linhas Desnecessárias</i></b></h3>
<p>Em seguida, o código remove as três primeiras linhas da planilha que contêm informações irrelevantes, além de eliminar as linhas com valores ausentes na coluna <code>BR</code>.</p>
<pre><code>df_informacoes_copy = df_informacoes_copy.drop([0, 1, 2], axis=0)</code></pre>
<pre><code>df_informacoes_copy = df_informacoes_copy.dropna(subset=['BR'])</code></pre>

<h3><b><i>3. Preenchimento de Colunas Nulas</i></b></h3>
<p>O código preenche as colunas vazias baseadas em um grupo de dados definido pela coluna <code>BR</code>. Isso é feito utilizando a função <code>transform(lambda x: x.ffill())</code> para realizar o preenchimento de valores faltantes (forward fill).</p>
<pre><code>columns_to_fill = ['ESTADO/LOTE', 'EMPREENDIMENTO', 'PROPONENTE', 'EXECUTOR (Grupo Controlador)',
       'ESTRUTURADOR DO PROJETO', 'ANO LEILÃO', 'DATA DE INÍCIO']</code></pre>
<pre><code>df_informacoes_copy[columns_to_fill] = df_informacoes_copy.groupby((df_informacoes_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())</code></pre>

<h3><b><i>4. Criação e Preenchimento de Colunas Derivadas</i></b></h3>
<p>A coluna <code>Situação</code> é criada com base na coluna <code>KM Inicial</code>, sendo preenchida apenas com valores específicos como 'ATIVO', 'CADUCIDADE', 'JUDICIAL', entre outros. O restante dos valores são marcados como <code>NaN</code> e posteriormente preenchidos com o método <code>ffill()</code>.</p>
<pre><code>df_informacoes_copy['Situação'] = df_informacoes_copy['KM Inicial'].apply(lambda x: x if str(x) in ['ATIVO', 'CADUCIDADE', 'JUDICIAL', 'RELICITAÇÃO', 'RENEGOCIAÇÃO'] else pd.NA)</code></pre>
<pre><code>df_informacoes_copy['Situação'] = df_informacoes_copy['Situação'].fillna(method='ffill')</code></pre>

<h3><b><i>5. Tratamento de Dados Numéricos e de Texto</i></b></h3>
<p>O código converte as colunas para o tipo adequado:
<ul>
    <li>Colunas categóricas como <code>SETOR</code>, <code>UF</code>, <code>ESTADO/LOTE</code>, <code>PROPONENTE</code>, etc., são convertidas para o tipo <code>category</code>.</li>
    <li>Colunas numéricas como <code>ANO DA CONCESSÃO</code>, <code>PRAZO (anos)</code> são convertidas para o tipo <code>int</code>.</li>
    <li>Colunas de valores monetários e distâncias como <code>CAPEX (BI)</code>, <code>OPEX (BI)</code>, <code>INVESTIMENTO TOTAL (BI)</code>, <code>km (i)</code>, <code>km (f)</code> e <code>EXTENSÃO (km)</code> são convertidas para o tipo <code>float</code>.</li>
</ul></p>
<pre><code>df_informacoes_copy['ANO DA CONCESSÃO'] = pd.to_numeric(df_informacoes_copy['ANO DA CONCESSÃO'], errors='coerce').fillna(0).astype(int)</code></pre>
<pre><code>df_informacoes_copy['CAPEX (BI)'] = pd.to_numeric(df_informacoes_copy['CAPEX (BI)'], errors='coerce')</code></pre>

<h3><b><i>6. Limpeza de Datas</i></b></h3>
<p>As datas na coluna <code>DATA DE INÍCIO</code> são limpas e convertidas para o formato <code>datetime</code> utilizando a função <code>limpar_e_converter_data()</code>, que substitui caracteres indesejados e converte as datas corretamente.</p>
<pre><code>df_informacoes_copy['DATA DE INÍCIO'] = df_informacoes_copy['DATA DE INÍCIO'].apply(limpar_e_converter_data)</code></pre>

<h3><b><i>7. Filtragem de Dados</i></b></h3>
<p>Após a limpeza, o código filtra os dados para considerar apenas os registros onde o valor da coluna <code>BR</code> seja igual a 1. Em seguida, algumas colunas desnecessárias, como <code>SETOR</code>, <code>Região</code>, <code>Descrição do Trecho</code>, etc., são removidas.</p>
<pre><code>df_informacoes_copy = df_informacoes_copy[df_informacoes_copy['BR'] == 1]</code></pre>
<pre><code>columns_to_drop = ['SETOR', 'Região', 'Descrição do Trecho', 'km (i)', 'km (f)']</code></pre>
<pre><code>df_informacoes_copy = df_informacoes_copy.drop(columns=columns_to_drop)</code></pre>

<h3><b><i>8. Renomeação de Colunas</i></b></h3>
<p>Algumas colunas são renomeadas para facilitar a leitura e padronizar os nomes. O dicionário <code>rename_dict</code> é utilizado para esse propósito.</p>
<pre><code>rename_dict = {'UF': 'Região', 'Etapa': 'ETAPA', 'Situação': 'SITUAÇÃO'}</code></pre>
<pre><code>df_informacoes_copy = df_informacoes_copy.rename(columns=rename_dict)</code></pre>

<h3><b><i>9. Substituição de Valores</i></b></h3>
<p>Na coluna <code>EMPREENDIMENTO</code>, ocorre uma substituição do valor 'CONCESSÃO VIA SUL' para 'VIA SUL'.</p>
<pre><code>df_informacoes_copy['EMPREENDIMENTO'] = df_informacoes_copy['EMPREENDIMENTO'].replace('CONCESSÃO VIA SUL', 'VIA SUL')</code></pre>

<h3><b><i>10. Exportação do Resultado</i></b></h3>
<p>Após todas as transformações e limpezas, o DataFrame resultante é exportado para um novo arquivo Excel.</p>
<pre><code>df_informacoes_copy.to_excel('NEW-SCRIPT - LIMPO/INFORMAÇÕES (1).xlsx', index=False, sheet_name='INFORMAÇÕES (1)')</code></pre>

<h2><b><i>Resumo das Alterações Realizadas</i></b></h2>
<ul>
    <li>Limpeza e formatação de dados.</li>
    <li>Preenchimento de valores ausentes com base em critérios definidos.</li>
    <li>Conversão de colunas para tipos de dados adequados.</li>
    <li>Remoção de colunas desnecessárias.</li>
    <li>Renomeação de colunas para padronização.</li>
    <li>Substituição de valores específicos na coluna <code>EMPREENDIMENTO</code>.</li>
    <li>Exportação dos dados limpos para um novo arquivo Excel.</li>
</ul>
<p>Essas etapas garantem que os dados estejam prontos para análises futuras, com todas as informações organizadas e padronizadas.</p>


---

<h1><b><i>PER (1)</i></b></h1>

<h2><b><i>Processamento dos Dados</i></b></h2>

<p>O código inicia com o carregamento da planilha <code>'PER (1)'</code> para o DataFrame e realiza os seguintes tratamentos:</p>

<h3><b><i>Passos Iniciais de Limpeza e Tratamento</i></b></h3>
<ul>
<li>Criação de uma cópia do DataFrame original <code>df_per_copy</code> e leitura da planilha <code>'PER (1)'</code>.</li>
<li>Remoção das três primeiras linhas (índices 0, 1, 2) que são apenas dados de layout.</li>
<li>Filtragem para excluir linhas onde a coluna <code>'BR'</code> seja igual a 1.</li>
<li>Remoção de linhas com valores nulos nas colunas <code>'BR'</code> e <code>'ESTADO/LOTE'</code>.</li>
<li>Ajuste na coluna <code>'ESTADO/LOTE'</code> para garantir que ela contenha apenas valores válidos (estados específicos), preenchendo os valores faltantes com o método de <code>forward fill</code>.</li>
</ul>
<pre><code>df_per_copy = df_per_copy.drop([0, 1, 2], axis=0)</code></pre>
<pre><code>df_per_copy = df_per_copy.loc[df_per_copy['BR'] != 1]</code></pre>
<pre><code>df_per_copy = df_per_copy.dropna(subset=['BR'])</code></pre>
<pre><code>df_per_copy = df_per_copy.dropna(subset=['ESTADO/LOTE'])</code></pre>

<h3><b><i>2. Renomeação de Colunas</i></b></h3>
<p>As colunas são renomeadas para melhorar a leitura e organizar a estrutura das informações, incluindo a adição de ano na descrição das colunas. Algumas colunas com descrições antigas foram atualizadas para refletir as mudanças nos anos de dados.</p>
<pre><code>df_per_copy.rename(columns={'PAVIMENTAÇÃO': 'PAVIMENTAÇÃO - < 2022 - Descrição'}, inplace=True)</code></pre>
<pre><code>df_per_copy.rename(columns={'DUPLICAÇÃO': 'DUPLICAÇÃO - < 2022 - Descrição'}, inplace=True)</code></pre>
<pre><code>df_per_copy.rename(columns={'OAE': 'OAE - < 2022 - Descrição'}, inplace=True)</code></pre>
<pre><code>df_per_copy.rename(columns={'CONTORNO': 'CONTORNO - < 2022 - Descrição'}, inplace=True)</code></pre>

<h3><b><i>3. Criação de Novas Colunas</i></b></h3>
<p>Foi criada uma lista com novos nomes para as colunas, de acordo com os anos de dados (2022 a 2026) e outros atributos específicos. O dicionário <code>rename_dict</code> foi criado para mapear as colunas existentes para os novos nomes e, em seguida, as renomeações foram aplicadas ao DataFrame.</p>
<pre><code>new_column_names = ['PAVIMENTAÇÃO - < 2022 - km (i)', 'PAVIMENTAÇÃO - < 2022 - km (f)', 'PAVIMENTAÇÃO - < 2022 - Ext. (km)', ...]</code></pre>
<pre><code>rename_dict = {'Unnamed: {}'.format(i): name for i, name in enumerate(new_column_names, start=9)}</code></pre>
<pre><code>df_per_copy.rename(columns=rename_dict, inplace=True)</code></pre>

<h3><b><i>4. Remoção de Colunas Desnecessárias</i></b></h3>
<p>O código também remove algumas colunas desnecessárias para otimizar o DataFrame e evitar que dados irrelevantes sejam considerados nas análises subsequentes. No caso, a coluna <code>Unnamed: 69</code> é identificada e removida:</p>
<pre>
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.iloc[:, :idx]
</pre>

<h3><b><i>5. Filtragem de Linhas Desnecessárias</i></b></h3>
<p>O código também aplica um filtro para remover linhas que contêm as palavras "TOTAL" ou "SOMA", que não são relevantes para a análise detalhada:</p>
<pre>
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy[~df_plan_exec_ano_ant_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>6. Salvar o Arquivo Final</i></b></h3>
<p>Por fim, o DataFrame final, já tratado e reorganizado, é salvo em um novo arquivo Excel:</p>
<pre>
df_plan_exec_ano_ant_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_PLAN_EXEC ATÉ ANO ANTERIOR (22).xlsx', index=False)
</pre>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processo descrito transforma e organiza os dados do arquivo "PLAN_EXEC ATÉ ANO ANTERIOR (22)" para facilitar análises futuras e a integração com outras bases de dados. A utilização de técnicas como <code>fillna</code>, renomeação de colunas e remoção de dados irrelevantes contribui para a qualidade e clareza dos dados manipulados.</p>

---

<h1><b><i>PLAN_EXEC ATÉ ANO ANTERIOR (22) -> (24)**</i></b></h1>

<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "PLAN_EXEC ATÉ ANO ANTERIOR (22)" e sua transformação para o ano de 2024. O processo inclui a leitura do arquivo, limpeza de dados, preenchimento de valores ausentes, renomeação de colunas, e transformação de dados para facilitar a análise e integração com outras bases.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>A primeira etapa do processamento envolve a leitura do arquivo Excel que contém os dados de execução até o ano de 2022. A leitura é feita utilizando a função <code>pd.read_excel</code>, especificando o caminho do arquivo e a planilha de interesse:</p>
<pre>
df_plan_exec_ano_ant_copy = pd.read_excel(file_path, sheet_name='PLAN_EXEC ATÉ ANO ANTERIOR (22)', header=2)
</pre>
<p>O parâmetro <code>header=2</code> é utilizado para pular as primeiras linhas do arquivo que não contêm dados relevantes para análise.</p>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>Após a leitura, o código realiza o processo de limpeza dos dados. A primeira ação é remover as linhas com valores nulos na coluna <code>ESTADO/LOTE</code>, utilizando <code>dropna</code>:</p>
<pre>
df_plan_exec_ano_ant_copy.dropna(subset=['ESTADO/LOTE'])
</pre>
<p>Em seguida, a função <code>groupby</code> é usada para preencher os valores nulos em várias colunas essenciais (como <code>SETOR</code>, <code>UF</code>, <code>BR</code>, etc.) com o último valor válido de cada grupo, utilizando <code>ffill</code>:</p>
<pre>
df_plan_exec_ano_ant_copy[columns_to_fill] = df_plan_exec_ano_ant_copy.groupby((df_plan_exec_ano_ant_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())
</pre>
<p>Essa abordagem preenche os valores ausentes para garantir que as informações sobre o setor, UF, e outros atributos importantes sejam propagadas corretamente.</p>

<h3><b><i>2.3. Filtragem de Estados</i></b></h3>
<p>Para garantir que os dados estejam corretos, o código aplica um filtro para manter apenas as linhas em que a coluna <code>ESTADO/LOTE</code> contenha valores específicos de estados válidos. A função <code>apply</code> é utilizada para verificar se o valor na coluna é um dos estados válidos e, caso contrário, marca o valor como <code>NaN</code>:</p>
<pre>
df_plan_exec_ano_ant_copy['ESTADO/LOTE'] = df_plan_exec_ano_ant_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
</pre>
<p>Após isso, os valores nulos em <code>ESTADO/LOTE</code> são preenchidos para baixo utilizando <code>fillna(method='ffill')</code>, para garantir que o estado de cada linha seja corretamente atribuído:</p>
<pre>
df_plan_exec_ano_ant_copy['ESTADO/LOTE'] = df_plan_exec_ano_ant_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.4. Renomeação de Colunas</i></b></h3>
<p>O próximo passo é renomear várias colunas para torná-las mais descritivas e consistentes. Por exemplo, as colunas de descrição para diferentes tipos de pavimentação, duplicação, e outros tipos de intervenções recebem nomes mais claros e padronizados:</p>
<pre>
df_plan_exec_ano_ant_copy.rename(columns={'PAVIMENTAÇÃO (Acumulada até 2022)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_plan_exec_ano_ant_copy.rename(columns={'DUPLICAÇÃO (Acumulada até 2022)': 'DUPLICAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_plan_exec_ano_ant_copy.rename(columns={'OAE (Acumulada até 2022)': 'OAE - (Acumulada até ano anterior) - Descrição'}, inplace=True)
df_plan_exec_ano_ant_copy.rename(columns={'CONTORNO (Acumulada até 2022)': 'CONTORNO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.5. Transformação de Dados</i></b></h3>
<p>Após a renomeação das colunas, os dados são reorganizados para um formato mais fácil de análise. Para isso, são definidas novas colunas para cada tipo de dado relevante. O código usa um dicionário para mapear as colunas e renomeá-las conforme necessário:</p>
<pre>
rename_dict = {'Unnamed: {}'.format(i): name for i, name in enumerate(new_column_names, start=9)}
df_plan_exec_ano_ant_copy.rename(columns=rename_dict, inplace=True)
</pre>

<h3><b><i>2.6. Remoção de Colunas Desnecessárias</i></b></h3>
<p>O código também remove algumas colunas desnecessárias para otimizar o DataFrame e evitar que dados irrelevantes sejam considerados nas análises subsequentes. No caso, a coluna <code>Unnamed: 69</code> é identificada e removida:</p>
<pre>
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.iloc[:, :idx]
</pre>

<h3><b><i>2.7. Filtragem de Linhas Desnecessárias</i></b></h3>
<p>O código também aplica um filtro para remover linhas que contêm as palavras "TOTAL" ou "SOMA", que não são relevantes para a análise. A função <code>apply</code> com uma expressão regular é utilizada para realizar essa remoção:</p>
<pre>
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy[~df_plan_exec_ano_ant_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.8. Salvar o Arquivo Final</i></b></h3>
<p>Por fim, o DataFrame final, já tratado e reorganizado, é salvo em um novo arquivo Excel para que possa ser utilizado posteriormente em análises:</p>
<pre>
df_plan_exec_ano_ant_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_PLAN_EXEC ATÉ ANO ANTERIOR (22).xlsx', index=False)
</pre>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processo descrito transforma e organiza os dados do arquivo "PLAN_EXEC ATÉ ANO ANTERIOR (22)" para facilitar análises futuras e a integração com outras bases de dados. A utilização de técnicas como <code>fillna</code>, renomeação de colunas e remoção de dados irrelevantes contribui para a qualidade e clareza dos dados manipulados.</p>

---

<h1><b><i>'META(2023)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processamento dos dados relativos ao arquivo "META(2023)", que são lidos, limpos e organizados para análise. O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para melhor entendimento e transformação dos dados para o formato ideal para análise.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido utilizando a função <code>pd.read_excel</code>, e a planilha 'META(1) 2023' é carregada. A leitura é feita a partir da terceira linha (header=2), pois as duas primeiras linhas não contêm dados válidos:</p>
<pre>
df_meta_23_copy = pd.read_excel(file_path, sheet_name='META(1) 2023', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas e colunas desnecessárias para garantir que os dados sejam bem formatados para análise. As linhas 0, 1 e 2 são removidas e as colunas de 'Região' são descartadas:</p>
<pre>
df_meta_23_copy.drop(columns=['Região'], inplace=True)
df_meta_23_copy = df_meta_23_copy.drop(df_meta_23_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas relacionadas a diferentes categorias, como Pavimentação, Duplicação, OAE, Contorno, FX Adicional e Terceira Faixa, são renomeadas para refletir com mais clareza a descrição dos dados:</p>
<pre>
df_meta_23_copy.rename(columns={'PAVIMENTAÇÃO (Ano 2023)': 'PAVIMENTAÇÃO - (Acumulada até ano anterior) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>A função <code>ffill</code> é utilizada para preencher valores ausentes nas colunas de interesse, como 'EMPREENDIMENTO' e 'ESTADO/LOTE'. Também é aplicado um filtro para garantir que apenas estados válidos sejam mantidos:</p>
<pre>
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', 'GOIÁS', 'SANTA CATARINA', 'TOCANTINS', 'GOIAS', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARÁ', 'SÃO PAULO', 'PARANÁ', 'BAHIA', 'ESPIRITO SANTO', 'MATO GROSSO DO SUL'] else pd.NA)
df_meta_23_copy['ESTADO/LOTE'] = df_meta_23_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas que contêm os valores "TOTAL" ou "SOMA" são removidas, pois representam agregações que não são úteis para a análise detalhada:</p>
<pre>
df_meta_23_copy = df_meta_23_copy[~df_meta_23_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>A transformação dos dados em um formato longo é realizada com a função <code>melt</code>, movendo as variáveis de colunas para linhas, o que facilita a análise. Além disso, é realizado um filtro para remover valores nulos e zerados:</p>
<pre>
df_meta_23_copy = df_meta_23_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>Por fim, o DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_meta_23_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_META(2023).xlsx', index=False)
</pre>

<h1><b><i>'META(2024)'</i></b></h1>
<p>A descrição do processo para "META(2024)" segue os mesmos passos descritos para o "META(2023)", com a principal diferença no nome da planilha lida e nas colunas renomeadas para refletir o ano de 2024.</p>

<h1><b><i>'META(2025)'</i></b></h1>
<p>A descrição do processo para "META(2025)" é semelhante à de "META(2024)", com as devidas atualizações no ano e nas colunas.</p>

<h1><b><i>'META(2026)'</i></b></h1>
<p>O processo de "META(2026)" segue os mesmos passos descritos anteriormente, com a atualização do ano e das colunas correspondentes para o ano de 2026.</p>

<h1><b><i>'META(2027)'</i></b></h1>
<p>A documentação do processo para "META(2027)" é idêntica aos anos anteriores, com a atualização para os dados do ano de 2027.</p>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O processamento dos dados para os anos de 2023 a 2027 segue a mesma metodologia, com a leitura dos arquivos, a limpeza, transformação, renomeação de colunas e finalmente a organização dos dados no formato desejado para análise. Esses processos garantem que os dados estejam prontos para serem utilizados em outros fluxos de trabalho e análises futuras.</p>

---

<h1><b><i>'iNEXECUTADO (pendências até 24)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>Esta seção descreve o processo de leitura, limpeza e organização dos dados referentes ao arquivo "iNEXECUTADO (pendências até 24)". O processo inclui a remoção de colunas irrelevantes, renomeação das colunas para refletir corretamente os dados e o tratamento de valores ausentes.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido a partir da planilha "iNEXECUTADO (pendências até 24)", e a leitura é feita a partir da terceira linha (header=2):</p>
<pre>
df_inexec_copy = pd.read_excel(file_path, sheet_name='iNEXECUTADO (pendências até 24)', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>O código realiza a remoção de linhas desnecessárias e a remoção de colunas irrelevantes. As linhas 0, 1 e 2 são removidas e as colunas de "Região" são descartadas:</p>
<pre>
df_inexec_copy.drop(columns=['Região'], inplace=True)
df_inexec_copy = df_inexec_copy.drop(df_inexec_copy.columns[8:176], axis=1)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas que descrevem as diferentes áreas de execução (Pavimentação, Duplicação, OAE, etc.) são renomeadas para garantir clareza e consistência nos dados:</p>
<pre>
df_inexec_copy.rename(columns={'PAVIMENTAÇÃO (até 2024)': 'PAVIMENTAÇÃO - (até 2024) - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>Valores ausentes nas colunas são preenchidos utilizando o método de preenchimento para frente <code>fillna(method='ffill')</code>, garantindo que a informação esteja presente para todas as linhas relevantes:</p>
<pre>
df_inexec_copy['ESTADO/LOTE'] = df_inexec_copy['ESTADO/LOTE'].apply(lambda x: x if str(x) in ['RIO GRANDE DO SUL', 'MINAS GERAIS', ...] else pd.NA)
df_inexec_copy['ESTADO/LOTE'] = df_inexec_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Remoção de Linhas com "TOTAL" ou "SOMA"</i></b></h3>
<p>As linhas contendo "TOTAL" ou "SOMA" são removidas, pois estas representam agregações que não são úteis para a análise detalhada dos dados:</p>
<pre>
df_inexec_copy = df_inexec_copy[~df_inexec_copy.apply(lambda row: row.astype(str).str.contains('TOTAL|SOMA').any(), axis=1)]
</pre>

<h3><b><i>2.6. Unpivot dos Dados</i></b></h3>
<p>O processo de unpivot é realizado para reorganizar as colunas em um formato longo, facilitando a análise. As variáveis são movidas para uma estrutura de linhas:</p>
<pre>
df_inexec_copy = df_inexec_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.7. Salvamento Final</i></b></h3>
<p>O DataFrame final é salvo em um novo arquivo Excel, pronto para ser utilizado para análise:</p>
<pre>
df_inexec_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_iNEXECUTADO (pendências até 24).xlsx', index=False)
</pre>

---

<h1><b><i>'À EXECUTAR (2026 DIANTE)'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>O processo de tratamento dos dados para "À EXECUTAR (2026 DIANTE)" segue uma metodologia similar à de "iNEXECUTADO (pendências até 24)", com o foco na limpeza, renomeação e transformação dos dados de execução para o ano de 2026 em diante.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Leitura do Arquivo</i></b></h3>
<p>O arquivo Excel é lido a partir da planilha "À EXECUTAR (2026 DIANTE)", e a leitura é feita a partir da terceira linha (header=2):</p>
<pre>
df_a_executar_copy = pd.read_excel(file_path, sheet_name='À EXECUTAR (2026 DIANTE)', header=2)
</pre>

<h3><b><i>2.2. Limpeza de Dados</i></b></h3>
<p>As linhas desnecessárias são removidas e as colunas irrelevantes são descartadas. A coluna 'Região' também é descartada:</p>
<pre>
df_a_executar_copy.drop(columns=['Região'], inplace=True)
</pre>

<h3><b><i>2.3. Renomeação de Colunas</i></b></h3>
<p>As colunas que representam diferentes áreas de execução, como Pavimentação, Duplicação, OAE, etc., são renomeadas para refletir a descrição atualizada dos dados:</p>
<pre>
df_a_executar_copy.rename(columns={'PAVIMENTAÇÃO': 'PAVIMENTAÇÃO - 2026 - Descrição'}, inplace=True)
</pre>

<h3><b><i>2.4. Preenchimento de Valores Ausentes</i></b></h3>
<p>Valores ausentes nas colunas são preenchidos de maneira semelhante ao processo anterior para garantir que os dados estejam completos:</p>
<pre>
df_a_executar_copy['ESTADO/LOTE'] = df_a_executar_copy['ESTADO/LOTE'].fillna(method='ffill')
</pre>

<h3><b><i>2.5. Unpivot dos Dados</i></b></h3>
<p>Os dados são transformados para o formato longo utilizando a função <code>melt</code>, movendo as variáveis de colunas para linhas para facilitar a análise:</p>
<pre>
df_a_executar_copy = df_a_executar_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot, var_name='Atributo', value_name='Valor')
</pre>

<h3><b><i>2.6. Salvamento Final</i></b></h3>
<p>O DataFrame processado é salvo em um novo arquivo Excel:</p>
<pre>
df_a_executar_copy.to_excel('NEW-SCRIPT - LIMPO/RAW_À EXECUTAR (2026 DIANTE).xlsx', index=False)
</pre>

<h2><b><i>3. Conclusão</i></b></h2>
<p>Os processos descritos acima são implementados para os arquivos "iNEXECUTADO (pendências até 24)" e "À EXECUTAR (2026 DIANTE)", garantindo que os dados estejam organizados e prontos para análises detalhadas em cada área específica.</p>

---

<h2><b><i>FINAL -> TT - NS</i></b></h2>
<p>Após todo esse processo de transformações e limpeza, os arquivos estão prontos para ser fundidos em uma tabela e planilha mãe para alimentação do Relatório, este processo acontece dentro de outro notebook dentro da pasta NEW-SCRIPT - LIMPO, aonde se econtram todos os arquivos gerados/limpos por este código e se localiza o notebook que acabamos de falar nomeado de "TT - NS" (Todas as Tabelas do New Script).</p>